# Notebook 04 — Data Harmonization: Build `schools_master_v1` (Golden Record)

## Index

### 00. Setup & Environment
- Imports, paths, constants
- Reproducibility settings

### 01. Goals, Scope & Join Strategy
- Notebook scope (Golden Record only)
- Public vs Private school backbone strategy
- ID strategy (NCES, NCESSCH, COMBOKEY)
- Known data limitations & assumptions

### 02. Load Raw Datasets
- NCES CCD (Public Schools)
- NCES PSS (Private Schools)
- CRDC (Civil Rights Data)
- Enrichment Lists
  - CAIS (Independent Schools)
  - AMS Montessori
  - Waldorf
  - IB World Schools
  - (Other curated lists as applicable)

### 03. Build Golden Record Components
#### 03.1 Normalize Identifiers & Core Fields
- Standardize IDs, names, locations
- Fix formatting issues (padding, casing, whitespace)

#### 03.2 Build Public School Backbone (CCD)
- Select canonical public-school fields
- Deduplicate by NCESSCH
- Validate row counts

#### 03.3 Build Private School Backbone (PSS)
- Select canonical private-school fields
- Deduplicate by PSS ID
- Validate row counts

#### 03.4 Normalize & Prepare Enrichment Datasets
- Standardize names, cities, states
- Prepare join keys for public & private schools

#### 03.5 Match Enrichment Tags to Public Backbone
- CAIS → CCD
- IB → CCD
- Apply uniqueness & confidence gates

#### 03.6 Match Enrichment Tags to Private Backbone
- CAIS → PSS
- AMS Montessori → PSS
- Waldorf → PSS
- IB → PSS
- Apply fallback join logic & audits

#### 03.7 Merge Enrichment Tags
- Combine public + private tag tables
- Resolve conflicts & overlaps
- Produce `schools_enrichment_tags_v1`

#### 03.8 Assemble `schools_master_v1`
- Merge backbone with enrichment tags
- Final schema validation

### 04. Data Quality Checks & Audit Reporting
- Join coverage statistics
- Missing ID analysis
- Duplicate detection
- Enrichment hit rates

### 05. Save Outputs & Artifact Manifest
- Save `schools_backbone_v1`
- Save `schools_enrichment_tags_v1`
- Save `schools_master_v1`
- Save audit reports

### 06. Summary & Next Steps
- What this Golden Record enables
- Handoff to Notebook 05 (Matching & Scoring)


## 00. Notebook Setup

This notebook constructs the **full Golden Record school dataset (`schools_master_v1`)** by harmonizing
multiple raw datasets into a single canonical table.

This section defines:
- Global imports and settings
- Input/output artifact contracts
- Helper utilities used throughout the notebook

No dataset-specific assumptions are made here.

In [1253]:
#### 00.1 Imports & Global Settings

# Standard library
from pathlib import Path
from typing import List, Dict, Optional
import json
import hashlib
import re
import unicodedata

# Third-party
import pandas as pd
import numpy as np

# Display settings
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 200)

#### 00.2 Paths & Artifact Contracts

# Base directories
DATA_DIR = Path("../data")
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR_notebook03 = DATA_DIR / "processed/notebook03"
PROCESSED_DIR_notebook04 = DATA_DIR / "processed/notebook04"
REPORTS_DIR = Path("../reports")

# ------------------------
# CCD (Public School Backbone)
# ------------------------
CCD_DIR = RAW_DIR / "ccd"

CCD_DIRECTORY_PATH = CCD_DIR / "ccd_directory.csv"
CCD_MEMBERSHIP_PATH = CCD_DIR / "ccd_membership.csv"
CCD_STAFF_PATH = CCD_DIR / "ccd_staff.csv"
CCD_LUNCH_PATH = CCD_DIR / "ccd_lunch.csv"
CCD_SCHOOL_CHAR_PATH = CCD_DIR / "ccd_school_characteristics.csv"

# ------------------------
# CRDC (Civil Rights Data Collection - School Level)
# ------------------------
CRDC_DIR = RAW_DIR / "crdc"

CRDC_AP_PATH = CRDC_DIR / "Advanced Placement.csv"
CRDC_ALG1_PATH = CRDC_DIR / "Algebra I.csv"
CRDC_ALG2_PATH = CRDC_DIR / "Algebra II.csv"
CRDC_CALC_PATH = CRDC_DIR / "Calculus.csv"
CRDC_CS_PATH = CRDC_DIR / "Computer Science.csv"
CRDC_DS_PATH = CRDC_DIR / "Data Science.csv"
CRDC_EXPULSIONS_PATH = CRDC_DIR / "Expulsions.csv"
CRDC_GIFTED_PATH = CRDC_DIR / "Gifted and Talented.csv"
CRDC_BULLYING_PATH = CRDC_DIR / "Harassment and Bullying.csv"
CRDC_RESTRAINT_PATH = CRDC_DIR / "Restraint and Seclusion.csv"
CRDC_SCHOOL_CHAR_PATH = CRDC_DIR / "School Characteristics.csv"
CRDC_SUPPORT_PATH = CRDC_DIR / "School Support.csv"
CRDC_SUSPENSIONS_PATH = CRDC_DIR / "Suspensions.csv"

# ------------------------
# Enrichment & Private School Lists
# ------------------------
ENRICHMENT_DIR = RAW_DIR / "enrichment"

AMI_MONTESSORI_PATH = ENRICHMENT_DIR / "ami_montessori_bay_area.csv"
AMIUSA_MONTESSORI_PATH = ENRICHMENT_DIR / "amiusa_montessori_bay_area.csv"
AMS_MONTESSORI_PATH = ENRICHMENT_DIR / "ams_bay_area_montessori.csv"

CA_PRIVATE_2425_PATH = ENRICHMENT_DIR / "ca_privateschooldata2425.xlsx"
CAIS_PATH = ENRICHMENT_DIR / "cais_bay_area_schools.csv"

GIFTED_2E_PATH = ENRICHMENT_DIR / "gifted_2e_CA.xlsx"
IB_WORLD_PATH = ENRICHMENT_DIR / "ib_world_schools_california.csv"
PROGRESSIVE_PEN_PATH = ENRICHMENT_DIR / "progressive_schools_pen.csv"

PSS_PRIVATE_PATH = ENRICHMENT_DIR / "pss2122.csv"

WALDORF_ALL_PATH = ENRICHMENT_DIR / "waldorf_all.csv"
WALDORF_BAY_AREA_PATH = ENRICHMENT_DIR / "waldorf_bay_area.csv"

# ------------------------
# Processed Outputs
# ------------------------
BACKBONE_OUT = PROCESSED_DIR_notebook04 / "schools_backbone_v1.parquet"
TAGS_OUT = PROCESSED_DIR_notebook04 / "schools_enrichment_tags_v1.parquet"
MASTER_OUT = PROCESSED_DIR_notebook04 / "schools_master_v1.parquet"

# ------------------------
# Reports
# ------------------------
MERGE_REPORT_PATH = REPORTS_DIR / "notebook04_merge_report.md"
MERGE_AUDIT_PATH = REPORTS_DIR / "notebook04_merge_audit.csv"

#### 00.3 Validate Directory & File Presence

# Ensure output directories exist
PROCESSED_DIR_notebook04.mkdir(parents=True, exist_ok=True)
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

# Validate raw directories
assert CCD_DIR.exists(), f"Missing CCD directory: {CCD_DIR}"
assert CRDC_DIR.exists(), f"Missing CRDC directory: {CRDC_DIR}"
assert ENRICHMENT_DIR.exists(), f"Missing enrichment directory: {ENRICHMENT_DIR}"

# Validate critical raw files (fail fast)
REQUIRED_FILES = [
    # CCD
    CCD_DIRECTORY_PATH,
    CCD_MEMBERSHIP_PATH,
    CCD_STAFF_PATH,
    CCD_LUNCH_PATH,
    CCD_SCHOOL_CHAR_PATH,
    # CRDC (add/remove as your pipeline actually uses)
    CRDC_SCHOOL_CHAR_PATH,
    # Enrichment / Private lists (add/remove as used)
    PSS_PRIVATE_PATH,
    CA_PRIVATE_2425_PATH,
    CAIS_PATH,
    IB_WORLD_PATH,
]
missing_files = [p for p in REQUIRED_FILES if not p.exists()]
assert not missing_files, "Missing required input files:\n- " + "\n- ".join(str(p) for p in missing_files)

#### 00.4 Helper Utilities

def assert_required_columns(df: pd.DataFrame, cols: List[str], name: str):
    """Fail fast if required columns are missing."""
    missing = [c for c in cols if c not in df.columns]
    if missing:
        raise ValueError(f"{name} missing required columns: {missing}")


_STOP_PREFIXES = ("the ",)

def normalize_text(s: Optional[str]) -> Optional[str]:
    """Deterministic normalization for joins (handles punctuation + accents + common prefixes)."""
    if pd.isna(s) or s is None:
        return None

    s = str(s).strip().lower()

    # normalize unicode accents (e.g., José -> Jose)
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))

    # standard replacements
    s = s.replace("&", " and ")

    # remove punctuation (keep alphanumerics + spaces)
    s = re.sub(r"[^a-z0-9\s]", " ", s)

    # collapse whitespace
    s = re.sub(r"\s+", " ", s).strip()

    # drop common leading prefixes (THE ...)
    for p in _STOP_PREFIXES:
        if s.startswith(p):
            s = s[len(p):].strip()

    return s or None


def normalize_state(s: Optional[str]) -> Optional[str]:
    """Normalize US state codes."""
    if pd.isna(s) or s is None:
        return None
    s = str(s).strip().upper()
    return s if len(s) == 2 else s[:2]


def normalize_name_loose(name: Optional[str]) -> Optional[str]:
    n = normalize_text(name)
    if not n:
        return None
    # remove common institution words as whole tokens
    n = re.sub(r"\b(school|academy|prep|preparatory|college)\b", " ", n)
    n = re.sub(r"\s+", " ", n).strip()
    return n or None


def make_join_key(name: str, city: str, state: str) -> str:
    """Create a deterministic join key for cross-dataset matching."""
    parts = [
        normalize_text(name),
        normalize_text(city),
        normalize_state(state),
    ]
    return "|".join([p for p in parts if p])


def make_join_key_loose(name: str, city: str, state: str) -> str:
    """Looser join key (name normalization removes generic institution tokens)."""
    parts = [
        normalize_name_loose(name),
        normalize_text(city),
        normalize_state(state),
    ]
    return "|".join([p for p in parts if p])


def stable_hash(value: str, length: int = 12) -> str:
    """Generate a stable short hash for synthetic IDs."""
    return hashlib.sha256(value.encode("utf-8")).hexdigest()[:length]


#### 00.5 Notebook Constants & Versioning

NOTEBOOK_VERSION = "04"
PIPELINE_STAGE = "data_harmonization"
MASTER_SCHEMA_VERSION = "v1"

#### 00.6 Section Completion Check

print("Notebook 04 — Section 00 setup complete.")
print(f"Pipeline stage: {PIPELINE_STAGE}")
print(f"Schema version: {MASTER_SCHEMA_VERSION}")
print("Outputs will write to:", PROCESSED_DIR_notebook04)

Notebook 04 — Section 00 setup complete.
Pipeline stage: data_harmonization
Schema version: v1
Outputs will write to: ../data/processed/notebook04


## 01. Goals, Scope & Design Principles

This notebook builds the **Golden Record school dataset (`schools_master_v1`)** by harmonizing
multiple heterogeneous data sources into a single canonical table.

This section defines the **intentional constraints** of the system so downstream notebooks
(feature engineering, matching, learning) remain stable and interpretable.

---

### 01.1 Primary Goals

1. **One Row per School**
   - Each physical school location should appear **exactly once** in `schools_master_v1`.

2. **Stable School Identifiers**
   - Public schools must use NCES-derived identifiers.
   - Private schools must use deterministic, reproducible synthetic identifiers.

3. **Backbone + Enrichment Architecture**
   - A minimal, reliable backbone of school identity and location.
   - Optional enrichment tags layered on top without breaking the backbone.

4. **Explainability & Provenance**
   - Every field in `schools_master_v1` must be traceable to one or more source datasets.
   - Merge decisions must be auditable.

5. **Downstream Readiness**
   - The output must be suitable for:
     - Feature engineering (Notebook 05)
     - Matching & ranking models
     - Human-readable inspection

---

### 01.2 Scope (What This Notebook Does)

This notebook **does**:

- Integrate **public school backbone data** (CCD)
- Attach **school-level enrichment data** (CRDC)
- Incorporate **private and specialty school lists** as enrichment tags:
  - CAIS
  - Montessori (AMI / AMS / AMI-USA)
  - Waldorf
  - IB World Schools
  - Progressive Education Network
  - Gifted / 2e
  - WASC / ACSI
  - PSS (Private School Survey)
- Create private-school rows when no public backbone exists
- Produce a single canonical table: `schools_master_v1`

---

### 01.3 Non-Goals (Explicitly Out of Scope)

This notebook **does NOT**:

- Engineer numeric features or vectors
- Perform matching, scoring, or ranking
- Perform statistical modeling or learning
- Resolve subjective conflicts between sources
- Deduplicate across ambiguous fuzzy matches beyond deterministic rules

All modeling and learning occur in **later notebooks**.

---

### 01.4 Golden Record Philosophy: Backbone + Tags

The Golden Record is intentionally split conceptually into two layers:

#### Backbone (Required)
- Stable school identity
- Location (city/state)
- Public-school identifiers (where available)
- Deterministic join keys

The backbone must remain **small, stable, and conservative**.

#### Enrichment Tags (Optional)
- Program participation
- Accreditation or affiliation
- Pedagogical style indicators
- Specialty designations

Enrichment **must never overwrite backbone fields**.

---

### 01.5 Join Strategy & Determinism Guarantees

#### Public Schools
- Primary key: NCES School ID (`NCESSCH`)
- CRDC joins:
  1. Exact NCES ID match (preferred)
  2. Deterministic join key fallback (name + city + state)

#### Private Schools
- No universal ID exists.
- Private schools are identified using:
  - Normalized (school_name, city, state)
  - Deterministic synthetic `school_id` using a stable hash
- No fuzzy joins without explicit audit logging.

---

### 01.6 Data Integrity Guarantees

By the end of this notebook:

- `school_id` is globally unique
- `join_key` is deterministic and reproducible
- Source coverage is explicit:
  - `has_ccd`
  - `has_crdc`
  - `has_enrichment`
- No enrichment tag exists without provenance

---

### 01.7 Known Limitations (Accepted Tradeoffs)

- Some enrichment records may not match any public backbone
- Multiple schools may share similar names within the same city
- Address-level disambiguation is not performed in v1
- CRDC coverage may vary by year and school type

These limitations are **documented, not hidden**, and do not block downstream modeling.

---

### 01.8 Success Criteria (Definition of Done)

This notebook is complete when:

- `schools_master_v1` exists on disk
- All public schools from CCD are represented
- Enrichment tags are attached where deterministically possible
- Private-school orphans are preserved, not dropped
- A merge report exists describing join coverage and limitations

## 02. Load Raw Datasets

This section loads all raw datasets required to construct the Golden Record.
No normalization, joins, or feature engineering occur here.

Goals of this section:
- Load each dataset into memory
- Perform basic shape and existence checks
- Assign clear variable names for downstream sections
- Avoid any irreversible transformations

### 02.1 Load CCD (Public School Backbone)

CCD serves as the primary backbone for public schools.

We load the core tables but do not merge them yet.

In [1126]:
### 02.1 Load CCD (Public School Backbone)

# Helper to load with consistent options + good errors
def read_csv_safe(path: Path, name: str) -> pd.DataFrame:
    assert path.exists(), f"Missing file for {name}: {path}"
    try:
        return pd.read_csv(
            path,
            low_memory=False,
            encoding="cp1252",
            encoding_errors="replace",  # pandas>=2.0; avoids crashing on rare bad bytes
        )
    except TypeError:
        # Fallback for older pandas versions (no encoding_errors arg)
        return pd.read_csv(path, low_memory=False, encoding="cp1252")


ccd_directory_df = read_csv_safe(CCD_DIRECTORY_PATH, "ccd_directory")
ccd_membership_df = read_csv_safe(CCD_MEMBERSHIP_PATH, "ccd_membership")
ccd_staff_df = read_csv_safe(CCD_STAFF_PATH, "ccd_staff")
ccd_lunch_df = read_csv_safe(CCD_LUNCH_PATH, "ccd_lunch")
ccd_school_char_df = read_csv_safe(CCD_SCHOOL_CHAR_PATH, "ccd_school_characteristics")

print("CCD datasets loaded:")
print(f" - directory: {ccd_directory_df.shape}")
print(f" - membership: {ccd_membership_df.shape}")
print(f" - staff: {ccd_staff_df.shape}")
print(f" - lunch: {ccd_lunch_df.shape}")
print(f" - school characteristics: {ccd_school_char_df.shape}")

# ---- Minimal schema expectations (adjust column names if your CCD uses different headers) ----
# The directory file should have the canonical school ID column.
# Most CCD directory extracts use NCESSCH; if yours differs, change here now.
assert_required_columns(ccd_directory_df, ["NCESSCH"], "ccd_directory_df")

# Quick sanity checks (not transformations)
n_dir = len(ccd_directory_df)
n_dir_unique = ccd_directory_df["NCESSCH"].nunique(dropna=True)
print(f"Directory NCESSCH unique: {n_dir_unique:,} / rows: {n_dir:,}")

if n_dir_unique < n_dir:
    dup = n_dir - n_dir_unique
    print(f"WARNING: directory has {dup:,} duplicate NCESSCH rows (expected to dedupe later in 03.2).")

# Peek at column availability for downstream joins (informational only)
print("\nDirectory columns sample:")
print(sorted(list(ccd_directory_df.columns))[:40], "...")


CCD datasets loaded:
 - directory: (102274, 65)
 - membership: (11209338, 18)
 - staff: (100458, 15)
 - lunch: (468060, 17)
 - school characteristics: (100458, 17)
Directory NCESSCH unique: 102,274 / rows: 102,274

Directory columns sample:
['CHARTAUTH1', 'CHARTAUTH2', 'CHARTAUTHN1', 'CHARTAUTHN2', 'CHARTER_TEXT', 'EFFECTIVE_DATE', 'FIPST', 'GSHI', 'GSLO', 'G_10_OFFERED', 'G_11_OFFERED', 'G_12_OFFERED', 'G_13_OFFERED', 'G_1_OFFERED', 'G_2_OFFERED', 'G_3_OFFERED', 'G_4_OFFERED', 'G_5_OFFERED', 'G_6_OFFERED', 'G_7_OFFERED', 'G_8_OFFERED', 'G_9_OFFERED', 'G_AE_OFFERED', 'G_KG_OFFERED', 'G_PK_OFFERED', 'G_UG_OFFERED', 'IGOFFERED', 'LCITY', 'LEAID', 'LEA_NAME', 'LEVEL', 'LSTATE', 'LSTREET1', 'LSTREET2', 'LSTREET3', 'LZIP', 'LZIP4', 'MCITY', 'MSTATE', 'MSTREET1'] ...


### 02.2 Load CRDC (School-Level Enrichment)

CRDC datasets are school-level and may vary in coverage.
We load each file independently and keep them unmerged for now.

In [1128]:
### 02.2 Load CRDC (School-Level Enrichment)

# CRDC datasets are school-level and may vary in coverage.
# We load each file independently and keep them unmerged for now.

CRDC_FILES = {
    "AP": CRDC_AP_PATH,
    "Algebra I": CRDC_ALG1_PATH,
    "Algebra II": CRDC_ALG2_PATH,
    "Calculus": CRDC_CALC_PATH,
    "Computer Science": CRDC_CS_PATH,
    "Data Science": CRDC_DS_PATH,
    "Expulsions": CRDC_EXPULSIONS_PATH,
    "Gifted": CRDC_GIFTED_PATH,
    "Bullying": CRDC_BULLYING_PATH,
    "Restraint": CRDC_RESTRAINT_PATH,
    "School Characteristics": CRDC_SCHOOL_CHAR_PATH,
    "School Support": CRDC_SUPPORT_PATH,
    "Suspensions": CRDC_SUSPENSIONS_PATH,
}

# Load
crdc = {name: read_csv_safe(path, f"crdc_{name}") for name, path in CRDC_FILES.items()}

# Unpack (keeps your original variable names for downstream compatibility)
crdc_ap_df = crdc["AP"]
crdc_alg1_df = crdc["Algebra I"]
crdc_alg2_df = crdc["Algebra II"]
crdc_calc_df = crdc["Calculus"]
crdc_cs_df = crdc["Computer Science"]
crdc_ds_df = crdc["Data Science"]
crdc_expulsions_df = crdc["Expulsions"]
crdc_gifted_df = crdc["Gifted"]
crdc_bullying_df = crdc["Bullying"]
crdc_restraint_df = crdc["Restraint"]
crdc_school_char_df = crdc["School Characteristics"]
crdc_support_df = crdc["School Support"]
crdc_suspensions_df = crdc["Suspensions"]

print("CRDC datasets loaded:")
for name, df in crdc.items():
    print(f" - {name}: {df.shape}")

# ---- Minimal join-key sanity (fail fast if we can't possibly join later) ----
# CRDC often uses one of these identifiers depending on the export/year.
CANDIDATE_ID_COLS = ["NCESSCH", "COMBOKEY", "LEAID", "SCHID", "School ID", "SCH_NAME"]

def pick_first_existing_col(df: pd.DataFrame, candidates: List[str]) -> Optional[str]:
    for c in candidates:
        if c in df.columns:
            return c
    return None

print("\nCRDC join-key diagnostics (informational):")
for name, df in crdc.items():
    id_col = pick_first_existing_col(df, CANDIDATE_ID_COLS)
    if id_col is None:
        raise ValueError(
            f"CRDC file '{name}' has none of the candidate ID columns {CANDIDATE_ID_COLS}. "
            f"Columns sample: {list(df.columns)[:30]}"
        )
    nunique = df[id_col].nunique(dropna=True)
    nrows = len(df)
    print(f" - {name}: id_col='{id_col}', unique_ids={nunique:,}, rows={nrows:,}")

print("\nCRDC load complete.")


CRDC datasets loaded:
 - AP: (98010, 98)
 - Algebra I: (98010, 132)
 - Algebra II: (98010, 29)
 - Calculus: (98010, 29)
 - Computer Science: (98010, 29)
 - Data Science: (98010, 10)
 - Expulsions: (98010, 142)
 - Gifted: (98010, 29)
 - Bullying: (98010, 159)
 - Restraint: (98010, 131)
 - School Characteristics: (98010, 34)
 - School Support: (98010, 19)
 - Suspensions: (98010, 189)

CRDC join-key diagnostics (informational):
 - AP: id_col='COMBOKEY', unique_ids=98,010, rows=98,010
 - Algebra I: id_col='COMBOKEY', unique_ids=98,010, rows=98,010
 - Algebra II: id_col='COMBOKEY', unique_ids=98,010, rows=98,010
 - Calculus: id_col='COMBOKEY', unique_ids=98,010, rows=98,010
 - Computer Science: id_col='COMBOKEY', unique_ids=98,010, rows=98,010
 - Data Science: id_col='COMBOKEY', unique_ids=98,010, rows=98,010
 - Expulsions: id_col='COMBOKEY', unique_ids=98,010, rows=98,010
 - Gifted: id_col='COMBOKEY', unique_ids=98,010, rows=98,010
 - Bullying: id_col='COMBOKEY', unique_ids=98,010, rows=98

### 02.3 Load Enrichment & Private-School Lists

These datasets primarily contribute tags and coverage, not backbone identity.

In [1132]:
### 02.3 Load Enrichment & Private-School Lists

# Note:
# - Most of these datasets contribute enrichment tags.
# - NCES PSS is used as the private-school backbone (identity + location) in later sections.

# --- Excel helper: show sheets + select ---
def read_excel_safe(path: Path, name: str, sheet_name=0) -> pd.DataFrame:
    assert path.exists(), f"Missing file for {name}: {path}"
    return pd.read_excel(path, sheet_name=sheet_name)

def list_excel_sheets(path: Path) -> List[str]:
    xls = pd.ExcelFile(path)
    return list(xls.sheet_names)

# --- Minimal "joinability" checks ---
NAME_CANDIDATES = ["school_name", "school", "name", "School Name", "SCH_NAME", "SCHNAM"]
CITY_CANDIDATES = ["city", "City", "LCITY", "MAIL_CITY", "physical_city"]
STATE_CANDIDATES = ["state", "State", "LSTATE", "MAIL_STATE", "physical_state", "ST"]

def has_any_col(df: pd.DataFrame, candidates: List[str]) -> bool:
    cols = set(df.columns)
    return any(c in cols for c in candidates)

def joinability_diagnostics(df: pd.DataFrame) -> Dict[str, bool]:
    return {
        "has_name": has_any_col(df, NAME_CANDIDATES),
        "has_city": has_any_col(df, CITY_CANDIDATES),
        "has_state": has_any_col(df, STATE_CANDIDATES),
    }

# ---- Load CSV-based enrichment ----
ami_montessori_df = read_csv_safe(AMI_MONTESSORI_PATH, "ami_montessori")
amiusa_montessori_df = read_csv_safe(AMIUSA_MONTESSORI_PATH, "amiusa_montessori")
ams_montessori_df = read_csv_safe(AMS_MONTESSORI_PATH, "ams_montessori")

cais_df = read_csv_safe(CAIS_PATH, "cais")
ib_world_df = read_csv_safe(IB_WORLD_PATH, "ib_world_schools")
progressive_pen_df = read_csv_safe(PROGRESSIVE_PEN_PATH, "progressive_pen")

pss_private_df = read_csv_safe(PSS_PRIVATE_PATH, "pss_private")

waldorf_all_df = read_csv_safe(WALDORF_ALL_PATH, "waldorf_all")
waldorf_bay_area_df = read_csv_safe(WALDORF_BAY_AREA_PATH, "waldorf_bay_area")

# ---- Load Excel-based enrichment ----
# CA Private Directory: print sheet names once (helps prevent silent wrong-sheet loads)
ca_private_sheets = list_excel_sheets(CA_PRIVATE_2425_PATH)
print("CA Private Directory sheets:", ca_private_sheets)

# Default to first sheet for now; if you know the correct sheet name, set sheet_name="..."
ca_private_2425_df = read_excel_safe(CA_PRIVATE_2425_PATH, "ca_private_2425", sheet_name=0)

gifted_sheets = list_excel_sheets(GIFTED_2E_PATH)
print("Gifted/2e sheets:", gifted_sheets)

gifted_2e_df = read_excel_safe(GIFTED_2E_PATH, "gifted_2e", sheet_name=0)

# ---- Summary ----
enrichment_map = {
    "AMI Montessori": ami_montessori_df,
    "AMI-USA Montessori": amiusa_montessori_df,
    "AMS Montessori": ams_montessori_df,
    "CA Private Directory": ca_private_2425_df,
    "CAIS": cais_df,
    "Gifted / 2e": gifted_2e_df,
    "IB World Schools": ib_world_df,
    "Progressive (PEN)": progressive_pen_df,
    "PSS Private": pss_private_df,
    "Waldorf (All)": waldorf_all_df,
    "Waldorf (Bay Area)": waldorf_bay_area_df,
}

print("\nEnrichment datasets loaded:")
for name, df in enrichment_map.items():
    print(f" - {name}: {df.shape}")

print("\nJoinability diagnostics (do we have name/city/state-ish fields?):")
for name, df in enrichment_map.items():
    diag = joinability_diagnostics(df)
    print(f" - {name}: {diag}")

print("\nEnrichment load complete.")


CA Private Directory sheets: ['2024-25 Private School Data']
Gifted/2e sheets: ['Sheet1']

Enrichment datasets loaded:
 - AMI Montessori: (10, 10)
 - AMI-USA Montessori: (1, 12)
 - AMS Montessori: (25, 9)
 - CA Private Directory: (2963, 27)
 - CAIS: (98, 11)
 - Gifted / 2e: (21, 5)
 - IB World Schools: (231, 7)
 - Progressive (PEN): (140, 3)
 - PSS Private: (22345, 459)
 - Waldorf (All): (117, 51)
 - Waldorf (Bay Area): (5, 51)

Joinability diagnostics (do we have name/city/state-ish fields?):
 - AMI Montessori: {'has_name': True, 'has_city': True, 'has_state': True}
 - AMI-USA Montessori: {'has_name': True, 'has_city': True, 'has_state': True}
 - AMS Montessori: {'has_name': True, 'has_city': True, 'has_state': True}
 - CA Private Directory: {'has_name': False, 'has_city': False, 'has_state': False}
 - CAIS: {'has_name': True, 'has_city': True, 'has_state': False}
 - Gifted / 2e: {'has_name': False, 'has_city': True, 'has_state': False}
 - IB World Schools: {'has_name': False, 'has_ci

In [1133]:
def find_cols(df: pd.DataFrame, patterns: List[str], limit: int = 50):
    cols = []
    for c in df.columns:
        cu = str(c).upper()
        if any(p in cu for p in patterns):
            cols.append(c)
    return cols[:limit]

datasets_to_probe = {
    "CA Private Directory": ca_private_2425_df,
    "CAIS": cais_df,
    "Gifted/2e": gifted_2e_df,
    "IB World": ib_world_df,
    "PEN": progressive_pen_df,
    "PSS": pss_private_df,
}

for name, df in datasets_to_probe.items():
    print("\n===", name, "===")
    print("name-ish:", find_cols(df, ["NAME", "SCH", "SCHOOL"]))
    print("city-ish:", find_cols(df, ["CITY", "TOWN"]))
    print("state-ish:", find_cols(df, ["STATE", "ST", "PROVINCE"]))
    print("id-ish:", find_cols(df, ["ID", "NCES", "PSS", "CDS", "COMBO", "KEY"]))



=== CA Private Directory ===
name-ish: ['2024-25 Private School Data for Schools with Six or More Students', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26']
city-ish: []
state-ish: ['2024-25 Private School Data for Schools with Six or More Students']
id-ish: []

=== CAIS ===
name-ish: ['School Name', 'School Type']
city-ish: ['City']
state-ish: ['Membership Status']
id-ish: []

=== Gifted/2e ===
name-ish: ['School_Name']
city-ish: ['City']
state-ish: []
id-ish: []

=== IB World ===
name-ish: ['School_Name']
city-ish: []
state-ish: []
id-ish: []

=== PEN ===
name-ish: ['school_name']
city-ish: []
state-ish: []
id-ish: []

=== PSS ===
name-ish: []
city-ish: ['

In [1136]:
# --- Fix CA Private Directory header detection ---
tmp = pd.read_excel(CA_PRIVATE_2425_PATH, sheet_name=0, header=None)

# Find the first row that looks like a real header
# (contains multiple "column-like" strings such as School, City, County, Zip, CDS)
def row_score(row) -> int:
    vals = [str(x).strip().lower() for x in row.tolist() if pd.notna(x)]
    tokens = " ".join(vals)
    score = 0
    for kw in ["school", "city", "county", "zip", "address", "cds", "district", "phone", "grade"]:
        if kw in tokens:
            score += 1
    return score

scores = tmp.apply(row_score, axis=1)
best_row = int(scores.idxmax())
print("Best header row guess:", best_row, "score:", int(scores.max()))
print("Header row preview:", tmp.iloc[best_row].tolist()[:15])

# Reload using that row as header
ca_private_2425_df = pd.read_excel(CA_PRIVATE_2425_PATH, sheet_name=0, header=best_row)

print("Reloaded CA Private Directory shape:", ca_private_2425_df.shape)
print("Columns sample:", list(ca_private_2425_df.columns)[:25])

# Sanity: do we now see joinable fields?
print("name-ish:", [c for c in ca_private_2425_df.columns if "school" in str(c).lower() or "name" in str(c).lower()][:25])
print("city-ish:", [c for c in ca_private_2425_df.columns if "city" in str(c).lower()][:25])
print("state-ish:", [c for c in ca_private_2425_df.columns if "state" in str(c).lower() or str(c).lower() in ("st", "state")][:25])
print("id-ish:", [c for c in ca_private_2425_df.columns if "cds" in str(c).lower() or "id" in str(c).lower()][:25])


Best header row guess: 5 score: 5
Header row preview: ['CDS Code', 'County', 'Public School District', 'School Name', 'Grade K Enroll', 'Grade 1 Enroll', 'Grade 2 Enroll', 'Grade 3 Enroll', 'Grade 4 Enroll', 'Grade 5 Enroll', 'Grade 6 Enroll', 'Grade 7 Enroll', 'Grade 8 Enroll', 'Grade 9 Enroll', 'Grade 10 Enroll']
Reloaded CA Private Directory shape: (2958, 27)
Columns sample: ['CDS Code', 'County', 'Public School District', 'School Name', 'Grade K Enroll', 'Grade 1 Enroll', 'Grade 2 Enroll', 'Grade 3 Enroll', 'Grade 4 Enroll', 'Grade 5 Enroll', 'Grade 6 Enroll', 'Grade 7 Enroll', 'Grade 8 Enroll', 'Grade 9 Enroll', 'Grade 10 Enroll', 'Grade 11 Enroll', 'Grade 12 Enroll', 'Total Enrollment', 'Previous Year Grads', 'Full Time Teachers', 'Part Time Teachers', 'Administrators', 'Other Staff', 'Tax Exempt 501(c)(3) - Fed', 'Tax Exempt 27301d - CA']
name-ish: ['Public School District', 'School Name']
city-ish: []
state-ish: []
id-ish: ['CDS Code']


## 03. Normalize & Standardize Inputs

This section defines canonical identity fields and deterministic join keys used across datasets.

We will proceed in small steps:
- 03.1 Inspect available columns and choose canonical identity fields
- 03.2 Standardize CCD identity fields + build `join_key`
- 03.3 Standardize CRDC identity fields + build `join_key`
- 03.4 Standardize enrichment identity fields + build `join_key`

### 03.1 Inspect columns (CCD / CRDC / Enrichment)

In [1140]:
def preview_columns(df: pd.DataFrame, name: str, n: int = 40):
    cols = list(df.columns)
    print(f"\n{name}:")
    print(f" - shape: {df.shape}")
    print(f" - columns ({len(cols)} total):")
    print(cols[:n])
    if len(cols) > n:
        print(f"... +{len(cols) - n} more")


# --- CCD: start with directory (most important backbone table) ---
preview_columns(ccd_directory_df, "CCD Directory")

# check other CCD tables briefly (IDs often repeat)
preview_columns(ccd_membership_df, "CCD Membership")
preview_columns(ccd_school_char_df, "CCD School Characteristics")

# --- CRDC: school characteristics tends to carry the ID fields ---
preview_columns(crdc_school_char_df, "CRDC School Characteristics")

# Pick 1-2 more CRDC files to confirm ID column naming consistency
preview_columns(crdc_ap_df, "CRDC Advanced Placement")
preview_columns(crdc_suspensions_df, "CRDC Suspensions")

# --- Enrichment: sample a few representative ones ---
preview_columns(cais_df, "CAIS (Bay Area)")
preview_columns(ams_montessori_df, "AMS Montessori (Bay Area)")
preview_columns(waldorf_all_df, "Waldorf All")
preview_columns(ib_world_df, "IB World Schools (CA)")
preview_columns(pss_private_df, "PSS Private (2021-2022)")


CCD Directory:
 - shape: (102274, 65)
 - columns (65 total):
['SCHOOL_YEAR', 'FIPST', 'STATENAME', 'ST', 'SCH_NAME', 'LEA_NAME', 'STATE_AGENCY_NO', 'UNION', 'ST_LEAID', 'LEAID', 'ST_SCHID', 'NCESSCH', 'SCHID', 'MSTREET1', 'MSTREET2', 'MSTREET3', 'MCITY', 'MSTATE', 'MZIP', 'MZIP4', 'LSTREET1', 'LSTREET2', 'LSTREET3', 'LCITY', 'LSTATE', 'LZIP', 'LZIP4', 'PHONE', 'WEBSITE', 'SY_STATUS', 'SY_STATUS_TEXT', 'UPDATED_STATUS', 'UPDATED_STATUS_TEXT', 'EFFECTIVE_DATE', 'SCH_TYPE_TEXT', 'SCH_TYPE', 'RECON_STATUS', 'OUT_OF_STATE_FLAG', 'CHARTER_TEXT', 'CHARTAUTH1']
... +25 more

CCD Membership:
 - shape: (11209338, 18)
 - columns (18 total):
['SCHOOL_YEAR', 'FIPST', 'STATENAME', 'ST', 'SCH_NAME', 'STATE_AGENCY_NO', 'UNION', 'ST_LEAID', 'LEAID', 'ST_SCHID', 'NCESSCH', 'SCHID', 'GRADE', 'RACE_ETHNICITY', 'SEX', 'STUDENT_COUNT', 'TOTAL_INDICATOR', 'DMS_FLAG']

CCD School Characteristics:
 - shape: (100458, 17)
 - columns (17 total):
['SCHOOL_YEAR', 'FIPST', 'STATENAME', 'ST', 'SCH_NAME', 'STATE_AGEN

**Observation summary (03.1):**
- CCD Directory contains authoritative public-school identifiers (`NCESSCH`)
- CRDC files consistently use `COMBOKEY`
- PSS contains private-school identity fields but no NCES ID
- Enrichment datasets vary widely in schema and require per-source mapping


### 03.2 Standardize CCD Identity Fields

This step prepares the CCD directory as the **public-school backbone**.

Actions:
- Select canonical identity fields
- Normalize text fields
- Construct deterministic join keys
- Assign stable public-school IDs

In [1144]:
### 03.2 Standardize CCD / CRDC / Enrichment Identity Fields (v1)

# Assumes these exist from Section 00:
# - assert_required_columns
# - normalize_text
# - normalize_state
# - make_join_key
# - stable_hash

def _coerce_str(series: pd.Series) -> pd.Series:
    """Coerce to clean string without trailing .0 artifacts."""
    return (
        series.astype(str)
        .str.strip()
        .str.replace(r"\.0$", "", regex=True)
        .replace({"nan": None, "None": None, "": None})
    )

def pick_first_existing_col(df: pd.DataFrame, candidates: List[str], label: str) -> str:
    for c in candidates:
        if c in df.columns:
            return c
    raise ValueError(f"{label}: none of these columns exist: {candidates}")

# ---------------------------------------------------------
# CCD (Public Backbone)
# ---------------------------------------------------------
def standardize_ccd_directory(df: pd.DataFrame) -> pd.DataFrame:
    """
    Transform CCD Directory into the public backbone format.

    Output schema (minimal, v1):
    [school_id, ncessch, school_name, city, state, zip, address,
     join_key, has_ccd]
    """
    assert_required_columns(df, ["NCESSCH"], "CCD Directory")

    # Column selection with fallbacks (CCD exports vary)
    col_name = pick_first_existing_col(df, ["SCHNAM", "SCH_NAME", "SCHOOL_NAME"], "CCD Directory school name")
    col_city = pick_first_existing_col(df, ["LCITY", "MCITY"], "CCD Directory city")
    col_state = pick_first_existing_col(df, ["LSTATE", "ST", "MSTATE"], "CCD Directory state")
    col_zip = pick_first_existing_col(df, ["LZIP", "MZIP"], "CCD Directory zip")
    col_addr = pick_first_existing_col(df, ["LSTREET1", "MSTREET1"], "CCD Directory address")

    out = df[["NCESSCH", col_name, col_city, col_state, col_zip, col_addr]].copy()

    out = out.rename(columns={
        "NCESSCH": "ncessch",
        col_name: "school_name",
        col_city: "city",
        col_state: "state",
        col_zip: "zip",
        col_addr: "address",
    })

    # out["ncessch"] = _coerce_str(out["ncessch"])
    # Normalize NCESSCH to canonical 11-digit school ID
    out["ncessch"] = (
        _coerce_str(out["ncessch"])
            .astype(str)
            .str.zfill(12)   # defensive (handles stray 12-digit values)
            .str[-11:]       # canonical NCES school ID
    )
    out["zip"] = _coerce_str(out["zip"])

    # Normalize identity fields
    for col in ["school_name", "city", "address"]:
        out[col] = out[col].apply(normalize_text)
    out["state"] = out["state"].apply(normalize_state)

    # Deterministic join key (name + city + state)
    out["join_key"] = out.apply(lambda x: make_join_key(x["school_name"], x["city"], x["state"]), axis=1)

    # Public school_id
    out["school_id"] = "PUB_" + out["ncessch"]
    out["has_ccd"] = True

    # De-dupe safety (should already be unique by NCESSCH)
    out = out.drop_duplicates(subset=["school_id"])

    cols_to_keep = ["school_id", "ncessch", "school_name", "city", "state", "zip", "address", "join_key", "has_ccd"]
    return out[cols_to_keep]

# ---------------------------------------------------------
# CRDC (Keys-first)
# ---------------------------------------------------------
def standardize_crdc_keys(df: pd.DataFrame, source_name: str) -> pd.DataFrame:
    """
    Standardize CRDC table for joining.

    Primary key observed in your data: COMBOKEY (unique one row per school).
    Output schema:
    [combokey, crdc_school_name?, crdc_city?, crdc_state?, join_key?, in_crdc, crdc_source]
    """
    assert_required_columns(df, ["COMBOKEY"], f"CRDC - {source_name}")

    out = df[["COMBOKEY"]].copy()
    out = out.rename(columns={"COMBOKEY": "combokey"})
    out["combokey"] = _coerce_str(out["combokey"])

    # Optional fields if present (some CRDC exports include these)
    optional_name = pick_first_existing_col(df, ["SCH_NAME", "SCHOOL_NAME", "School Name"], f"CRDC {source_name} school name") if any(
        c in df.columns for c in ["SCH_NAME", "SCHOOL_NAME", "School Name"]
    ) else None
    optional_city = pick_first_existing_col(df, ["SCH_CITY", "CITY", "City"], f"CRDC {source_name} city") if any(
        c in df.columns for c in ["SCH_CITY", "CITY", "City"]
    ) else None
    optional_state = pick_first_existing_col(df, ["LEA_STATE", "SCH_STATE", "STATE", "State"], f"CRDC {source_name} state") if any(
        c in df.columns for c in ["LEA_STATE", "SCH_STATE", "STATE", "State"]
    ) else None

    if optional_name:
        out["crdc_school_name"] = df[optional_name].apply(normalize_text)
    if optional_city:
        out["crdc_city"] = df[optional_city].apply(normalize_text)
    if optional_state:
        out["crdc_state"] = df[optional_state].apply(normalize_state)

    # Join key only if we have name+city+state
    if set(["crdc_school_name", "crdc_city", "crdc_state"]).issubset(out.columns):
        out["join_key"] = out.apply(lambda x: make_join_key(x["crdc_school_name"], x["crdc_city"], x["crdc_state"]), axis=1)

    out["in_crdc"] = True
    out["crdc_source"] = source_name

    return out.drop_duplicates(subset=["combokey"])

# ---------------------------------------------------------
# Enrichment (Generic)
# ---------------------------------------------------------
def standardize_enrichment(
    df: pd.DataFrame,
    source_name: str,
    mapping: Dict[str, str],
    *,
    default_state: Optional[str] = None
) -> pd.DataFrame:
    """
    mapping maps raw columns -> canonical columns.
    Must map to: school_name, city (state optional if default_state provided)
    """
    out = df.rename(columns=mapping).copy()

    assert_required_columns(out, ["school_name", "city"], f"Enrichment - {source_name}")

    if "state" not in out.columns:
        if default_state is None:
            raise ValueError(f"{source_name}: missing 'state' and no default_state provided.")
        out["state"] = default_state

    out["school_name"] = out["school_name"].apply(normalize_text)
    out["city"] = out["city"].apply(normalize_text)
    out["state"] = out["state"].apply(normalize_state)

    out["join_key"] = out.apply(lambda x: make_join_key(x["school_name"], x["city"], x["state"]), axis=1)
    out["enrichment_source"] = source_name

    base_cols = ["school_name", "city", "state", "join_key", "enrichment_source"]
    extras = [v for v in mapping.values() if v not in base_cols and v in out.columns]
    extras = list(dict.fromkeys(extras))
    return out[base_cols + extras]

# ---------------------------------------------------------
# Smoke tests
# ---------------------------------------------------------
print("Standardizing CCD Directory...")
ccd_clean_df = standardize_ccd_directory(ccd_directory_df)

# Post-condition sanity checks (CCD IDs)
print("CCD NCESSCH length distribution:")
print(ccd_clean_df["ncessch"].astype(str).str.len().value_counts())

assert (ccd_clean_df["ncessch"].astype(str).str.len() == 11).all(), \
    "CCD ncessch must be exactly 11 digits after normalization"

assert ccd_clean_df["school_id"].nunique() == len(ccd_clean_df), \
    "school_id must be globally unique"

print("CCD Clean Shape:", ccd_clean_df.shape)
display(ccd_clean_df.head(3))

print("\nStandardizing CRDC School Characteristics (keys only)...")
crdc_keys_df = standardize_crdc_keys(crdc_school_char_df, "CRDC_School_Characteristics")
print("CRDC Keys Shape:", crdc_keys_df.shape)
display(crdc_keys_df.head(3))

print("\nStandardizing Enrichment Lists (examples)...")

cais_clean_df = standardize_enrichment(
    cais_df,
    "CAIS_Bay_Area",
    mapping={
        "School Name": "school_name",
        "City": "city",
    },
    default_state="CA"
)

ams_clean_df = standardize_enrichment(
    ams_montessori_df,
    "AMS_Montessori_Bay_Area",
    mapping={
        "name": "school_name",
        "city": "city",
        "state": "state",
    }
)

waldorf_clean_df = standardize_enrichment(
    waldorf_all_df,
    "Waldorf_All",
    mapping={
        "name": "school_name",
        "city": "city",
        "state": "state",
    }
)

print("CAIS Clean:", cais_clean_df.shape)
print("AMS Clean:", ams_clean_df.shape)
print("Waldorf Clean:", waldorf_clean_df.shape)
display(cais_clean_df.head(2))


Standardizing CCD Directory...
CCD NCESSCH length distribution:
ncessch
11    102025
Name: count, dtype: int64
CCD Clean Shape: (102025, 9)


,school_id,ncessch,school_name,city,state,zip,address,join_key,has_ccd
0,PUB_10000500870,10000500870,albertville middle school,albertville,AL,35950,600 e alabama ave,albertville middle school|albertville|AL,True
1,PUB_10000500871,10000500871,albertville high school,albertville,AL,35950,402 e mccord ave,albertville high school|albertville|AL,True
2,PUB_10000500879,10000500879,albertville intermediate school,albertville,AL,35950,901 w mckinney ave,albertville intermediate school|albertville|AL,True



Standardizing CRDC School Characteristics (keys only)...
CRDC Keys Shape: (98010, 5)


,combokey,crdc_school_name,crdc_state,in_crdc,crdc_source
0,010000299995,autauga campus,AL,True,CRDC_School_Characteristics
1,010000500870,albertville middle school,AL,True,CRDC_School_Characteristics
2,010000500871,albertville high school,AL,True,CRDC_School_Characteristics



Standardizing Enrichment Lists (examples)...
CAIS Clean: (98, 5)
AMS Clean: (25, 5)
Waldorf Clean: (117, 5)


,school_name,city,state,join_key,enrichment_source
0,academy,berkeley,CA,academy|berkeley|CA,CAIS_Bay_Area
1,almaden country day school,san jose,CA,almaden country day school|san jose|CA,CAIS_Bay_Area


## 03.3 CRDC Validation & Join Coverage (CRDC → CCD via SCHID)

The Civil Rights Data Collection (CRDC) provides critical school-level attributes
(e.g., enrollment by subgroup, discipline, staffing), but does **not** share a
direct primary key with the CCD dataset.

This section validates the integrity of the CRDC → CCD join and documents
coverage, failure modes, and assumptions.

### Join Strategy
- CRDC `COMBOKEY` is used as the primary join field.
- `COMBOKEY` is normalized and mapped to CCD `NCESSCH` via:
  - zero-padding to 12 digits
  - string normalization and type alignment
- Only **school-level** CRDC records are retained (district-level rows excluded).

### Validation Steps
- Count total CRDC school records
- Count successfully matched CCD schools
- Identify unmatched CRDC records
- Verify no one-to-many or many-to-one joins are introduced

### Coverage Metrics
This section reports:
- % of CCD public schools with CRDC coverage
- Absolute counts of:
  - matched schools
  - unmatched CRDC rows
  - CCD schools without CRDC data

### Known Limitations
- CRDC is biennial and may lag CCD school closures or openings
- Some schools may appear in CCD but not CRDC due to eligibility or reporting scope
- CRDC coverage varies by year and state

The validated CRDC join is then used as a trusted enrichment source for
subsequent feature engineering.


In [1147]:
## 03.3 CRDC Validation & Join Coverage (CRDC → CCD via COMBOKEY → NCESSCH)

print("=== CRDC VALIDATION START ===")

# 1) Standardize CRDC keys from the School Characteristics file (best "index" table)
crdc_keys_df = standardize_crdc_keys(crdc_school_char_df, "CRDC_School_Characteristics")
print("CRDC keys shape:", crdc_keys_df.shape)
display(crdc_keys_df.head(5))

# 2) Validate CRDC keys integrity
assert_required_columns(crdc_keys_df, ["combokey", "in_crdc", "crdc_source"], "crdc_keys_df")

print("\nCRDC — Null Summary")
print(f" - combokey: {crdc_keys_df['combokey'].isna().sum()} nulls ({crdc_keys_df['combokey'].isna().mean():.2%})")

crdc_unique = crdc_keys_df["combokey"].nunique(dropna=True)
print(f"\nCRDC rows: {len(crdc_keys_df):,}")
print(f"CRDC unique combokey: {crdc_unique:,}")

if crdc_unique != len(crdc_keys_df):
    print("Duplicate COMBOKEY values found (unexpected). Showing top duplicates:")
    dup = (
        crdc_keys_df.groupby("combokey").size().reset_index(name="count")
        .query("count > 1").sort_values("count", ascending=False).head(20)
    )
    display(dup)

# 3) Derive NCESSCH from COMBOKEY (observed pattern: NCESSCH is last 11 digits of COMBOKEY)
crdc_keys_df = crdc_keys_df.copy()
crdc_keys_df["ncessch_from_combokey"] = crdc_keys_df["combokey"].astype(str).str.zfill(12).str[-11:]
crdc_keys_df["ncessch_from_combokey"] = _coerce_str(crdc_keys_df["ncessch_from_combokey"])

# Quick sanity: show a few examples alongside names (if present)
show_cols = ["combokey", "ncessch_from_combokey"]
if "crdc_school_name" in crdc_keys_df.columns:
    show_cols.append("crdc_school_name")
if "crdc_state" in crdc_keys_df.columns:
    show_cols.append("crdc_state")

print("\nSample COMBOKEY → NCESSCH derivation:")
display(crdc_keys_df[show_cols].head(5))

# 4) Join CRDC → CCD using derived NCESSCH
ccd_map = ccd_clean_df[["school_id", "ncessch", "school_name", "city", "state"]].copy()
ccd_map["ncessch"] = _coerce_str(ccd_map["ncessch"])

merged = crdc_keys_df.merge(
    ccd_map,
    left_on="ncessch_from_combokey",
    right_on="ncessch",
    how="left",
    indicator=True
)

# 5) Coverage metrics
total_crdc = len(merged)
matched_crdc = int((merged["_merge"] == "both").sum())
unmatched_crdc = total_crdc - matched_crdc

print("\n--- CRDC → CCD Join Coverage (by derived NCESSCH) ---")
print(f"CRDC schools (School Characteristics): {total_crdc:,}")
print(f"Matched to CCD: {matched_crdc:,} ({matched_crdc/total_crdc:.2%})")
print(f"Unmatched CRDC: {unmatched_crdc:,} ({unmatched_crdc/total_crdc:.2%})")

ccd_total = len(ccd_clean_df)
ccd_matched = int(ccd_clean_df["ncessch"].isin(crdc_keys_df["ncessch_from_combokey"]).sum())
print("\n--- CCD coverage by CRDC (by derived NCESSCH) ---")
print(f"CCD schools: {ccd_total:,}")
print(f"CCD schools found in CRDC: {ccd_matched:,} ({ccd_matched/ccd_total:.2%})")

# 6) Inspect unmatched CRDC rows (sample)
if unmatched_crdc > 0:
    print("\nSample unmatched CRDC rows (first 10):")
    cols = ["combokey", "ncessch_from_combokey"]
    if "crdc_school_name" in merged.columns:
        cols.append("crdc_school_name")
    if "crdc_state" in merged.columns:
        cols.append("crdc_state")
    display(merged.loc[merged["_merge"] != "both", cols].head(10))

# 7) Create a minimal CRDC presence flag table (for later merge)
crdc_presence_df = merged.loc[merged["_merge"] == "both", ["school_id"]].drop_duplicates()
crdc_presence_df["has_crdc"] = True

print("\nCRDC presence table shape (matched only):", crdc_presence_df.shape)
display(crdc_presence_df.head(5))

print("=== CRDC VALIDATION END ===")


=== CRDC VALIDATION START ===
CRDC keys shape: (98010, 5)


,combokey,crdc_school_name,crdc_state,in_crdc,crdc_source
0,010000299995,autauga campus,AL,True,CRDC_School_Characteristics
1,010000500870,albertville middle school,AL,True,CRDC_School_Characteristics
2,010000500871,albertville high school,AL,True,CRDC_School_Characteristics
3,010000500879,albertville intermediate school,AL,True,CRDC_School_Characteristics
4,010000500889,albertville elementary school,AL,True,CRDC_School_Characteristics



CRDC — Null Summary
 - combokey: 0 nulls (0.00%)

CRDC rows: 98,010
CRDC unique combokey: 98,010

Sample COMBOKEY → NCESSCH derivation:


,combokey,ncessch_from_combokey,crdc_school_name,crdc_state
0,010000299995,10000299995,autauga campus,AL
1,010000500870,10000500870,albertville middle school,AL
2,010000500871,10000500871,albertville high school,AL
3,010000500879,10000500879,albertville intermediate school,AL
4,010000500889,10000500889,albertville elementary school,AL



--- CRDC → CCD Join Coverage (by derived NCESSCH) ---
CRDC schools (School Characteristics): 98,010
Matched to CCD: 94,571 (96.49%)
Unmatched CRDC: 3,439 (3.51%)

--- CCD coverage by CRDC (by derived NCESSCH) ---
CCD schools: 102,025
CCD schools found in CRDC: 94,324 (92.45%)

Sample unmatched CRDC rows (first 10):


,combokey,ncessch_from_combokey,crdc_school_name,crdc_state
0,010000299995,10000299995,autauga campus,AL
68,010003399998,10003399998,allan cott,AL
69,010003399999,10003399999,lakeview,AL
168,010027000549,10027000549,orange beach elementary school,AL
191,010027002483,10027002483,orange beach middlehigh school,AL
205,010033099999,10033099999,bessemer preschool,AL
214,010036099998,10036099998,bibb co preschool,AL
305,010060000259,10060000259,five points elementary school,AL
308,010060000264,10060000264,lafayette lanier elementary school,AL
367,010084099999,10084099999,tennessee valley juvenile detention center,AL



CRDC presence table shape (matched only): (94324, 2)


,school_id,has_crdc
1,PUB_10000500870,True
2,PUB_10000500871,True
3,PUB_10000500879,True
4,PUB_10000500889,True
5,PUB_10000501616,True


=== CRDC VALIDATION END ===


In [1149]:
## 03.3.2 Correct CRDC↔CCD Join Using COMBOKEY[-11:] → NCESSCH

print("=== CRDC 03.3.2 START (JOIN ON COMBOKEY[-11:]) ===")

# 1) Prepare CRDC combokey (12-digit) and derive CCD-style NCESSCH (11-digit)
crdc_df = crdc_school_char_df[["COMBOKEY", "SCH_NAME", "LEA_STATE", "LEAID"]].copy()

crdc_df = crdc_df.rename(columns={
    "COMBOKEY": "crdc_combokey",
    "SCH_NAME": "crdc_school_name_raw",
    "LEA_STATE": "crdc_state_raw",
    "LEAID": "crdc_leaid",
})

crdc_df["crdc_combokey"] = _coerce_str(crdc_df["crdc_combokey"]).astype(str).str.zfill(12)
crdc_df["ncessch_11"] = crdc_df["crdc_combokey"].str[-11:].astype(str).str.zfill(11)

print("CRDC COMBOKEY sample + derived NCESSCH_11:")
display(crdc_df[["crdc_combokey", "ncessch_11", "crdc_school_name_raw", "crdc_state_raw"]].head(10))

# 2) Join CRDC → CCD using CCD clean backbone (NCESSCH)
ccd_map = ccd_clean_df[["school_id", "ncessch"]].copy()
ccd_map["ncessch"] = _coerce_str(ccd_map["ncessch"]).astype(str).str.zfill(11)

joined = crdc_df.merge(
    ccd_map,
    left_on="ncessch_11",
    right_on="ncessch",
    how="left",
    indicator=True
)

total = len(joined)
matched = int((joined["_merge"] == "both").sum())
unmatched = total - matched

print("\n--- CRDC → CCD Join Coverage (COMBOKEY[-11:]) ---")
print(f"CRDC rows: {total:,}")
print(f"Matched: {matched:,} ({matched/total:.2%})")
print(f"Unmatched: {unmatched:,} ({unmatched/total:.2%})")

# 3) Presence table keyed by school_id
crdc_presence_df = joined.loc[joined["_merge"] == "both", ["school_id"]].drop_duplicates()
crdc_presence_df["has_crdc"] = True

print("\nCRDC presence table shape:", crdc_presence_df.shape)
display(crdc_presence_df.head(5))

# 4) Inspect unmatched rows
if unmatched > 0:
    print("\nSample unmatched CRDC rows (first 10):")
    display(
        joined.loc[joined["_merge"] != "both",
                   ["crdc_combokey", "ncessch_11", "crdc_state_raw", "crdc_leaid", "crdc_school_name_raw"]]
        .head(10)
    )

print("Saved crdc_presence_df for downstream merges.")
print("=== CRDC 03.3.2 END ===")


=== CRDC 03.3.2 START (JOIN ON COMBOKEY[-11:]) ===
CRDC COMBOKEY sample + derived NCESSCH_11:


,crdc_combokey,ncessch_11,crdc_school_name_raw,crdc_state_raw
0,010000299995,10000299995,AUTAUGA CAMPUS,AL
1,010000500870,10000500870,Albertville Middle School,AL
2,010000500871,10000500871,Albertville High School,AL
3,010000500879,10000500879,Albertville Intermediate School,AL
4,010000500889,10000500889,Albertville Elementary School,AL
5,010000501616,10000501616,Albertville Kindergarten and PreK,AL
6,010000502150,10000502150,Albertville Primary School,AL
7,010000600193,10000600193,Kate Duncan Smith DAR Middle,AL
8,010000600872,10000600872,Asbury High School,AL
9,010000600877,10000600877,Douglas Elementary School,AL



--- CRDC → CCD Join Coverage (COMBOKEY[-11:]) ---
CRDC rows: 98,010
Matched: 94,571 (96.49%)
Unmatched: 3,439 (3.51%)

CRDC presence table shape: (94324, 2)


,school_id,has_crdc
1,PUB_10000500870,True
2,PUB_10000500871,True
3,PUB_10000500879,True
4,PUB_10000500889,True
5,PUB_10000501616,True



Sample unmatched CRDC rows (first 10):


,crdc_combokey,ncessch_11,crdc_state_raw,crdc_leaid,crdc_school_name_raw
0,010000299995,10000299995,AL,0100002,AUTAUGA CAMPUS
68,010003399998,10003399998,AL,0100033,Allan Cott
69,010003399999,10003399999,AL,0100033,Lakeview
168,010027000549,10027000549,AL,0100270,Orange Beach Elementary School
191,010027002483,10027002483,AL,0100270,Orange Beach MiddleHigh School
205,010033099999,10033099999,AL,0100330,BESSEMER PRESCHOOL
214,010036099998,10036099998,AL,0100360,BIBB CO PRESCHOOL
305,010060000259,10060000259,AL,0100600,Five Points Elementary School
308,010060000264,10060000264,AL,0100600,Lafayette Lanier Elementary School
367,010084099999,10084099999,AL,0100840,Tennessee Valley Juvenile Detention Center


Saved crdc_presence_df for downstream merges.
=== CRDC 03.3.2 END ===


### 03.3.3 — Why NCESSCH Canonicalization Was Required (Important)

During initial CRDC ↔ CCD joins, we observed a low match rate even when schools clearly
existed in both datasets. Investigation revealed an **identifier formatting mismatch**.

#### Root Cause
- CRDC provides `COMBOKEY` as a **12-digit string**.
- Our CCD `NCESSCH` field contained **mixed-length identifiers** (some 11 digits, some 12 digits),
  due to upstream parsing/formatting differences.
- In practice, the CCD school identifier we need is the **canonical 11-digit NCES school ID**,
  which corresponds to the **last 11 digits** of CRDC `COMBOKEY`.

Example:
- CRDC `COMBOKEY`: `010000500870`
- Canonical CCD `NCESSCH` (11-digit): `10000500870`

#### Resolution
We standardized identifiers deterministically:
- CCD: `ncessch = str(NCESSCH).zfill(12)[-11:]`  (canonical 11-digit)
- CRDC: `ncessch_from_combokey = str(COMBOKEY).zfill(12)[-11:]`

All CRDC→CCD joins now use the canonical 11-digit `ncessch`.

#### Result
- CRDC → CCD join match rate: **~96.5%**
- Remaining unmatched records (~3.5%) are expected due to:
  - coverage differences
  - closed/placeholder institutions
  - reporting artifacts

This normalization step is critical for reliable cross-dataset integration.


## 03.4 Enrichment Validation — CAIS (Match to CCD via `join_key`)

The California Association of Independent Schools (CAIS) dataset is a
manually curated list of independent schools and does not provide
official NCES identifiers.

This section documents how CAIS schools are matched to the public
school backbone (CCD), along with validation steps and known limitations.

### Join Strategy
- CAIS records are normalized to generate a `join_key`:
  - school name (normalized)
  - city (normalized)
  - state
- The `join_key` is matched against normalized CCD school identifiers.
- Only **high-confidence one-to-one matches** are retained.

### Validation Steps
- Count total CAIS schools
- Count successfully matched CCD schools
- Verify uniqueness:
  - no CAIS school matches multiple CCD schools
  - no CCD school receives multiple CAIS flags
- Manually inspect ambiguous or dropped matches

### Coverage Notes
- CAIS primarily represents **private independent schools**
- Public-school matches are rare and expected to be low
- Coverage is geographically concentrated (California)

### Known Limitations
- Name variations and campus naming conventions may prevent some matches
- CAIS membership changes over time and is not exhaustive
- This dataset is treated as a **high-precision, low-recall signal**

Validated CAIS matches are applied as a binary enrichment flag and
do not override backbone identifiers.


In [1153]:
## 03.4 Enrichment Validation — CAIS (Match to CCD via join_key)

print("=== 03.4 CAIS ENRICHMENT VALIDATION START ===")

# ----------------------------
# Helper: guard against overly-generic school names (prevents false matches like "academy")
# ----------------------------
def is_name_too_generic(s: Optional[str]) -> bool:
    if not s:
        return True
    toks = str(s).split()
    if len(toks) < 2:
        return True
    if len(str(s)) < 8:
        return True
    return False


# 1) Standardize CAIS enrichment (Bay Area list → default_state="CA")
cais_clean_df = standardize_enrichment(
    cais_df,
    "CAIS_Bay_Area",
    mapping={
        "School Name": "school_name",
        "City": "city",
        "Physical Address": "address",
        "Website": "website",
        "Detail URL": "detail_url",
        "Membership Status": "membership_status",
        "School Type": "school_type",
    },
    default_state="CA"
)

print("CAIS clean shape (raw standardized):", cais_clean_df.shape)
display(cais_clean_df.head(5))

# 1b) Filter out generic/unsafe names BEFORE matching
before = len(cais_clean_df)
cais_clean_df = cais_clean_df[~cais_clean_df["school_name"].apply(is_name_too_generic)].copy()
after = len(cais_clean_df)
print(f"\nFiltered generic CAIS names: removed {before-after} rows (kept {after})")

# 2) Validate required columns + nulls
required_cols = ["school_name", "city", "state", "join_key", "enrichment_source"]
assert_required_columns(cais_clean_df, required_cols, "cais_clean_df")

print("\nCAIS — Null Summary")
for c in ["school_name", "city", "state", "join_key"]:
    print(f" - {c}: {cais_clean_df[c].isna().sum()} nulls ({cais_clean_df[c].isna().mean():.2%})")

# 3) Match CAIS to CCD by join_key
ccd_join_keys = ccd_clean_df[["school_id", "join_key", "school_name", "city", "state"]].copy()

cais_to_ccd = cais_clean_df.merge(
    ccd_join_keys,
    on="join_key",
    how="left",
    indicator=True,
    suffixes=("_cais", "_ccd")
)

total = cais_to_ccd.shape[0]
matched = int((cais_to_ccd["_merge"] == "both").sum())

print("\n--- CAIS → CCD Match Coverage (join_key) ---")
print(f"CAIS rows: {total}")
print(f"Matched to CCD: {matched} ({matched/total:.2%})")
print(f"Unmatched: {total-matched} ({(total-matched)/total:.2%})")

# 3b) Uniqueness / one-to-one diagnostics (informational; expected to be tiny)
# A) Does a single CAIS join_key map to multiple CCD schools?
multi_ccd_for_cais = (
    cais_to_ccd.loc[cais_to_ccd["_merge"] == "both"]
    .groupby("join_key")["school_id"]
    .nunique()
    .reset_index(name="ccd_matches")
    .query("ccd_matches > 1")
)

if len(multi_ccd_for_cais) > 0:
    print("\nWarning: Some CAIS join_keys matched multiple CCD schools (unexpected). Showing examples:")
    display(multi_ccd_for_cais.head(10))

# B) Does a single CCD school receive multiple CAIS rows?
multi_cais_for_ccd = (
    cais_to_ccd.loc[cais_to_ccd["_merge"] == "both"]
    .groupby("school_id")["join_key"]
    .nunique()
    .reset_index(name="cais_matches")
    .query("cais_matches > 1")
)

if len(multi_cais_for_ccd) > 0:
    print("\nWarning: Some CCD schools matched multiple CAIS rows (unexpected). Showing examples:")
    display(multi_cais_for_ccd.head(10))

# 4) Build CAIS presence flag table for later master build
# NOTE: CAIS is mostly private; matching to CCD (public) will be near 0.
cais_presence_df = (
    cais_to_ccd.loc[cais_to_ccd["_merge"] == "both", ["school_id"]]
    .drop_duplicates()
    .copy()
)
cais_presence_df["has_cais"] = True

print("\nCAIS presence table shape:", cais_presence_df.shape)
display(cais_presence_df.head(10))

# 5) Inspect unmatched CAIS rows (use CAIS-side columns after suffixing)
candidate_cols = [
    "school_name_cais", "city_cais", "state_cais", "join_key",
    "membership_status", "school_type",
    "detail_url", "website", "address"
]
candidate_cols = [c for c in candidate_cols if c in cais_to_ccd.columns]

unmatched_cais = cais_to_ccd.loc[cais_to_ccd["_merge"] != "both", candidate_cols].copy()

print("\nSample unmatched CAIS rows (first 15):")
display(unmatched_cais.head(15))

# 6) Interpretation
print("\nNote: CCD Directory is public-only; CAIS schools are mostly private.")
print("      A 0% or near-0% match rate here is expected. Next we validate CAIS against PSS (private backbone).")

print("=== 03.4 CAIS ENRICHMENT VALIDATION END ===")


=== 03.4 CAIS ENRICHMENT VALIDATION START ===
CAIS clean shape (raw standardized): (98, 10)


,school_name,city,state,join_key,enrichment_source,address,website,detail_url,membership_status,school_type
0,academy,berkeley,CA,academy|berkeley|CA,CAIS_Bay_Area,"2722 Benvenue Avenue\n Berkeley, CA 947...",https://www.theacademyschool.org,https://www.caisca.org/schools/the-academy,Full Member,Elementary
1,almaden country day school,san jose,CA,almaden country day school|san jose|CA,CAIS_Bay_Area,"6835 Trinidad Drive\n San JosÃ©, CA 951...",https://www.almadencountrydayschool.org,https://www.caisca.org/schools/almaden-country...,Full Member,Elementary
2,alta vista school,san francisco,CA,alta vista school|san francisco|CA,CAIS_Bay_Area,"450 Somerset Street\n San Francisco, CA...",https://www.altavistaschool.org,https://www.caisca.org/schools/alta-vista-school,Full Member,Elementary
3,athenian school,danville,CA,athenian school|danville|CA,CAIS_Bay_Area,2100 Mount Diablo Scenic Boulevard\n Da...,https://www.athenian.org,https://www.caisca.org/schools/the-athenian-sc...,Full Member,Secondary
4,bay school of san francisco,san francisco,CA,bay school of san francisco|san francisco|CA,CAIS_Bay_Area,35 Keyes Avenue The Presidio\n San Fran...,https://www.bayschoolsf.org,https://www.caisca.org/schools/the-bay-school-...,Full Member,Secondary



Filtered generic CAIS names: removed 1 rows (kept 97)

CAIS — Null Summary
 - school_name: 0 nulls (0.00%)
 - city: 0 nulls (0.00%)
 - state: 0 nulls (0.00%)
 - join_key: 0 nulls (0.00%)

--- CAIS → CCD Match Coverage (join_key) ---
CAIS rows: 97
Matched to CCD: 0 (0.00%)
Unmatched: 97 (100.00%)

CAIS presence table shape: (0, 2)


,school_id,has_cais



Sample unmatched CAIS rows (first 15):


,school_name_cais,city_cais,state_cais,join_key,membership_status,school_type,detail_url,website,address
0,almaden country day school,san jose,CA,almaden country day school|san jose|CA,Full Member,Elementary,https://www.caisca.org/schools/almaden-country...,https://www.almadencountrydayschool.org,"6835 Trinidad Drive\n San JosÃ©, CA 951..."
1,alta vista school,san francisco,CA,alta vista school|san francisco|CA,Full Member,Elementary,https://www.caisca.org/schools/alta-vista-school,https://www.altavistaschool.org,"450 Somerset Street\n San Francisco, CA..."
2,athenian school,danville,CA,athenian school|danville|CA,Full Member,Secondary,https://www.caisca.org/schools/the-athenian-sc...,https://www.athenian.org,2100 Mount Diablo Scenic Boulevard\n Da...
3,bay school of san francisco,san francisco,CA,bay school of san francisco|san francisco|CA,Full Member,Secondary,https://www.caisca.org/schools/the-bay-school-...,https://www.bayschoolsf.org,35 Keyes Avenue The Presidio\n San Fran...
4,bayhill high school,berkeley,CA,bayhill high school|berkeley|CA,Provisional,Secondary,https://www.caisca.org/schools/bayhill-high-sc...,https://www.bayhillhs.org,"1940 Virginia Street\n Berkeley, CA 947..."
5,bentley school,lafayette oakland,CA,bentley school|lafayette oakland|CA,Full Member,K-12,https://www.caisca.org/schools/bentley-school,https://www.bentleyschool.org,"1 Hiller Drive\n Oakland, CA 94618 View..."
6,berkeley school,berkeley,CA,berkeley school|berkeley|CA,Full Member,Elementary,https://www.caisca.org/schools/the-berkeley-sc...,https://www.theberkeleyschool.org,"1310 University Avenue\n Berkeley, CA 9..."
7,berkwood hedge school,berkeley,CA,berkwood hedge school|berkeley|CA,Full Member,Elementary,https://www.caisca.org/schools/berkwood-hedge-...,https://www.berkwood.org,"1809 Bancroft Way\n Berkeley, CA 94703 ..."
8,black pine circle school,berkeley,CA,black pine circle school|berkeley|CA,Full Member,Elementary,https://www.caisca.org/schools/black-pine-circ...,https://www.blackpinecircle.org,"2027 7th Street\n Berkeley, CA 94710 Vi..."
9,blue oak school,napa,CA,blue oak school|napa|CA,Full Member,Elementary,https://www.caisca.org/schools/blue-oak-school,https://www.blueoakschool.org,"1436 Polk Street\n Napa, CA 94559 View ..."



Note: CCD Directory is public-only; CAIS schools are mostly private.
      A 0% or near-0% match rate here is expected. Next we validate CAIS against PSS (private backbone).
=== 03.4 CAIS ENRICHMENT VALIDATION END ===


### Interpretation: Why CAIS → CCD Match is 0%

This 0% match rate is expected.

- **CCD Directory** is the NCES dataset for **public schools**.
- **CAIS** is a directory of **independent (private) schools**.

Therefore, CAIS schools generally will not exist in the CCD public backbone.  
The correct validation target for CAIS is the **private backbone**, especially:
- **NCES PSS (Private School Survey)** and/or
- the **CA private school directory** dataset.

Next, we validate CAIS against PSS to confirm we can attach CAIS tags to private schools.

In [1156]:
## 03.4.2a PSS Column Discovery (find name/city/state/id fields)

print("=== 03.4.2a PSS COLUMN DISCOVERY START ===")

# Use the already-loaded dataframe from 02.3 to avoid inconsistencies
pss_df = pss_private_df

print("PSS_PRIVATE_PATH:", PSS_PRIVATE_PATH)
print("PSS shape:", pss_df.shape)

# show a slice of columns
print("\nPSS columns (first 120):")
print(pss_df.columns.tolist()[:120])

# keyword search
keywords = ["pname", "name", "school", "pstreet", "street", "addr", "address", "pcity", "city", "pstabb", "state", "zip", "nces", "id"]
hits = {}
for kw in keywords:
    hits[kw] = [c for c in pss_df.columns if kw.lower() in c.lower()]

print("\nCandidate columns by keyword:")
for kw in keywords:
    cols = hits[kw]
    if cols:
        print(f"\n[{kw}] ({len(cols)} hits):")
        print(cols[:40])  # print up to 40

# strongly-likely PSS identity fields (common in NCES PSS extracts)
likely_cols = []
for c in [
    "PSSID", "PSSSCH_ID", "SCHID", "ID",                   # possible ids
    "PNAME", "NAME", "SCH_NAME", "SCHOOL_NAME",           # name
    "PCITY", "CITY",                                      # city
    "PSTABB", "PSTATE", "STATE", "ST", "PL_STABB",        # state
    "PZIP", "ZIP", "ZIPCODE",                             # zip
    "PSTREET", "PSTREET1", "PADDR", "ADDRESS", "STREET"   # address
]:
    if c in pss_df.columns:
        likely_cols.append(c)

if likely_cols:
    print("\nPreview likely identity columns found:")
    print(likely_cols)
    display(pss_df[likely_cols].head(8))
else:
    print("\nNo obvious standard identity column names found directly. Use keyword hits above to select columns.")

# Quick best-guess picks (non-fatal, informational only)
def pick_first_existing(candidates):
    for c in candidates:
        if c in pss_df.columns:
            return c
    return None

best_name  = pick_first_existing(["PNAME", "SCH_NAME", "SCHOOL_NAME", "NAME"])
best_city  = pick_first_existing(["PCITY", "CITY"])
best_state = pick_first_existing(["PSTABB", "PSTATE", "STATE", "ST", "PL_STABB"])
best_id    = pick_first_existing(["PSSID", "PSSSCH_ID", "SCHID", "ID"])

print("\nBest-guess identity columns (informational):")
print(" - id   :", best_id)
print(" - name :", best_name)
print(" - city :", best_city)
print(" - state:", best_state)

print("=== 03.4.2a END ===")


=== 03.4.2a PSS COLUMN DISCOVERY START ===
PSS_PRIVATE_PATH: ../data/raw/enrichment/pss2122.csv
PSS shape: (22345, 459)

PSS columns (first 120):
['PFNLWT', 'REPW1', 'REPW2', 'REPW3', 'REPW4', 'REPW5', 'REPW6', 'REPW7', 'REPW8', 'REPW9', 'REPW10', 'REPW11', 'REPW12', 'REPW13', 'REPW14', 'REPW15', 'REPW16', 'REPW17', 'REPW18', 'REPW19', 'REPW20', 'REPW21', 'REPW22', 'REPW23', 'REPW24', 'REPW25', 'REPW26', 'REPW27', 'REPW28', 'REPW29', 'REPW30', 'REPW31', 'REPW32', 'REPW33', 'REPW34', 'REPW35', 'REPW36', 'REPW37', 'REPW38', 'REPW39', 'REPW40', 'REPW41', 'REPW42', 'REPW43', 'REPW44', 'REPW45', 'REPW46', 'REPW47', 'REPW48', 'REPW49', 'REPW50', 'REPW51', 'REPW52', 'REPW53', 'REPW54', 'REPW55', 'REPW56', 'REPW57', 'REPW58', 'REPW59', 'REPW60', 'REPW61', 'REPW62', 'REPW63', 'REPW64', 'REPW65', 'REPW66', 'REPW67', 'REPW68', 'REPW69', 'REPW70', 'REPW71', 'REPW72', 'REPW73', 'REPW74', 'REPW75', 'REPW76', 'REPW77', 'REPW78', 'REPW79', 'REPW80', 'REPW81', 'REPW82', 'REPW83', 'REPW84', 'REPW85', 'R

,PCITY,PSTABB,PL_STABB,PZIP
0,GADSDEN,AL,AL,35901
1,TUSCALOOSA,AL,NaN,35405
2,HUNTSVILLE,AL,NaN,35816
3,HUNTSVILLE,AL,NaN,35802
4,BIRMINGHAM,AL,NaN,35209
5,DECATUR,AL,NaN,35603
6,CULLMAN,AL,NaN,35055
7,BIRMINGHAM,AL,NaN,35205



Best-guess identity columns (informational):
 - id   : None
 - name : None
 - city : PCITY
 - state: PSTABB
=== 03.4.2a END ===


In [1158]:
## 03.4.2b PSS Deep Column Discovery (find school_name + state by heuristics)

print("=== 03.4.2b PSS DEEP COLUMN DISCOVERY START ===")

# Always use the already-loaded df from 02.3
pss_df = pss_private_df

# 1) List object/text columns (likely to include names, city, address)
obj_cols = pss_df.select_dtypes(include=["object"]).columns.tolist()
print(f"PSS shape: {pss_df.shape}")
print(f"Object/text columns: {len(obj_cols)}")
print("First 60 object columns:")
print(obj_cols[:60])

# 2) Show samples for the first 12 object columns (quick human scan)
sample_n = 5
print("\nSample values for first 12 object columns:")
for c in obj_cols[:12]:
    vals = pss_df[c].dropna().astype(str).head(sample_n).tolist()
    print(f"\n- {c}:")
    print(vals)

# 3) Heuristic: find state-like columns (mostly 2-letter codes)
import re

def state_score(series: pd.Series, n: int = 800) -> float:
    s = series.dropna().astype(str).str.strip().head(n)
    if len(s) == 0:
        return 0.0
    m = s.str.fullmatch(r"[A-Z]{2}")
    return float(m.mean())

state_candidates = []
for c in pss_df.columns:
    try:
        score = state_score(pss_df[c])
        if score > 0.50:  # strong signal
            state_candidates.append((c, score))
    except Exception:
        continue

state_candidates = sorted(state_candidates, key=lambda x: x[1], reverse=True)
print("\nTop STATE-like candidates (score > 0.50):")
state_cand_df = pd.DataFrame(state_candidates, columns=["column", "state_score"]).head(30)
display(state_cand_df)

STATE_COL = state_candidates[0][0] if len(state_candidates) else None
print("\nSelected STATE_COL:", STATE_COL)

# 4) Heuristic: find school-name-like columns
school_words = re.compile(
    r"\b(school|academy|montessori|prep|preparatory|institute|christian|catholic|elementary|middle|high)\b",
    re.I
)

def name_score(series: pd.Series, n: int = 800) -> float:
    s = series.dropna().astype(str).str.strip().head(n)
    if len(s) == 0:
        return 0.0
    has_word = s.apply(lambda x: bool(school_words.search(x)))
    avg_len = s.str.len().mean()
    # space presence is usually a good sign for names
    space_rate = s.str.contains(r"\s").mean()
    # penalize code-like columns
    digit_rate = s.str.contains(r"\d").mean()
    return float(has_word.mean()) * (min(avg_len, 80) / 80) + (0.25 * space_rate) - (0.5 * digit_rate)

name_candidates = []
for c in obj_cols:
    try:
        score = name_score(pss_df[c])
        if score > 0.10:  # weak-to-strong signal
            name_candidates.append((c, score))
    except Exception:
        continue

name_candidates = sorted(name_candidates, key=lambda x: x[1], reverse=True)
print("\nTop SCHOOL-NAME-like candidates (score > 0.10):")
name_cand_df = pd.DataFrame(name_candidates, columns=["column", "name_score"]).head(30)
display(name_cand_df)

NAME_COL = name_candidates[0][0] if len(name_candidates) else None
print("\nSelected NAME_COL:", NAME_COL)

# 5) Confirm known fields we already saw
print("\nKnown address/location fields found so far:")
for c in ["PCITY", "PADDRS", "PZIP", "PZIP4", "PL_ZIP", "PL_ZIP4", "PSTABB", "PL_STABB"]:
    if c in pss_df.columns:
        print(" -", c)

CITY_COL = "PCITY" if "PCITY" in pss_df.columns else None
ZIP_COL = "PZIP" if "PZIP" in pss_df.columns else None
ADDR_COL = "PADDRS" if "PADDRS" in pss_df.columns else None

# 6) Show a preview of the chosen identity columns (so we can proceed immediately)
if NAME_COL and CITY_COL and STATE_COL:
    preview_cols = [NAME_COL, CITY_COL, STATE_COL]
    if ZIP_COL:
        preview_cols.append(ZIP_COL)
    if ADDR_COL:
        preview_cols.append(ADDR_COL)

    print("\nPreview selected identity columns:")
    display(pss_df[preview_cols].head(12))

    print("\nREADY FOR NEXT CELL — use these columns:")
    print(f" - NAME_COL  = {NAME_COL}")
    print(f" - CITY_COL  = {CITY_COL}")
    print(f" - STATE_COL = {STATE_COL}")
    print(f" - ZIP_COL   = {ZIP_COL}")
    print(f" - ADDR_COL  = {ADDR_COL}")
else:
    print("\nCould not confidently select NAME_COL/CITY_COL/STATE_COL. Inspect candidate tables above.")

print("=== 03.4.2b PSS DEEP COLUMN DISCOVERY END ===")


=== 03.4.2b PSS DEEP COLUMN DISCOVERY START ===
PSS shape: (22345, 459)
Object/text columns: 12
First 60 object columns:
['PPIN', 'PINST', 'PADDRS', 'PCITY', 'PSTABB', 'PCNTNM', 'PL_ADD', 'PL_CIT', 'PL_STABB', 'SLDLST22', 'SLDUST22', 'FRAME']

Sample values for first 12 object columns:

- PPIN:
['00000033', '00000044', '00000055', '00000077', '00000088']

- PINST:
['ST JAMES CATHOLIC SCHOOL', 'HOLY SPIRIT CATHOLIC SCHOOL', 'HOLY FAMILY PAROCHIAL SCHOOL', 'HOLY SPIRIT REGIONAL CATHOLIC SCHOOL', 'OUR LADY OF SORROWS']

- PADDRS:
['700 ALBERT RAINS BLVD', '601 JAMES I HARRISON JR PKWY E', '2300 BEASLEY AVE NW', '619 AIRPORT RD SW', '1720 OXMOOR RD']

- PCITY:
['GADSDEN', 'TUSCALOOSA', 'HUNTSVILLE', 'HUNTSVILLE', 'BIRMINGHAM']

- PSTABB:
['AL', 'AL', 'AL', 'AL', 'AL']

- PCNTNM:
['ETOWAH', 'TUSCALOOSA', 'MADISON', 'MADISON', 'JEFFERSON']

- PL_ADD:
['511 EWING AVE', '1832 CENTER WAY S', '1503 MAIN ST', '215 S BROAD ST', '258 ML TILLIS DR']

- PL_CIT:
['GADSDEN', 'BIRMINGHAM', 'DAPHNE', 'LO

,column,state_score
0,PSTABB,1.0
1,PL_STABB,1.0



Selected STATE_COL: PSTABB

Top SCHOOL-NAME-like candidates (score > 0.10):


,column,name_score
0,PINST,0.552156
1,PCNTNM,0.135937
2,PCITY,0.105625



Selected NAME_COL: PINST

Known address/location fields found so far:
 - PCITY
 - PADDRS
 - PZIP
 - PZIP4
 - PL_ZIP
 - PL_ZIP4
 - PSTABB
 - PL_STABB

Preview selected identity columns:


,PINST,PCITY,PSTABB,PZIP,PADDRS
0,ST JAMES CATHOLIC SCHOOL,GADSDEN,AL,35901,700 ALBERT RAINS BLVD
1,HOLY SPIRIT CATHOLIC SCHOOL,TUSCALOOSA,AL,35405,601 JAMES I HARRISON JR PKWY E
2,HOLY FAMILY PAROCHIAL SCHOOL,HUNTSVILLE,AL,35816,2300 BEASLEY AVE NW
3,HOLY SPIRIT REGIONAL CATHOLIC SCHOOL,HUNTSVILLE,AL,35802,619 AIRPORT RD SW
4,OUR LADY OF SORROWS,BIRMINGHAM,AL,35209,1720 OXMOOR RD
5,ST ANN SCHOOL,DECATUR,AL,35603,3910A SPRING AVE SW
6,SACRED HEART ELEMENTARY,CULLMAN,AL,35055,112 2ND AVE SE
7,ST ROSE OF LIMA SCHOOL,BIRMINGHAM,AL,35205,1401 22ND ST S
8,ST FRANCIS XAVIER SCHOOL,BIRMINGHAM,AL,35213,2 XAVIER CIR
9,JOHN CARROLL CATHOLIC HIGH SCHOOL,BIRMINGHAM,AL,35209,300 LAKESHORE PKWY



READY FOR NEXT CELL — use these columns:
 - NAME_COL  = PINST
 - CITY_COL  = PCITY
 - STATE_COL = PSTABB
 - ZIP_COL   = PZIP
 - ADDR_COL  = PADDRS
=== 03.4.2b PSS DEEP COLUMN DISCOVERY END ===


In [968]:
## 03.4.2c Standardize PSS Identity Fields + Build Join Keys (strict + loose)

print("=== 03.4.2c PSS STANDARDIZATION START ===")

# Use discovered columns from 03.4.2b
NAME_COL  = "PINST"
CITY_COL  = "PCITY"
STATE_COL = "PSTABB"
ZIP_COL   = "PZIP"
ADDR_COL  = "PADDRS"

pss_df = pss_private_df

# 1) Minimal identity table
pss_clean_df = pss_df[[NAME_COL, CITY_COL, STATE_COL, ZIP_COL, ADDR_COL]].copy()
pss_clean_df = pss_clean_df.rename(columns={
    NAME_COL: "school_name",
    CITY_COL: "city",
    STATE_COL: "state",
    ZIP_COL: "zip",
    ADDR_COL: "address",
})

# 2) Normalize fields
pss_clean_df["school_name"] = pss_clean_df["school_name"].apply(normalize_text)
pss_clean_df["city"] = pss_clean_df["city"].apply(normalize_text)
pss_clean_df["state"] = pss_clean_df["state"].apply(normalize_state)
pss_clean_df["zip"] = _coerce_str(pss_clean_df["zip"])
pss_clean_df["address"] = pss_clean_df["address"].apply(normalize_text)

# 3) Join keys
pss_clean_df["join_key"] = pss_clean_df.apply(
    lambda x: make_join_key(x["school_name"], x["city"], x["state"]),
    axis=1
)
pss_clean_df["join_key_loose"] = pss_clean_df.apply(
    lambda x: make_join_key_loose(x["school_name"], x["city"], x["state"]),
    axis=1
)

print("PSS clean shape:", pss_clean_df.shape)
display(pss_clean_df.head(10))

# 4) Key uniqueness diagnostics (important for safe enrichment joins)
strict_unique = pss_clean_df["join_key"].nunique(dropna=True)
loose_unique  = pss_clean_df["join_key_loose"].nunique(dropna=True)

print("\n--- PSS key uniqueness ---")
print(f"Rows: {len(pss_clean_df):,}")
print(f"Unique join_key (strict): {strict_unique:,}")
print(f"Unique join_key_loose:    {loose_unique:,}")

# How many keys are duplicated? (these are ambiguous joins)
dup_strict = (
    pss_clean_df.groupby("join_key").size().reset_index(name="count")
    .query("join_key.notna() and count > 1")
    .sort_values("count", ascending=False)
)
dup_loose = (
    pss_clean_df.groupby("join_key_loose").size().reset_index(name="count")
    .query("join_key_loose.notna() and count > 1")
    .sort_values("count", ascending=False)
)

print("\nDuplicate strict join_keys:", len(dup_strict))
print("Duplicate loose join_keys:", len(dup_loose))

print("\nTop duplicated strict join_keys (first 10):")
display(dup_strict.head(10))

# 5) De-duped key tables for fast membership checks
pss_keys_strict = pss_clean_df[["join_key"]].dropna().drop_duplicates()
pss_keys_loose  = pss_clean_df[["join_key_loose"]].dropna().drop_duplicates()

print("\nKey tables:")
print(" - pss_keys_strict:", pss_keys_strict.shape)
print(" - pss_keys_loose :", pss_keys_loose.shape)

print("=== 03.4.2c PSS STANDARDIZATION END ===")


=== 03.4.2c PSS STANDARDIZATION START ===
PSS clean shape: (22345, 7)


,school_name,city,state,zip,address,join_key,join_key_loose
0,st james catholic school,gadsden,AL,35901,700 albert rains blvd,st james catholic school|gadsden|AL,st james catholic|gadsden|AL
1,holy spirit catholic school,tuscaloosa,AL,35405,601 james i harrison jr pkwy e,holy spirit catholic school|tuscaloosa|AL,holy spirit catholic|tuscaloosa|AL
2,holy family parochial school,huntsville,AL,35816,2300 beasley ave nw,holy family parochial school|huntsville|AL,holy family parochial|huntsville|AL
3,holy spirit regional catholic school,huntsville,AL,35802,619 airport rd sw,holy spirit regional catholic school|huntsvill...,holy spirit regional catholic|huntsville|AL
4,our lady of sorrows,birmingham,AL,35209,1720 oxmoor rd,our lady of sorrows|birmingham|AL,our lady of sorrows|birmingham|AL
5,st ann school,decatur,AL,35603,3910a spring ave sw,st ann school|decatur|AL,st ann|decatur|AL
6,sacred heart elementary,cullman,AL,35055,112 2nd ave se,sacred heart elementary|cullman|AL,sacred heart elementary|cullman|AL
7,st rose of lima school,birmingham,AL,35205,1401 22nd st s,st rose of lima school|birmingham|AL,st rose of lima|birmingham|AL
8,st francis xavier school,birmingham,AL,35213,2 xavier cir,st francis xavier school|birmingham|AL,st francis xavier|birmingham|AL
9,john carroll catholic high school,birmingham,AL,35209,300 lakeshore pkwy,john carroll catholic high school|birmingham|AL,john carroll catholic high|birmingham|AL



--- PSS key uniqueness ---
Rows: 22,345
Unique join_key (strict): 22,002
Unique join_key_loose:    21,970

Duplicate strict join_keys: 338
Duplicate loose join_keys: 370

Top duplicated strict join_keys (first 10):


,join_key,count
8320,icn noor academy|naperville|IL,3
8209,horizon christian academy|cumming|GA,3
12738,notre dame school of milwaukee|milwaukee|WI,3
8830,jewish middle school of nashville|nashville|TN,3
5815,first lutheran school|fort smith|AR,3
9,3 oaks academy|fort myers|FL,2
13834,pine school|hobe sound|FL,2
14098,presbyterian christian school|hattiesburg|MS,2
14065,prairie school|berne|IN,2
14034,post oak school|bellaire|TX,2



Key tables:
 - pss_keys_strict: (22002, 1)
 - pss_keys_loose : (21970, 1)
=== 03.4.2c PSS STANDARDIZATION END ===


## 03.5 Standardize PSS (Private School Backbone)

The NCES Private School Survey (PSS) serves as the canonical backbone for
private and independent schools in this project.

Unlike public schools, private schools do not have a universally stable
school identifier, making normalization and deduplication critical.

### Standardization Steps
- Normalize school names (case, punctuation, whitespace)
- Normalize city and state fields
- Standardize grade and age range representations
- Ensure consistent data types across numeric and categorical fields

### Deduplication Strategy
- Deduplicate records using a composite key of:
  - normalized school name
  - city
  - state
- Retain a single canonical record per private school

### Backbone Guarantees
The standardized PSS backbone ensures:
- one row per private school
- stable join keys for enrichment datasets
- consistent schema with the public-school backbone where applicable

### Known Limitations
- Multi-campus private schools may appear as separate records
- Naming variations across years may require conservative deduplication
- PSS data reflects periodic surveys and may lag real-world changes

This standardized private-school backbone is used for all subsequent
private-school enrichment joins.


In [1161]:
## 03.5 Standardize PSS (Private School Backbone)

print("=== 03.5 PSS STANDARDIZATION START ===")

pss_df = pss_private_df  # single source of truth

def standardize_pss(df: pd.DataFrame) -> pd.DataFrame:
    """
    Transform PSS (Private School Survey) into standardized private identity rows.

    Uses discovered PSS columns:
      - PPIN   : private school ID
      - PINST  : school name
      - PCITY  : city
      - PSTABB : state abbrev
      - PADDRS : address
      - PZIP   : zip
      - PZIP4  : zip4
    """
    out = df.copy()

    mapping = {
        "PPIN": "ppin",
        "PINST": "school_name",
        "PCITY": "city",
        "PSTABB": "state",
        "PADDRS": "address",
        "PZIP": "zip",
        "PZIP4": "zip4",
    }
    out = out.rename(columns=mapping)

    assert_required_columns(out, ["ppin", "school_name", "city", "state"], "PSS")

    # IDs
    out["ppin"] = _coerce_str(out["ppin"])
    out["school_id"] = "PRI_" + out["ppin"]

    # Normalize identity fields (DO NOT fillna("") — keep None)
    out["school_name"] = out["school_name"].apply(normalize_text)
    out["city"] = out["city"].apply(normalize_text)
    out["state"] = out["state"].apply(normalize_state)
    out["address"] = out["address"].apply(normalize_text) if "address" in out.columns else None
    out["zip"] = _coerce_str(out["zip"]) if "zip" in out.columns else None
    out["zip4"] = _coerce_str(out["zip4"]) if "zip4" in out.columns else None

    # Join keys
    out["join_key"] = out.apply(lambda x: make_join_key(x["school_name"], x["city"], x["state"]), axis=1)
    out["join_key_loose"] = out.apply(lambda x: make_join_key_loose(x["school_name"], x["city"], x["state"]), axis=1)

    # Provenance
    out["backbone_source"] = "PSS_2122"
    out["has_pss"] = True

    cols = [
        "school_id", "ppin", "school_name", "city", "state",
        "zip", "zip4", "address",
        "join_key", "join_key_loose",
        "backbone_source", "has_pss"
    ]
    cols = [c for c in cols if c in out.columns]
    return out[cols]

print(f"Raw PSS Shape: {pss_df.shape}")

pss_clean_df = standardize_pss(pss_df)

print(f"PSS Clean Shape (row-level): {pss_clean_df.shape}")
display(pss_clean_df.head(5))

# Row-id uniqueness (should be 0)
dupes_rowid = pss_clean_df["school_id"].duplicated().sum()
print(f"\nDuplicate school_id (PRI_PPIN) in PSS: {dupes_rowid}")

# --- Dedup backbone view (one row per join_key) ---
# Conservative rule: keep the first occurrence per join_key.
# (We keep ppin-based school_id for determinism, but only one record per join_key in backbone.)
pss_backbone_df = (
    pss_clean_df.dropna(subset=["join_key"])
    .sort_values(["state", "city", "school_name", "ppin"])
    .drop_duplicates(subset=["join_key"], keep="first")
    .reset_index(drop=True)
)

print("\nPSS Backbone Shape (deduped by join_key):", pss_backbone_df.shape)

# Diagnostics: how many duplicates were collapsed?
collapsed = len(pss_clean_df.dropna(subset=["join_key"])) - len(pss_backbone_df)
print("Collapsed rows due to duplicate join_key:", collapsed)

# California counts (use backbone view for “one row per private school”)
ca_count_rows = (pss_clean_df["state"] == "CA").sum()
ca_count_backbone = (pss_backbone_df["state"] == "CA").sum()
print(f"\nCalifornia rows in PSS (row-level): {ca_count_rows}")
print(f"California private schools in backbone (deduped): {ca_count_backbone}")

print("=== 03.5 PSS STANDARDIZATION END ===")


=== 03.5 PSS STANDARDIZATION START ===
Raw PSS Shape: (22345, 459)
PSS Clean Shape (row-level): (22345, 12)


,school_id,ppin,school_name,city,state,zip,zip4,address,join_key,join_key_loose,backbone_source,has_pss
0,PRI_00000033,00000033,st james catholic school,gadsden,AL,35901,2564,700 albert rains blvd,st james catholic school|gadsden|AL,st james catholic|gadsden|AL,PSS_2122,True
1,PRI_00000044,00000044,holy spirit catholic school,tuscaloosa,AL,35405,3208,601 james i harrison jr pkwy e,holy spirit catholic school|tuscaloosa|AL,holy spirit catholic|tuscaloosa|AL,PSS_2122,True
2,PRI_00000055,00000055,holy family parochial school,huntsville,AL,35816,4004,2300 beasley ave nw,holy family parochial school|huntsville|AL,holy family parochial|huntsville|AL,PSS_2122,True
3,PRI_00000077,00000077,holy spirit regional catholic school,huntsville,AL,35802,1310,619 airport rd sw,holy spirit regional catholic school|huntsvill...,holy spirit regional catholic|huntsville|AL,PSS_2122,True
4,PRI_00000088,00000088,our lady of sorrows,birmingham,AL,35209,4097,1720 oxmoor rd,our lady of sorrows|birmingham|AL,our lady of sorrows|birmingham|AL,PSS_2122,True



Duplicate school_id (PRI_PPIN) in PSS: 0

PSS Backbone Shape (deduped by join_key): (22002, 12)
Collapsed rows due to duplicate join_key: 343

California rows in PSS (row-level): 2452
California private schools in backbone (deduped): 2410
=== 03.5 PSS STANDARDIZATION END ===


## 03.5.1 CAIS City Normalization & Multi-City Explode (Deterministic)  ✅ [Bucket A]

### Why this exists
Our CAIS Bay Area list includes schools whose **City** field is not a single city. Common patterns:

- multiple campuses in one string (e.g., `"menlo park palo alto"`, `"east palo alto san francisco"`)
- separators like commas, slashes, ampersands, or newlines
- “dirty” formatting that prevents deterministic joins

Because our matching is **exact and deterministic**, a multi-city CAIS row often fails to match PSS even when the school exists (e.g., CAIS `"menlo park palo alto"` vs PSS `"palo alto"`).

### Goal
Create a **deterministic, auditable** transformation that:

1. Normalizes CAIS city strings (trim, lowercase, remove hidden whitespace/newlines)
2. Detects “multi-city” values and **explodes** them into multiple rows:
   - one CAIS school → multiple `city_candidate` rows
3. Builds a new join key per candidate city:
   - `join_key_cityfix = school_name | city_candidate | state`
4. Preserves provenance so we can trace every match:
   - keep original `city` (raw/normalized) and add `city_candidate`

### Output
This section produces:

- `cais_ca_base` — CA-only, deduped CAIS rows (one per CAIS school)
- `cais_ca_exploded` — exploded CAIS rows (one per city candidate)
- `join_key_cityfix` — join key using `city_candidate` (for use in 03.6.1+)

### Determinism Rules (No Fuzzy Matching)
- We do **not** guess a “primary” city.
- A school is considered matched later if **any** `city_candidate` matches PSS.
- We do **not** cross state boundaries (CA only here).
- We do **not** relax the name in this step (name cleanup is handled in fallback later).

### Success Criteria
- No CAIS schools are lost (base row count stays the same).
- `cais_ca_exploded` row count ≥ base row count (only increases).
- We can quantify how many CAIS rows contain multi-city patterns.
- Downstream (03.6.1) baseline match rate should improve on previously-unmatched schools that were “multi-city / dirty city”.

### Notes
This is a **data normalization** step, not a matching step.
It must run **before** CAIS → PSS matching (03.6.1) so coverage metrics remain meaningful and auditable.


In [1164]:
## 03.5.1 CAIS City Normalization & Multi-City Explode (Deterministic)

print("=== 03.5.1 START (CAIS CITY NORMALIZE + EXPLODE) ===")

# Preconditions
assert "cais_clean_df" in globals(), "Run CAIS standardization first (cais_clean_df)."
assert "make_join_key" in globals(), "make_join_key() is not defined."
assert "make_join_key_loose" in globals(), "make_join_key_loose() is not defined."
assert "normalize_text" in globals(), "normalize_text() is not defined."
assert "normalize_state" in globals(), "normalize_state() is not defined."

# ---------------------------------------------------------
# 0) City splitting helpers (deterministic)
# ---------------------------------------------------------

def normalize_city_for_split(x: str) -> str:
    """
    Deterministic cleanup for city strings prior to splitting.
    NOTE: CAIS city is already normalized by normalize_text() upstream,
    but we still defensively collapse whitespace and standardize casing.
    """
    if x is None or pd.isna(x):
        return ""
    s = str(x).strip().lower()
    s = s.replace("\n", " ").replace("\r", " ").replace("\t", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s

# Overrides for known "space-separated multi-city" patterns
MULTI_CITY_OVERRIDES: dict[str, list[str]] = {
    "lafayette oakland": ["lafayette", "oakland"],
    "belmont hillsborough": ["belmont", "hillsborough"],
    "emeryville oakland": ["emeryville", "oakland"],
    "corte madera san rafael": ["corte madera", "san rafael"],
    "east palo alto san francisco": ["east palo alto", "san francisco"],
    "menlo park palo alto": ["menlo park", "palo alto"],
}

# Split tokens: commas, slashes, pipes, semicolons, ampersands, " and "
SEP_REGEX = re.compile(r"\s*(?:,|/|\||;|&|\band\b)\s*", flags=re.IGNORECASE)

def split_city_candidates(city_raw: str) -> list[str]:
    """
    Deterministic city splitting:
    1) normalize for split
    2) apply overrides for no-separator multi-city strings
    3) else split on separators
    4) normalize each candidate with normalize_text() so it matches PSS city normalization
    """
    city_norm = normalize_city_for_split(city_raw)
    if not city_norm:
        return []

    if city_norm in MULTI_CITY_OVERRIDES:
        parts = MULTI_CITY_OVERRIDES[city_norm]
    else:
        parts = [p.strip() for p in SEP_REGEX.split(city_norm) if p and p.strip()]
        if not parts:
            parts = [city_norm]

    # normalize candidates using the SAME function used for join keys elsewhere
    parts = [normalize_text(p) for p in parts]
    parts = [p for p in parts if p]  # drop None/empty
    return parts

# ---------------------------------------------------------
# 1) Prepare CAIS CA base (dedupe)
# ---------------------------------------------------------
cais_ca_base = (
    cais_clean_df[cais_clean_df["state"] == "CA"]
    .drop_duplicates(subset=["school_name", "city", "state"])
    .copy()
)

# Audit columns
cais_ca_base["city_for_split"] = cais_ca_base["city"].apply(normalize_city_for_split)

print("CAIS (CA) base unique rows:", cais_ca_base.shape[0])
display(cais_ca_base[["school_name", "city", "city_for_split", "state", "detail_url"]].head(10))

# ---------------------------------------------------------
# 2) Explode into city candidates + rebuild join keys
# ---------------------------------------------------------
cais_ca_exploded = cais_ca_base.copy()
cais_ca_exploded["city_candidate"] = cais_ca_exploded["city_for_split"].apply(split_city_candidates)
cais_ca_exploded = cais_ca_exploded.explode("city_candidate").copy()

# drop empty candidates (already mostly handled, but defensive)
cais_ca_exploded["city_candidate"] = (
    cais_ca_exploded["city_candidate"]
    .astype(str)
    .str.strip()
    .replace({"": np.nan, "nan": np.nan, "none": np.nan})
)
cais_ca_exploded = cais_ca_exploded.dropna(subset=["city_candidate"]).copy()

# Strict + loose join keys that are compatible with PSS backbone
cais_ca_exploded["join_key_strict_cityfix"] = cais_ca_exploded.apply(
    lambda r: make_join_key(r["school_name"], r["city_candidate"], r["state"]),
    axis=1
)
cais_ca_exploded["join_key_loose_cityfix"] = cais_ca_exploded.apply(
    lambda r: make_join_key_loose(r["school_name"], r["city_candidate"], r["state"]),
    axis=1
)

print("\nCAIS exploded rows:", cais_ca_exploded.shape[0])

# ---------------------------------------------------------
# 3) Diagnostics (how many are multi-city?)
# ---------------------------------------------------------
city_counts = (
    cais_ca_exploded.groupby(["school_name", "city", "state"])
    .size()
    .reset_index(name="city_candidate_count")
    .sort_values("city_candidate_count", ascending=False)
)

multi_city = city_counts[city_counts["city_candidate_count"] > 1].copy()
print("\nMulti-city CAIS rows (count>1):", multi_city.shape[0])
display(multi_city.head(25))

if multi_city.shape[0] > 0:
    ex = multi_city.head(10)[["school_name", "city", "state"]]
    for _, row in ex.iterrows():
        sn, ct, st = row["school_name"], row["city"], row["state"]
        subset = cais_ca_exploded[
            (cais_ca_exploded["school_name"] == sn) &
            (cais_ca_exploded["city"] == ct) &
            (cais_ca_exploded["state"] == st)
        ][["school_name", "city", "city_candidate", "join_key_strict_cityfix", "join_key_loose_cityfix", "detail_url"]]
        print(f"\n--- Example: {sn} | city='{ct}' ---")
        display(subset)

print("\nSanity checks:")
print(" - base rows:", cais_ca_base.shape[0])
print(" - exploded rows:", cais_ca_exploded.shape[0])
print(" - exploded >= base:", cais_ca_exploded.shape[0] >= cais_ca_base.shape[0])

print("=== 03.5.1 END ===")


=== 03.5.1 START (CAIS CITY NORMALIZE + EXPLODE) ===
CAIS (CA) base unique rows: 97


,school_name,city,city_for_split,state,detail_url
1,almaden country day school,san jose,san jose,CA,https://www.caisca.org/schools/almaden-country...
2,alta vista school,san francisco,san francisco,CA,https://www.caisca.org/schools/alta-vista-school
3,athenian school,danville,danville,CA,https://www.caisca.org/schools/the-athenian-sc...
4,bay school of san francisco,san francisco,san francisco,CA,https://www.caisca.org/schools/the-bay-school-...
5,bayhill high school,berkeley,berkeley,CA,https://www.caisca.org/schools/bayhill-high-sc...
6,bentley school,lafayette oakland,lafayette oakland,CA,https://www.caisca.org/schools/bentley-school
7,berkeley school,berkeley,berkeley,CA,https://www.caisca.org/schools/the-berkeley-sc...
8,berkwood hedge school,berkeley,berkeley,CA,https://www.caisca.org/schools/berkwood-hedge-...
9,black pine circle school,berkeley,berkeley,CA,https://www.caisca.org/schools/black-pine-circ...
10,blue oak school,napa,napa,CA,https://www.caisca.org/schools/blue-oak-school



CAIS exploded rows: 103

Multi-city CAIS rows (count>1): 6


,school_name,city,state,city_candidate_count
26,escuela bilinga1 4e internacional,emeryville oakland,CA,2
51,marin montessori school,corte madera san rafael,CA,2
5,bentley school,lafayette oakland,CA,2
44,la scuola international school,east palo alto san francisco,CA,2
21,crystal springs uplands school,belmont hillsborough,CA,2
79,silicon valley international school,menlo park palo alto,CA,2



--- Example: escuela bilinga1 4e internacional | city='emeryville oakland' ---


...


--- Example: crystal springs uplands school | city='belmont hillsborough' ---


,school_name,city,city_candidate,join_key_strict_cityfix,join_key_loose_cityfix,detail_url
22,crystal springs uplands school,belmont hillsborough,belmont,crystal springs uplands school|belmont|CA,crystal springs uplands|belmont|CA,https://www.caisca.org/schools/crystal-springs...
22,crystal springs uplands school,belmont hillsborough,hillsborough,crystal springs uplands school|hillsborough|CA,crystal springs uplands|hillsborough|CA,https://www.caisca.org/schools/crystal-springs...



--- Example: silicon valley international school | city='menlo park palo alto' ---


,school_name,city,city_candidate,join_key_strict_cityfix,join_key_loose_cityfix,detail_url
81,silicon valley international school,menlo park palo alto,menlo park,silicon valley international school|menlo park|CA,silicon valley international|menlo park|CA,https://www.caisca.org/schools/silicon-valley-...
81,silicon valley international school,menlo park palo alto,palo alto,silicon valley international school|palo alto|CA,silicon valley international|palo alto|CA,https://www.caisca.org/schools/silicon-valley-...



Sanity checks:
 - base rows: 97
 - exploded rows: 103
 - exploded >= base: True
=== 03.5.1 END ===


## 03.5.2 CAIS → PSS Matching Using City-Fix Join Keys

This step attempts to match CAIS schools to the **PSS private-school backbone** using
a *deterministic city-fix strategy* introduced in **03.5.1**.

### Problem Addressed
The CAIS dataset frequently lists **multiple cities in a single field**, for example:

- `“Menlo Park Palo Alto”`
- `“East Palo Alto San Francisco”`
- `“Lafayette Oakland”`

Meanwhile, the PSS backbone contains **exactly one city per school**.

A strict `(school_name, city, state)` join therefore fails even when the schools
are clearly the same institution.

---

### Matching Strategy (City-Fix)

For each CAIS school:

1. **Explode multi-city values** into multiple `city_candidate` rows  
   - One row per possible city
2. **Rebuild a deterministic join key**: school_name | city_candidate | state
3. **Join CAIS → PSS** on this city-fixed join key
4. **Collapse results back to one row per CAIS school**:
- If *any* city candidate matches → school is considered matched
- A deterministic rule selects the first valid PSS school_id

This approach is:
- ✅ deterministic
- ✅ explainable
- ❌ not fuzzy (no aliasing, no name normalization beyond earlier steps)

---

### What This Step Produces

- `cais_cityfix_summary`
- One row per CAIS school (98 total)
- Includes:
 - `matched_cityfix` (boolean)
 - matched `school_id` where applicable
- `cais_presence_pss_cityfix_df`
- Presence table keyed by **PSS school_id**
- Used later when merging enrichment flags
- A reduced **unmatched CAIS list** for further analysis

---

### Why We Stop Here (For Now)

This step intentionally **does not** apply:
- name alias rules
- accent stripping
- punctuation normalization
- heuristic or fuzzy matching

Those techniques are deferred to later sections so that:
- baseline coverage remains auditable
- heuristic logic can be toggled independently

The output of this step represents the **strict, city-aware baseline** for CAIS coverage.



In [1167]:
## 03.5.2 CAIS → PSS Matching Using City-Fix Join Keys (Deterministic + 1:1 gated)

print("=== 03.5.2 START (CAIS → PSS MATCH USING CITYFIX KEY) ===")

assert "cais_ca_exploded" in globals(), "Run 03.5.1 first (cais_ca_exploded)."
assert "pss_clean_df" in globals(), "Run 03.5 first (pss_clean_df)."
assert "cais_clean_df" in globals(), "Run CAIS standardization first (cais_clean_df)."

# --- Pick which CAIS key to use (STRICT baseline) ---
CAIS_CITYFIX_KEY_COL = "join_key_strict_cityfix"
assert CAIS_CITYFIX_KEY_COL in cais_ca_exploded.columns, f"Missing {CAIS_CITYFIX_KEY_COL} in cais_ca_exploded"

# ---------------------------------------------------------
# 0) Build a CA-only PSS backbone (dedupe by join_key)
# ---------------------------------------------------------
pss_ca = pss_clean_df[pss_clean_df["state"] == "CA"].copy()

# Deterministic: one row per join_key, pick smallest ppin
pss_ca_backbone = (
    pss_ca.sort_values(["join_key", "ppin"])
    .drop_duplicates(subset=["join_key"])
    .copy()
)

print("PSS CA row-level:", pss_ca.shape)
print("PSS CA backbone (deduped by join_key):", pss_ca_backbone.shape)

# ---------------------------------------------------------
# 1) Join exploded CAIS rows to PSS backbone by cityfix join key
# ---------------------------------------------------------
cais_to_pss_cityfix = cais_ca_exploded.merge(
    pss_ca_backbone[["school_id", "ppin", "school_name", "city", "state", "join_key"]],
    left_on=CAIS_CITYFIX_KEY_COL,
    right_on="join_key",
    how="left",
    indicator=True,
    suffixes=("_cais", "_pss")
)

total_exploded = cais_to_pss_cityfix.shape[0]
matched_exploded = int((cais_to_pss_cityfix["_merge"] == "both").sum())

print("\n--- Exploded-row coverage (not final school-level coverage) ---")
print(f"Exploded CAIS rows: {total_exploded}")
print(f"Exploded rows matched to PSS backbone: {matched_exploded} ({matched_exploded/total_exploded:.2%})")

matched_rows = cais_to_pss_cityfix[cais_to_pss_cityfix["_merge"] == "both"].copy()

# ---------------------------------------------------------
# 2) Gate to 1:1 matches (avoid silently picking wrong school)
#    Use CAIS identity columns from the merged dataframe:
#    - school_name_cais, city_cais, detail_url
# ---------------------------------------------------------
LEFT_NAME_COL = "school_name_cais" if "school_name_cais" in matched_rows.columns else "school_name"
LEFT_CITY_COL = "city_cais" if "city_cais" in matched_rows.columns else "city"

assert LEFT_NAME_COL in matched_rows.columns, f"Missing {LEFT_NAME_COL} in matched_rows"
assert LEFT_CITY_COL in matched_rows.columns, f"Missing {LEFT_CITY_COL} in matched_rows"
assert "detail_url" in matched_rows.columns, "Missing detail_url in matched_rows (expected from CAIS)"

matched_rows["_cais_left_id"] = matched_rows[[LEFT_NAME_COL, LEFT_CITY_COL, "detail_url"]].astype(str).agg("|".join, axis=1)

left_to_right = matched_rows.groupby("_cais_left_id")["school_id"].nunique().reset_index(name="n_pss")
right_to_left = matched_rows.groupby("school_id")["_cais_left_id"].nunique().reset_index(name="n_cais")

gated = (
    matched_rows
    .merge(left_to_right, on="_cais_left_id", how="left")
    .merge(right_to_left, on="school_id", how="left")
)

cais_cityfix_ambiguous = gated[(gated["n_pss"] > 1) | (gated["n_cais"] > 1)].copy()
cais_cityfix_gated = gated[(gated["n_pss"] == 1) & (gated["n_cais"] == 1)].copy()

print("\n--- 1:1 gating ---")
print("Matched rows (pre-gate):", matched_rows.shape[0])
print("Ambiguous rows dropped:", cais_cityfix_ambiguous.shape[0])
print("Rows kept (1:1):", cais_cityfix_gated.shape[0])

# Deterministically select one row per CAIS school (safe)
cais_to_pss_cityfix_matches = (
    cais_cityfix_gated.sort_values(["_cais_left_id", "ppin"])
    .drop_duplicates(subset=["_cais_left_id"])
    .copy()
)

# ---------------------------------------------------------
# 3) Build school-level summary (one row per CAIS school in CA)
# ---------------------------------------------------------
cais_ca_base = (
    cais_clean_df[cais_clean_df["state"] == "CA"]
    .drop_duplicates(subset=["school_name", "city", "state"])
    .copy()
)

cais_ca_base["_cais_left_id"] = cais_ca_base[["school_name", "city", "detail_url"]].astype(str).agg("|".join, axis=1)

summary = cais_ca_base.merge(
    cais_to_pss_cityfix_matches[[
        "_cais_left_id",
        "city_candidate",
        "school_id", "ppin",
        "school_name_pss", "city_pss", "state_pss"
    ]],
    on="_cais_left_id",
    how="left"
)

summary["matched_cityfix"] = summary["school_id"].notna()

total = summary.shape[0]
matched = int(summary["matched_cityfix"].sum())

print("\n--- School-level coverage after city-fix + gating ---")
print(f"CAIS schools (CA): {total}")
print(f"Matched to PSS: {matched} ({matched/total:.2%})")
print(f"Unmatched: {total-matched} ({(total-matched)/total:.2%})")

cais_cityfix_summary = summary.drop(columns=["_cais_left_id"]).copy()

print("\nSample matched (first 15):")
display(
    cais_cityfix_summary[cais_cityfix_summary["matched_cityfix"]][
        ["school_name", "city", "city_candidate", "school_id", "ppin"]
    ].head(15)
)

print("\nSample still unmatched (first 15):")
display(
    cais_cityfix_summary[~cais_cityfix_summary["matched_cityfix"]][
        ["school_name", "city", "detail_url"]
    ].head(15)
)

# ---------------------------------------------------------
# 4) Presence table keyed by PSS school_id
# ---------------------------------------------------------
cais_presence_pss_cityfix_df = (
    cais_cityfix_summary.loc[cais_cityfix_summary["matched_cityfix"], ["school_id"]]
    .drop_duplicates()
    .copy()
)
cais_presence_pss_cityfix_df["has_cais"] = True

print("\nCityfix presence table shape:", cais_presence_pss_cityfix_df.shape)
display(cais_presence_pss_cityfix_df.head(10))

# Optional: show ambiguous cases (don’t auto-resolve)
if len(cais_cityfix_ambiguous) > 0:
    print("\n⚠️ Ambiguous CAIS↔PSS candidates (review later, not auto-resolved):")
    display(
        cais_cityfix_ambiguous[[
            LEFT_NAME_COL, LEFT_CITY_COL, "city_candidate",
            "school_id", "ppin", "school_name_pss", "city_pss",
            "n_pss", "n_cais", "detail_url"
        ]].head(30)
    )

print("=== 03.5.2 END ===")


=== 03.5.2 START (CAIS → PSS MATCH USING CITYFIX KEY) ===
PSS CA row-level: (2452, 12)
PSS CA backbone (deduped by join_key): (2410, 12)

--- Exploded-row coverage (not final school-level coverage) ---
Exploded CAIS rows: 103
Exploded rows matched to PSS backbone: 72 (69.90%)

--- 1:1 gating ---
Matched rows (pre-gate): 72
Ambiguous rows dropped: 0
Rows kept (1:1): 72

--- School-level coverage after city-fix + gating ---
CAIS schools (CA): 97
Matched to PSS: 72 (74.23%)
Unmatched: 25 (25.77%)

Sample matched (first 15):


,school_name,city,city_candidate,school_id,ppin
0,almaden country day school,san jose,san jose,PRI_02013539,02013539
1,alta vista school,san francisco,san francisco,PRI_A1300133,A1300133
2,athenian school,danville,danville,PRI_00083611,00083611
3,bay school of san francisco,san francisco,san francisco,PRI_A0500717,A0500717
4,bayhill high school,berkeley,berkeley,PRI_A0900219,A0900219
6,berkeley school,berkeley,berkeley,PRI_00084058,00084058
7,berkwood hedge school,berkeley,berkeley,PRI_00084091,00084091
8,black pine circle school,berkeley,berkeley,PRI_00083881,00083881
9,blue oak school,napa,napa,PRI_A0500178,A0500178
11,brandeis school of san francisco,san francisco,san francisco,PRI_00093539,00093539



Sample still unmatched (first 15):


,school_name,city,detail_url
5,bentley school,lafayette oakland,https://www.caisca.org/schools/bentley-school
10,brandeis marin,san rafael,https://www.caisca.org/schools/brandeis-marin
14,cathedral school for boys,san francisco,https://www.caisca.org/schools/cathedral-schoo...
17,chinese american international school,san francisco,https://www.caisca.org/schools/chinese-america...
20,convent and stuart hall schools of the sacred ...,san francisco,https://www.caisca.org/schools/convent-and-stu...
21,crystal springs uplands school,belmont hillsborough,https://www.caisca.org/schools/crystal-springs...
23,east bay school,berkeley,https://www.caisca.org/schools/east-bay-school
26,escuela bilinga1 4e internacional,emeryville oakland,https://www.caisca.org/schools/escuela-bilingu...
27,field middle school,san mateo,https://www.caisca.org/schools/field-middle-sc...
29,german international school of silicon valley,mountain view,https://www.caisca.org/schools/german-internat...



Cityfix presence table shape: (72, 2)


,school_id,has_cais
0,PRI_02013539,True
1,PRI_A1300133,True
2,PRI_00083611,True
3,PRI_A0500717,True
4,PRI_A0900219,True
6,PRI_00084058,True
7,PRI_00084091,True
8,PRI_00083881,True
9,PRI_A0500178,True
11,PRI_00093539,True


=== 03.5.2 END ===


## 03.5.3 Finalize CAIS → PSS Matching (City-Fix Baseline)

At this stage, we finalize the **first deterministic pass** of CAIS enrichment matching against the
**PSS private-school backbone**, using only **strict name + city normalization** and **multi-city
explosion**.

### What has been applied so far

1. **Canonical text normalization**
   - School name normalization (case, punctuation, accents)
   - City normalization (lowercase, trimming, whitespace cleanup)

2. **Multi-city expansion**
   - CAIS rows with compound city values (e.g.  
     `"Menlo Park Palo Alto"`, `"East Palo Alto San Francisco"`) are **exploded**
     into multiple candidate city rows
   - Each city candidate produces its own deterministic join key

3. **Deterministic join strategy**
   - Join key:  
     `school_name_normalized | city_candidate | state`
   - No fuzzy matching
   - No manual aliases
   - No secondary backbones

---

### Results (City-Fix Baseline)

- **Total CAIS schools (Bay Area):** 98
- **Matched to PSS backbone:** 73 (**~74% coverage**)
- **Unmatched:** 25 (**~26% remaining**)

This establishes a **clean, explainable baseline** before applying any fallback logic.

---

### Output artifacts from this step

| Artifact | Description |
|--------|-------------|
| `cais_final_summary` | School-level CAIS match summary (1 row per CAIS school) |
| `school_id_cityfix` | Matched PSS `school_id` from city-fix join |
| `matched_cityfix` | Boolean flag for city-fix match |
| `cais_presence_pss_cityfix_df` | `[school_id, has_cais=True]` presence table (PSS backbone) |
| `cais_unmatched_cityfix` | Remaining unmatched CAIS schools for diagnostics |

---

### Why we stop here (intentionally)

The remaining ~25% of CAIS schools are **not failures** — they fall into known, addressable categories:

- Multi-campus schools with non-standard city strings
- Name aliases or institutional branding differences
- Schools missing from PSS or requiring an alternate backbone

Rather than forcing matches, we **explicitly bucket and diagnose** these cases next
(see **03.5.4**) to preserve correctness and auditability.

This ensures all future improvements are **intentional, traceable, and reversible**.


In [1170]:
print("=== 03.5.3 START (FINALIZE CAIS CITYFIX + PRESENCE TABLE) ===")

# 0) Sanity: required columns exist in cityfix summary
req = ["school_name", "city", "state", "matched_cityfix"]
missing = [c for c in req if c not in cais_cityfix_summary.columns]
assert not missing, f"cais_cityfix_summary missing columns: {missing}"

# Make sure detail_url exists for downstream unmatched inspection
if "detail_url" not in cais_cityfix_summary.columns and "detail_url" in cais_clean_df.columns:
    # restore it by joining back to CAIS base on strict identity
    _cais_base = (
        cais_clean_df[cais_clean_df["state"] == "CA"]
        .drop_duplicates(subset=["school_name", "city", "state"])[["school_name", "city", "state", "detail_url"]]
        .copy()
    )
    cais_cityfix_summary = cais_cityfix_summary.merge(
        _cais_base,
        on=["school_name", "city", "state"],
        how="left"
    )

# 1) Create final summary (for now: cityfix only; fallback comes later as 03.5.4)
cais_final_summary = cais_cityfix_summary.copy()

# Your cityfix matched school_id column might already be named 'school_id'
# Normalize to explicit fields:
if "school_id_cityfix" not in cais_final_summary.columns:
    cais_final_summary = cais_final_summary.rename(columns={"school_id": "school_id_cityfix"})

cais_final_summary["school_id_final"] = cais_final_summary["school_id_cityfix"]
cais_final_summary["matched_final"] = cais_final_summary["school_id_final"].notna()

total = cais_final_summary.shape[0]
matched = int(cais_final_summary["matched_final"].sum())

print(f"CAIS total: {total}")
print(f"Matched (cityfix): {matched} ({matched/total:.2%})")
print(f"Unmatched: {total-matched} ({(total-matched)/total:.2%})")

# 2) Presence table (PSS backbone ids)
cais_presence_pss_cityfix_df_final = (
    cais_final_summary.loc[cais_final_summary["matched_final"], ["school_id_final"]]
    .drop_duplicates()
    .rename(columns={"school_id_final": "school_id"})
    .copy()
)
cais_presence_pss_cityfix_df_final["has_cais"] = True

print("Presence table shape:", cais_presence_pss_cityfix_df_final.shape)
display(cais_presence_pss_cityfix_df_final.head(10))

# 3) Keep unmatched list for the next step (fallback)
unmatched_cols = ["school_name", "city", "state"]
if "detail_url" in cais_final_summary.columns:
    unmatched_cols.append("detail_url")

cais_unmatched_cityfix = cais_final_summary.loc[~cais_final_summary["matched_final"], unmatched_cols].copy()

print("\nUnmatched after cityfix (sample 15):")
display(cais_unmatched_cityfix.head(15))

print("=== 03.5.3 END ===")


=== 03.5.3 START (FINALIZE CAIS CITYFIX + PRESENCE TABLE) ===
CAIS total: 97
Matched (cityfix): 72 (74.23%)
Unmatched: 25 (25.77%)
Presence table shape: (72, 2)


,school_id,has_cais
0,PRI_02013539,True
1,PRI_A1300133,True
2,PRI_00083611,True
3,PRI_A0500717,True
4,PRI_A0900219,True
6,PRI_00084058,True
7,PRI_00084091,True
8,PRI_00083881,True
9,PRI_A0500178,True
11,PRI_00093539,True



Unmatched after cityfix (sample 15):


,school_name,city,state,detail_url
5,bentley school,lafayette oakland,CA,https://www.caisca.org/schools/bentley-school
10,brandeis marin,san rafael,CA,https://www.caisca.org/schools/brandeis-marin
14,cathedral school for boys,san francisco,CA,https://www.caisca.org/schools/cathedral-schoo...
17,chinese american international school,san francisco,CA,https://www.caisca.org/schools/chinese-america...
20,convent and stuart hall schools of the sacred ...,san francisco,CA,https://www.caisca.org/schools/convent-and-stu...
21,crystal springs uplands school,belmont hillsborough,CA,https://www.caisca.org/schools/crystal-springs...
23,east bay school,berkeley,CA,https://www.caisca.org/schools/east-bay-school
26,escuela bilinga1 4e internacional,emeryville oakland,CA,https://www.caisca.org/schools/escuela-bilingu...
27,field middle school,san mateo,CA,https://www.caisca.org/schools/field-middle-sc...
29,german international school of silicon valley,mountain view,CA,https://www.caisca.org/schools/german-internat...


=== 03.5.3 END ===


## 03.5.4 CAIS Unmatched Analysis & Bucketing (Post City-Fix + Fallback)

After applying **city normalization**, **multi-city explode**, and a **conservative fallback name key**,
a subset of CAIS schools still remain unmatched to the PSS private-school backbone.

This section performs a **human-auditable analysis** of the remaining unmatched schools to determine:

- which failures are *systematic and fixable*
- which require *explicit alias rules*
- which are *missing entirely from PSS* and need an alternate backbone

---

### Current Matching Status

At this stage:

- Total CAIS schools (Bay Area): **98**
- Matched via city-fix + fallback: **~73–75%**
- Remaining unmatched: **~25 schools**

These unmatched schools are **not treated as errors**.
Instead, they represent known limitations of deterministic matching.

---

### Bucketing Strategy

Each remaining unmatched CAIS school is categorized into one of the following buckets:

#### **Bucket A — Multi-City / Dirty City (Fixable)**
Schools whose CAIS `city` field:
- contains multiple cities (`"Menlo Park Palo Alto"`)
- contains hidden whitespace or formatting artifacts
- already partially fixed but needs stricter tokenization

➡️ Action: Extend city parsing rules (already mostly handled in 03.5.1)

---

#### **Bucket B — Name Alias Required**
Schools whose official CAIS name differs from the PSS canonical name, e.g.:
- abbreviations
- missing suffixes (`School`, `Academy`)
- branded vs legal names

➡️ Action: Add **explicit alias mappings** (deterministic, versioned)

---

#### **Bucket C — Missing from PSS Backbone**
Legitimate CAIS schools that do **not exist** in the PSS dataset, often because:
- they opened after the PSS survey year
- they are non-traditional or multi-campus entities
- they operate under a parent organization

➡️ Action: Add alternate backbone sources (e.g. CA Private Directory, CAIS itself)

---

#### **Bucket D — Intentional Exclusion**
Cases where:
- multiple PSS schools match ambiguously
- deterministic matching would risk false positives

➡️ Action: Leave unmatched by design (no heuristic guess)

---

### Why This Matters

This bucketing step ensures that:

- matching logic remains **explainable and deterministic**
- no silent false positives are introduced
- future improvements can be targeted and measured

Each bucket represents a **different class of data engineering work**, not a single “bug”.

---

### Output of This Section

- A table of unmatched CAIS schools with:
  - school name
  - city
  - detail URL
  - assigned bucket
- Clear documentation of **why** each school failed to match
- A roadmap for future enrichment improvements (Notebook 05+)

This concludes the **CAIS → PSS deterministic matching phase**.


In [1173]:
print("=== 03.5.4 START (BUCKET UNMATCHED CAIS AFTER CITYFIX) ===")

# Preconditions
assert "cais_unmatched_cityfix" in globals(), "Run 03.5.3 first (needs cais_unmatched_cityfix)."
assert "pss_clean_df" in globals(), "Need pss_clean_df (PSS backbone)."
assert "cais_ca_exploded" in globals(), "Need cais_ca_exploded from 03.5.1."
assert "make_join_key" in globals(), "make_join_key() not defined."

unmatched = cais_unmatched_cityfix.copy()
unmatched["school_name"] = unmatched["school_name"].fillna("").astype(str)
unmatched["city"] = unmatched["city"].fillna("").astype(str)
unmatched["state"] = unmatched["state"].fillna("").astype(str)

pss_ca = pss_clean_df[pss_clean_df["state"] == "CA"].copy()
pss_ca["school_name"] = pss_ca["school_name"].fillna("").astype(str)
pss_ca["city"] = pss_ca["city"].fillna("").astype(str)

# ---------------------------------------------------------
# 0) Deterministic: which CAIS base rows were multi-city in 03.5.1?
# ---------------------------------------------------------
# In 03.5.1 you effectively define a CAIS base identity as (school_name, city, state).
# We reconstruct the city_candidate_count deterministically from cais_ca_exploded.
city_counts = (
    cais_ca_exploded.groupby(["school_name", "city", "state"])
    .size()
    .reset_index(name="city_candidate_count")
)

multi_city_keys = set(
    city_counts.loc[city_counts["city_candidate_count"] > 1, ["school_name", "city", "state"]]
    .astype(str)
    .agg("|".join, axis=1)
    .tolist()
)

def is_multi_city_row(school_name: str, city: str, state: str) -> bool:
    k = f"{school_name}|{city}|{state}"
    return k in multi_city_keys

# ---------------------------------------------------------
# 1) Soft name normalization for candidate hints (NOT matching yet)
# ---------------------------------------------------------
def strip_accents(s: str) -> str:
    return "".join(
        ch for ch in unicodedata.normalize("NFKD", str(s))
        if not unicodedata.combining(ch)
    )

def normalize_name_soft(s: str) -> str:
    x = strip_accents(s).lower().strip()
    x = re.sub(r"^the\s+", "", x)              # drop leading "the"
    x = re.sub(r"&", " and ", x)
    x = re.sub(r"[^\w\s]", " ", x)             # punctuation -> space
    x = re.sub(r"\s+", " ", x).strip()
    # conservative suffix removal
    x = re.sub(r"\b(school|schools)\b$", "", x).strip()
    return x

def normalize_city_soft(s: str) -> str:
    x = strip_accents(s).lower().strip()
    x = re.sub(r"[^\w\s]", " ", x)
    x = re.sub(r"\s+", " ", x).strip()
    return x

# Create a stable city_norm for PSS
pss_ca["city_norm"] = pss_ca["city"].apply(normalize_city_soft)

# Group PSS by normalized city for narrow candidate pool
pss_city_groups = pss_ca.groupby("city_norm")

school_words = re.compile(r"\b(school|academy|montessori|prep|preparatory|institute|christian|catholic|elementary|middle|high|international)\b", re.I)

def best_pss_name_candidates(cais_name: str, city: str, topn: int = 5) -> list[str]:
    """
    Returns PSS school_name candidates (string list) with highest token-overlap,
    using same-city pool when available.
    """
    city_norm = normalize_city_soft(city)
    if city_norm in pss_city_groups.groups:
        pool = pss_ca.loc[pss_city_groups.groups[city_norm], "school_name"].tolist()
    else:
        pool = pss_ca["school_name"].tolist()

    target = normalize_name_soft(cais_name)
    tset = set(target.split())

    scored = []
    for nm in pool:
        cand = normalize_name_soft(nm)
        cset = set(cand.split())
        if not tset or not cset:
            continue
        overlap = len(tset & cset) / len(tset | cset)
        # tiny boost if candidate contains school-ish words and target does too
        boost = 0.02 if (school_words.search(nm) and school_words.search(cais_name)) else 0.0
        scored.append((overlap + boost, nm))

    scored.sort(reverse=True, key=lambda x: x[0])
    return [nm for s, nm in scored[:topn]]

def token_overlap_ratio(a: str, b: str) -> float:
    a2 = normalize_name_soft(a)
    b2 = normalize_name_soft(b)
    A = set(a2.split())
    B = set(b2.split())
    if not A or not B:
        return 0.0
    return len(A & B) / len(A | B)

# ---------------------------------------------------------
# 2) Bucket unmatched rows
# ---------------------------------------------------------
rows = []
for _, r in unmatched.iterrows():
    nm = r["school_name"]
    city = r["city"]
    st = r["state"]

    multi_city = is_multi_city_row(nm, city, st)

    candidates = best_pss_name_candidates(nm, city, topn=5)

    alias_like = False
    best_overlap = 0.0
    if candidates:
        best_overlap = max(token_overlap_ratio(nm, cand) for cand in candidates)
        # if any candidate shares >= ~0.60 tokens -> likely name formatting/alias mismatch
        if best_overlap >= 0.60:
            alias_like = True

    if multi_city:
        bucket = "A_multi_city_still_unmatched"
    elif alias_like:
        bucket = "B_name_alias_or_formatting"
    else:
        bucket = "C_probably_missing_from_PSS_or_hard_case"

    rows.append({
        "school_name": nm,
        "city": city,
        "state": st,
        "bucket": bucket,
        "best_overlap_hint": round(best_overlap, 3),
        "candidate_1": candidates[0] if len(candidates) > 0 else None,
        "candidate_2": candidates[1] if len(candidates) > 1 else None,
        "candidate_3": candidates[2] if len(candidates) > 2 else None,
        "detail_url": r.get("detail_url")
    })

bucket_df = pd.DataFrame(rows).sort_values(["bucket", "best_overlap_hint"], ascending=[True, False]).reset_index(drop=True)

print("Bucket counts:")
display(bucket_df["bucket"].value_counts().reset_index().rename(columns={"index": "bucket", "bucket": "count"}))

print("\nBucketed unmatched (full table):")
display(bucket_df)

print("=== 03.5.4 END ===")


=== 03.5.4 START (BUCKET UNMATCHED CAIS AFTER CITYFIX) ===
Bucket counts:


,count,count
0,C_probably_missing_from_PSS_or_hard_case,15
1,B_name_alias_or_formatting,7
2,A_multi_city_still_unmatched,3



Bucketed unmatched (full table):


,school_name,city,state,bucket,best_overlap_hint,candidate_1,candidate_2,candidate_3,detail_url
0,escuela bilinga1 4e internacional,emeryville oakland,CA,A_multi_city_still_unmatched,0.400,escuela bilingue internacional,escuela plus elementary,escuela bilingue international,https://www.caisca.org/schools/escuela-bilingu...
1,crystal springs uplands school,belmont hillsborough,CA,A_multi_city_still_unmatched,0.250,laurel springs school,learning springs academy,big springs center and school,https://www.caisca.org/schools/crystal-springs...
2,bentley school,lafayette oakland,CA,A_multi_city_still_unmatched,0.000,st thomas more school,holy angels school,our lady of fatima school,https://www.caisca.org/schools/bentley-school
3,harker school,san jose,CA,B_name_alias_or_formatting,1.000,harker,liberty baptist school,st victor elementary school,https://www.caisca.org/schools/the-harker-school
4,convent and stuart hall schools of the sacred ...,san francisco,CA,B_name_alias_or_formatting,0.818,convent and stuart hall schools of the sacred ...,san francisco high school of the arts,urban school of san francisco,https://www.caisca.org/schools/convent-and-stu...
5,german international school of silicon valley,mountain view,CA,B_name_alias_or_formatting,0.714,german intl school of silicon valley,yew chung international school sv,waldorf school peninsula,https://www.caisca.org/schools/german-internat...
6,international school of san francisco,san francisco,CA,B_name_alias_or_formatting,0.667,urban school of san francisco,brandeis school of san francisco,bay school of san francisco,https://www.caisca.org/schools/international-s...
7,san francisco school,san francisco,CA,B_name_alias_or_formatting,0.667,san francisco christian school,san francisco adventist school,san francisco waldorf school,https://www.caisca.org/schools/the-san-francis...
8,cathedral school for boys,san francisco,CA,B_name_alias_or_formatting,0.600,town school for boys,school of the epiphany,stratford school san francisco,https://www.caisca.org/schools/cathedral-schoo...
9,saint andrew s episcopal school,saratoga,CA,B_name_alias_or_formatting,0.600,st andrew s episcopal school,sacred heart school,primary plus el quito school,https://www.caisca.org/schools/saint-andrews-e...


=== 03.5.4 END ===


## 03.5.5 CAIS → PSS Alias Table (Manual Overrides, Bucket B)

Even after applying **city normalization**, **multi-city explode**, and a **conservative fallback key**, a small set of CAIS schools can remain unmatched due to:

- legal/registered names differing from public-facing names  
- merged entities in CAIS vs separate records in PSS  
- ambiguous generic names (e.g., “San Francisco School”) that require human confirmation

To preserve determinism and avoid false positives, we introduce an **explicit alias table**:

- Only applied to the *remaining unmatched* CAIS rows  
- Only for a small, reviewable set (Bucket B: name/formatting mismatches)  
- Stored as a versioned CSV artifact

**Outputs**
- `cais_alias_to_pss_v1.csv` (manual mappings)
- Updated CAIS presence flag `has_cais_pss.csv` regenerated after aliases are applied


In [1176]:
# ============================
# 03.5.5 Option A — Alias Table (Bucket B) + Re-match to PSS
# ============================

print("=== 03.5.5 START (ALIAS TABLE + REMATCH) ===")

# ---------------------------------------------------------
# Preconditions
# ---------------------------------------------------------
assert "pss_clean_df" in globals(), "Need pss_clean_df (run 03.5)."
assert "cais_cityfix_summary" in globals(), "Need cais_cityfix_summary (run 03.5.2 / 03.5.3)."
assert "bucket_df" in globals(), "Need bucket_df from 03.5.4."

bucketed_unmatched = bucket_df.copy()

# ---------------------------------------------------------
# Helpers (self-contained; same for CAIS + PSS inside this cell)
# ---------------------------------------------------------
def strip_accents(s: str) -> str:
    if s is None:
        return ""
    s = str(s)
    return "".join(
        ch for ch in unicodedata.normalize("NFKD", s)
        if not unicodedata.combining(ch)
    )

def norm_state(s: str) -> str:
    x = strip_accents(s).upper().strip()
    return x

def norm_city(s: str) -> str:
    x = strip_accents(s).lower().strip()
    x = re.sub(r"[^\w\s]", " ", x)
    x = re.sub(r"\s+", " ", x).strip()
    return x

def norm_name_fallback(name: str) -> str:
    """
    Conservative fallback normalization:
    - strip accents
    - drop leading 'the '
    - '&' -> 'and'
    - punctuation -> space
    - collapse whitespace
    - DO NOT remove meaningful tokens besides leading 'the'
    """
    x = strip_accents(name).lower().strip()
    x = re.sub(r"^the\s+", "", x)
    x = x.replace("&", " and ")
    x = re.sub(r"[^\w\s]", " ", x)
    x = re.sub(r"\s+", " ", x).strip()
    return x

def make_fallback_key(name: str, city: str, state: str) -> str:
    return f"{norm_name_fallback(name)}|{norm_city(city)}|{norm_state(state)}"

def token_overlap_ratio(a: str, b: str) -> float:
    A = set(norm_name_fallback(a).split())
    B = set(norm_name_fallback(b).split())
    if not A or not B:
        return 0.0
    return len(A & B) / len(A | B)

# ---------------------------------------------------------
# 0) Build PSS aggregated fallback map (unique-gated)
# ---------------------------------------------------------
pss_ca = pss_clean_df[pss_clean_df["state"] == "CA"].copy()

pss_ca["fallback_key"] = pss_ca.apply(
    lambda r: make_fallback_key(r["school_name"], r["city"], r["state"]),
    axis=1
)

pss_fb_map = (
    pss_ca.groupby("fallback_key")
    .agg(
        pss_school_id_count=("school_id", "nunique"),
        school_id_pss=("school_id", "first"),
        ppin_pss=("ppin", "first"),
        pss_school_name_pss=("school_name", "first"),
        pss_city_pss=("city", "first"),
    )
    .reset_index()
)

print("PSS CA fallback map rows:", pss_fb_map.shape[0])
print("PSS CA unique-gated keys (count==1):", int((pss_fb_map["pss_school_id_count"] == 1).sum()))

# ---------------------------------------------------------
# 1) Create Alias Table from Bucket B only (auto-seeded)
# ---------------------------------------------------------
alias_candidates = bucketed_unmatched[bucketed_unmatched["bucket"] == "B_name_alias_or_formatting"].copy()

print("\nBucket B rows (alias/formatting candidates):", alias_candidates.shape[0])
display(alias_candidates[["school_name", "city", "best_overlap_hint", "candidate_1", "candidate_2", "candidate_3", "detail_url"]])

alias_table = alias_candidates[["school_name", "city", "state", "detail_url", "best_overlap_hint", "candidate_1"]].copy()
alias_table = alias_table.rename(columns={"school_name": "cais_school_name"})

# Auto-seed: if overlap is strong, use candidate_1 as alias_name
alias_table["alias_name"] = np.where(
    alias_table["best_overlap_hint"].fillna(0) >= 0.65,
    alias_table["candidate_1"].fillna(""),
    ""
)

alias_table["notes"] = np.where(
    alias_table["alias_name"] != "",
    "auto_seed_from_candidate_1",
    ""
)

# Manual seed overrides (optional)
seed_aliases = {
    "harker school": "harker",
    "saint andrew s episcopal school": "st andrew s episcopal school",
    "german international school of silicon valley": "german intl school of silicon valley",
}
alias_table["alias_name"] = alias_table.apply(
    lambda r: seed_aliases.get(str(r["cais_school_name"]).strip().lower(), r["alias_name"]),
    axis=1
)

print("\n=== Editable alias_table (you may edit alias_name / notes) ===")
display(alias_table[["cais_school_name", "city", "state", "alias_name", "notes", "detail_url"]])

print("""
HOW TO USE:
- alias_name is what PSS likely uses for the same school.
- You can edit alias_table['alias_name'] and re-run this cell.
- We only accept matches where the resulting fallback_key maps to exactly ONE PSS school_id (unique-gated).
""")

# ---------------------------------------------------------
# 2) Apply alias_name ONLY for Bucket B rows among remaining unmatched
# ---------------------------------------------------------
cais_work = cais_cityfix_summary.copy()
for c in ["school_name", "city", "state"]:
    cais_work[c] = cais_work[c].fillna("").astype(str)

remaining = cais_work[~cais_work["matched_cityfix"]].copy()

# Build alias lookup keyed by (school_name, city, state)
alias_lookup = alias_table.copy()
alias_lookup["alias_name"] = alias_lookup["alias_name"].fillna("").astype(str).str.strip()
alias_lookup = alias_lookup[alias_lookup["alias_name"] != ""].copy()

alias_map = alias_lookup.set_index(["cais_school_name", "city", "state"])["alias_name"]

def apply_alias(row):
    return alias_map.get((row["school_name"], row["city"], row["state"]), row["school_name"])

remaining["school_name_alias_applied"] = remaining.apply(apply_alias, axis=1)

# ---------------------------------------------------------
# 3) Recompute fallback_key using alias-applied name, then join to unique-gated PSS map
# ---------------------------------------------------------
remaining["fallback_key_alias"] = remaining.apply(
    lambda r: make_fallback_key(r["school_name_alias_applied"], r["city"], r["state"]),
    axis=1
)

fb_join = remaining.merge(
    pss_fb_map,
    left_on="fallback_key_alias",
    right_on="fallback_key",
    how="left"
)

# Accept only unique-gated matches
fb_join["school_id_alias_match"] = np.where(
    (fb_join["school_id_pss"].notna()) & (fb_join["pss_school_id_count"] == 1),
    fb_join["school_id_pss"],
    np.nan
)

matched_high_conf = int(pd.Series(fb_join["school_id_alias_match"]).notna().sum())
print("\nAlias-table fallback matches (high-confidence, unique-gated):", matched_high_conf)

print("\nAlias matched examples (first 25):")
display(
    fb_join[pd.notna(fb_join["school_id_alias_match"])][
        ["school_name", "school_name_alias_applied", "city",
         "school_id_alias_match", "pss_school_name_pss", "pss_city_pss"]
    ].head(25)
)

# ---------------------------------------------------------
# 4) Merge alias-fallback matches back into full summary
#    final school_id: prefer cityfix, then alias-fallback
# ---------------------------------------------------------
alias_match_map = fb_join.set_index(["school_name", "city", "state"])["school_id_alias_match"]

def _lookup_alias_match(row):
    return alias_match_map.get((row["school_name"], row["city"], row["state"]), np.nan)

cais_final_summary_v2 = cais_work.copy()
cais_final_summary_v2["school_id_alias_fallback"] = cais_final_summary_v2.apply(_lookup_alias_match, axis=1)

# If your cityfix summary used column 'school_id' for PSS matches, keep it.
# If it used 'school_id_cityfix', adjust here accordingly.
cityfix_col = "school_id"
if "school_id_cityfix" in cais_final_summary_v2.columns:
    cityfix_col = "school_id_cityfix"

cais_final_summary_v2["school_id_final_v2"] = (
    cais_final_summary_v2[cityfix_col]
    .fillna(cais_final_summary_v2["school_id_alias_fallback"])
)

cais_final_summary_v2["matched_final_v2"] = cais_final_summary_v2["school_id_final_v2"].notna()

total = cais_final_summary_v2.shape[0]
matched_final = int(cais_final_summary_v2["matched_final_v2"].sum())

print("\n=== CAIS FINAL COVERAGE v2 (cityfix + alias-table fallback) ===")
print(f"CAIS total: {total}")
print(f"Matched final v2: {matched_final} ({matched_final/total:.2%})")
print(f"Unmatched final v2: {total-matched_final} ({(total-matched_final)/total:.2%})")

print("\nStill unmatched after alias-table (first 30):")
display(
    cais_final_summary_v2[~cais_final_summary_v2["matched_final_v2"]][
        ["school_name", "city", "detail_url"]
    ].head(30)
)

# ---------------------------------------------------------
# 5) Presence table v2 (PSS backbone IDs)
# ---------------------------------------------------------
cais_presence_pss_final_v2_df = (
    cais_final_summary_v2.loc[cais_final_summary_v2["matched_final_v2"], ["school_id_final_v2"]]
    .drop_duplicates()
    .rename(columns={"school_id_final_v2": "school_id"})
    .copy()
)
cais_presence_pss_final_v2_df["has_cais"] = True

print("\nFinal CAIS presence table v2 shape:", cais_presence_pss_final_v2_df.shape)
display(cais_presence_pss_final_v2_df.head(15))

print("=== 03.5.5 END ===")


=== 03.5.5 START (ALIAS TABLE + REMATCH) ===
PSS CA fallback map rows: 2410
PSS CA unique-gated keys (count==1): 2368

Bucket B rows (alias/formatting candidates): 7


,school_name,city,best_overlap_hint,candidate_1,candidate_2,candidate_3,detail_url
3,harker school,san jose,1.000,harker,liberty baptist school,st victor elementary school,https://www.caisca.org/schools/the-harker-school
4,convent and stuart hall schools of the sacred ...,san francisco,0.818,convent and stuart hall schools of the sacred ...,san francisco high school of the arts,urban school of san francisco,https://www.caisca.org/schools/convent-and-stu...
5,german international school of silicon valley,mountain view,0.714,german intl school of silicon valley,yew chung international school sv,waldorf school peninsula,https://www.caisca.org/schools/german-internat...
6,international school of san francisco,san francisco,0.667,urban school of san francisco,brandeis school of san francisco,bay school of san francisco,https://www.caisca.org/schools/international-s...
7,san francisco school,san francisco,0.667,san francisco christian school,san francisco adventist school,san francisco waldorf school,https://www.caisca.org/schools/the-san-francis...
8,cathedral school for boys,san francisco,0.600,town school for boys,school of the epiphany,stratford school san francisco,https://www.caisca.org/schools/cathedral-schoo...
9,saint andrew s episcopal school,saratoga,0.600,st andrew s episcopal school,sacred heart school,primary plus el quito school,https://www.caisca.org/schools/saint-andrews-e...



=== Editable alias_table (you may edit alias_name / notes) ===


,cais_school_name,city,state,alias_name,notes,detail_url
3,harker school,san jose,CA,harker,auto_seed_from_candidate_1,https://www.caisca.org/schools/the-harker-school
4,convent and stuart hall schools of the sacred ...,san francisco,CA,convent and stuart hall schools of the sacred ...,auto_seed_from_candidate_1,https://www.caisca.org/schools/convent-and-stu...
5,german international school of silicon valley,mountain view,CA,german intl school of silicon valley,auto_seed_from_candidate_1,https://www.caisca.org/schools/german-internat...
6,international school of san francisco,san francisco,CA,urban school of san francisco,auto_seed_from_candidate_1,https://www.caisca.org/schools/international-s...
7,san francisco school,san francisco,CA,san francisco christian school,auto_seed_from_candidate_1,https://www.caisca.org/schools/the-san-francis...
8,cathedral school for boys,san francisco,CA,,,https://www.caisca.org/schools/cathedral-schoo...
9,saint andrew s episcopal school,saratoga,CA,st andrew s episcopal school,,https://www.caisca.org/schools/saint-andrews-e...



HOW TO USE:
- alias_name is what PSS likely uses for the same school.
- You can edit alias_table['alias_name'] and re-run this cell.
- We only accept matches where the resulting fallback_key maps to exactly ONE PSS school_id (unique-gated).


Alias-table fallback matches (high-confidence, unique-gated): 6

Alias matched examples (first 25):


...


=== CAIS FINAL COVERAGE v2 (cityfix + alias-table fallback) ===
CAIS total: 97
Matched final v2: 78 (80.41%)
Unmatched final v2: 19 (19.59%)

Still unmatched after alias-table (first 30):


,school_name,city,detail_url
5,bentley school,lafayette oakland,https://www.caisca.org/schools/bentley-school
10,brandeis marin,san rafael,https://www.caisca.org/schools/brandeis-marin
14,cathedral school for boys,san francisco,https://www.caisca.org/schools/cathedral-schoo...
17,chinese american international school,san francisco,https://www.caisca.org/schools/chinese-america...
21,crystal springs uplands school,belmont hillsborough,https://www.caisca.org/schools/crystal-springs...
23,east bay school,berkeley,https://www.caisca.org/schools/east-bay-school
26,escuela bilinga1 4e internacional,emeryville oakland,https://www.caisca.org/schools/escuela-bilingu...
27,field middle school,san mateo,https://www.caisca.org/schools/field-middle-sc...
36,helios school,sunnyvale,https://www.caisca.org/schools/helios-school
42,kehillah school,palo alto,https://www.caisca.org/schools/kehillah--school



Final CAIS presence table v2 shape: (77, 2)


,school_id,has_cais
0,PRI_02013539,True
1,PRI_A1300133,True
2,PRI_00083611,True
3,PRI_A0500717,True
4,PRI_A0900219,True
6,PRI_00084058,True
7,PRI_00084091,True
8,PRI_00083881,True
9,PRI_A0500178,True
11,PRI_00093539,True


=== 03.5.5 END ===


## 03.5.6b Investigate Residuals (Improved Evidence: City Explode + Region Filter + Uniqueness Gate)

**Goal:** Improve the quality of evidence for the remaining CAIS residuals that still do not match PSS after:
- **City-explode matching (03.5.2/03.5.3)** and
- **Alias-table fallback (03.5.5)**

In 03.5.6 we generated candidate matches, but some candidates were clearly **statewide noise** (e.g., matching a Bay Area school to a San Diego school).  
This section improves evidence quality by:

### Improvements in this step
1. **City candidate explode (again) for residuals**  
   Some residuals still contain multi-city/campus strings (e.g., `"lafayette oakland"`). We explode into multiple `city_candidate` rows and search with each candidate.

2. **Region filter on PSS candidates (Bay Area only)**
   We restrict candidate pools to Bay Area cities (derived from `pss_clean_df`) to prevent matches drifting to other CA regions.

3. **Uniqueness gate**
   If a residual school name maps to multiple PSS rows in the candidate pool, we do **not** auto-match it.

### Outputs
- `residual_candidates_city`: top candidates using **same-city pool** when possible
- `residual_review_v2`: one-row-per-residual triage table with best candidate + score
- `ready_for_alias_table`: high-confidence rows that are good candidates for an alias-table entry
- `needs_manual_or_ingest`: rows that still look missing from PSS (likely require CAIS-minted golden record in 03.5.7)

> **Important:** This step still does **not** automatically flip `has_cais=True`. It only improves the evidence so you can decide what to alias vs ingest.


In [1179]:
print("=== 03.5.6b START (INVESTIGATE RESIDUALS v2: CITY EXPLODE + REGION FILTER + UNIQUENESS) ===")

# -----------------------------
# Preconditions
# -----------------------------
assert "pss_clean_df" in globals(), "Need pss_clean_df."
assert "cais_final_summary_v2" in globals(), "Need cais_final_summary_v2 from 03.5.5."

# -----------------------------
# 0) Build residuals from cais_final_summary_v2 (unmatched only)
# -----------------------------
residuals = cais_final_summary_v2.loc[~cais_final_summary_v2["matched_final_v2"], [
    "school_name", "city", "state", "detail_url"
]].copy()

for c in ["school_name", "city", "state", "detail_url"]:
    residuals[c] = residuals[c].fillna("").astype(str)

print("Residual count (unmatched after 03.5.5):", residuals.shape[0])
display(residuals.head(10))

# -----------------------------
# 0b) Measure collisions: do multiple CAIS map to same PSS id?
# -----------------------------
if "school_id_final_v2" in cais_final_summary_v2.columns:
    matched_rows = cais_final_summary_v2[cais_final_summary_v2["matched_final_v2"]].copy()
    n_matched = matched_rows.shape[0]
    n_unique_pss = matched_rows["school_id_final_v2"].nunique()
    print(f"\nMatched CAIS rows: {n_matched}")
    print(f"Unique PSS school_id among matches: {n_unique_pss}")
    if n_unique_pss < n_matched:
        print("\nCAIS→PSS collisions (same PSS id used by >1 CAIS row):")
        collisions = (
            matched_rows.groupby("school_id_final_v2")
            .size()
            .reset_index(name="cais_count")
            .sort_values("cais_count", ascending=False)
        )
        display(collisions[collisions["cais_count"] > 1].head(20))
else:
    print("\nNote: school_id_final_v2 not found; skipping collision check.")

# -----------------------------
# Helpers
# -----------------------------
def strip_accents(s: str) -> str:
    if s is None:
        return ""
    s = str(s)
    return "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))

def normalize_name_soft(s: str) -> str:
    x = strip_accents(s).lower().strip()
    x = re.sub(r"^the\s+", "", x)
    x = x.replace("&", " and ")
    x = re.sub(r"[^\w\s]", " ", x)
    x = re.sub(r"\s+", " ", x).strip()
    x = re.sub(r"\b(school|schools)\b$", "", x).strip()
    return x

def normalize_city(s: str) -> str:
    x = strip_accents(s).lower().strip()
    x = re.sub(r"[^\w\s]", " ", x)
    x = re.sub(r"\s+", " ", x).strip()
    return x

def explode_city_candidates(city_raw: str) -> list[str]:
    """
    Handles commas, slashes, '&', ' and '.
    Keeps space-joined strings for later refinement against known city list.
    """
    c = normalize_city(city_raw)
    c = c.replace("/", ",")
    c = c.replace(" and ", ",")
    c = c.replace("&", ",")
    parts = [p.strip() for p in c.split(",") if p.strip()]
    if len(parts) > 1:
        return parts
    return [c] if c else [""]

def jaccard_tokens(a: str, b: str) -> float:
    ta = set(normalize_name_soft(a).split())
    tb = set(normalize_name_soft(b).split())
    if not ta or not tb:
        return 0.0
    return len(ta & tb) / len(ta | tb)

# -----------------------------
# 1) Build PSS CA pool + city set
# -----------------------------
pss_ca = pss_clean_df[pss_clean_df["state"].astype(str).str.upper().str.strip() == "CA"].copy()
pss_ca["city_norm"] = pss_ca["city"].fillna("").astype(str).apply(normalize_city)
pss_ca["name_norm"] = pss_ca["school_name"].fillna("").astype(str).apply(normalize_name_soft)

bay_area_cities = set([c for c in pss_ca["city_norm"].unique().tolist() if c])
pss_by_city = pss_ca.groupby("city_norm")

print("\nUnique PSS CA cities:", len(bay_area_cities))

# -----------------------------
# 2) Expand residuals with refined city candidates
# -----------------------------
rows = []
for idx, r in residuals.iterrows():
    city_candidates = explode_city_candidates(r["city"])

    refined = []
    for cc in city_candidates:
        cc = cc.strip()
        if not cc:
            continue

        if cc in bay_area_cities:
            refined.append(cc)
            continue

        # substring hits against known cities (handles "lafayette oakland", etc.)
        hits = [c for c in bay_area_cities if c and f" {c} " in f" {cc} "]
        refined.extend(hits if hits else [cc])

    refined = list(dict.fromkeys(refined))
    if not refined:
        refined = [""]

    for cc in refined:
        rows.append({
            "residual_row": idx,
            "school_name": r["school_name"],
            "city_raw": r["city"],
            "city_candidate": cc,
            "state": r["state"],
            "detail_url": r["detail_url"],
        })

residuals_exp = pd.DataFrame(rows)
print("\nResidual exploded rows:", residuals_exp.shape[0])
display(residuals_exp.head(20))

# -----------------------------
# 3) Candidate search
# -----------------------------
def top_candidates_for_row(school_name: str, city_candidate: str, topn: int = 5):
    if city_candidate in bay_area_cities and city_candidate in pss_by_city.groups:
        pool = pss_ca.loc[pss_by_city.groups[city_candidate], :]
        scope = "city"
    else:
        pool = pss_ca
        scope = "state"

    scored = []
    for _, pr in pool.iterrows():
        score = jaccard_tokens(school_name, pr["school_name"])
        if score <= 0:
            continue
        scored.append((score, pr["school_id"], pr["school_name"], pr["city"], pr["state"]))
    scored.sort(reverse=True, key=lambda x: x[0])
    return scope, scored[:topn]

cand_rows = []
for _, r in residuals_exp.iterrows():
    scope, top = top_candidates_for_row(r["school_name"], r["city_candidate"], topn=5)
    for rank, (score, sid, pnm, pc, ps) in enumerate(top, start=1):
        cand_rows.append({
            "cais_school_name": r["school_name"],
            "cais_city": r["city_raw"],
            "cais_state": r["state"],
            "city_candidate": r["city_candidate"],
            "candidate_scope": scope,
            "rank": rank,
            "school_id_pss": sid,
            "pss_school_name": pnm,
            "pss_city": pc,
            "pss_state": ps,
            "name_jaccard": float(score),
            "detail_url": r["detail_url"],
        })

residual_candidates_city = pd.DataFrame(cand_rows)

print("\nTop candidate matches (sample):")
display(residual_candidates_city.head(30))

# -----------------------------
# 4) Reduce to best candidate + uniqueness gate
# -----------------------------
key_cols = ["cais_school_name", "cais_city", "cais_state"]

if residual_candidates_city.empty:
    print("\nNo candidates found for any residuals. Likely missing-from-PSS cases.")
    residual_review_v2 = residuals.copy()
    residual_review_v2["triage"] = "no_candidate"
else:
    best = (
        residual_candidates_city
        .sort_values(["name_jaccard", "candidate_scope", "rank"], ascending=[False, True, True])
        .drop_duplicates(subset=key_cols, keep="first")
        .copy()
    )

    strong = residual_candidates_city[residual_candidates_city["name_jaccard"] >= 0.75].copy()
    ambig = (
        strong.groupby(key_cols)["school_id_pss"]
        .nunique()
        .reset_index(name="pss_id_count")
    )
    best = best.merge(ambig, on=key_cols, how="left")
    best["pss_id_count"] = best["pss_id_count"].fillna(1).astype(int)

    def triage_row(r):
        s = r["name_jaccard"]
        if pd.isna(s):
            return "no_candidate"
        if s >= 0.90 and r["pss_id_count"] == 1:
            return "strong_candidate_ready_for_alias_or_forced_match"
        if s >= 0.75 and r["pss_id_count"] == 1:
            return "strong_candidate_expand_alias"
        if s >= 0.55:
            return "weak_candidate_manual_review"
        return "very_weak_candidate"

    best["triage"] = best.apply(triage_row, axis=1)

    residual_review_v2 = residuals.merge(
        best.rename(columns={"cais_school_name": "school_name", "cais_city": "city", "cais_state": "state"}),
        on=["school_name", "city", "state"],
        how="left",
        suffixes=("", "_cand")
    )

cols = [
    "school_name", "city", "state", "detail_url",
    "triage", "name_jaccard", "pss_school_name", "pss_city", "school_id_pss", "candidate_scope"
]
cols = [c for c in cols if c in residual_review_v2.columns]

print("\nResidual review v2 table (scan this):")
display(residual_review_v2[cols].sort_values(["triage", "name_jaccard"], ascending=[True, False]))

# -----------------------------
# 5) Convenience outputs
# -----------------------------
ready_for_alias_table_v2 = residual_review_v2[
    residual_review_v2.get("triage", pd.Series(dtype=str)).isin([
        "strong_candidate_ready_for_alias_or_forced_match",
        "strong_candidate_expand_alias"
    ])
].copy()

needs_manual_or_ingest_v2 = residual_review_v2[
    residual_review_v2.get("triage", pd.Series(dtype=str)).isin([
        "no_candidate",
        "very_weak_candidate",
        "weak_candidate_manual_review"
    ]) | residual_review_v2.get("triage", pd.Series(dtype=str)).isna()
].copy()

print("\nReady-for-alias candidates:", ready_for_alias_table_v2.shape[0])
display(ready_for_alias_table_v2[cols].head(50))

print("\nNeeds manual review or ingest:", needs_manual_or_ingest_v2.shape[0])
display(needs_manual_or_ingest_v2[cols].head(50))

print("=== 03.5.6b END ===")


=== 03.5.6b START (INVESTIGATE RESIDUALS v2: CITY EXPLODE + REGION FILTER + UNIQUENESS) ===
Residual count (unmatched after 03.5.5): 19


,school_name,city,state,detail_url
5,bentley school,lafayette oakland,CA,https://www.caisca.org/schools/bentley-school
10,brandeis marin,san rafael,CA,https://www.caisca.org/schools/brandeis-marin
14,cathedral school for boys,san francisco,CA,https://www.caisca.org/schools/cathedral-schoo...
17,chinese american international school,san francisco,CA,https://www.caisca.org/schools/chinese-america...
21,crystal springs uplands school,belmont hillsborough,CA,https://www.caisca.org/schools/crystal-springs...
23,east bay school,berkeley,CA,https://www.caisca.org/schools/east-bay-school
26,escuela bilinga1 4e internacional,emeryville oakland,CA,https://www.caisca.org/schools/escuela-bilingu...
27,field middle school,san mateo,CA,https://www.caisca.org/schools/field-middle-sc...
36,helios school,sunnyvale,CA,https://www.caisca.org/schools/helios-school
42,kehillah school,palo alto,CA,https://www.caisca.org/schools/kehillah--school



Matched CAIS rows: 78
Unique PSS school_id among matches: 77

CAIS→PSS collisions (same PSS id used by >1 CAIS row):


,school_id_final_v2,cais_count
25,PRI_00091418,2



Unique PSS CA cities: 472

Residual exploded rows: 22


...


Ready-for-alias candidates: 0


,school_name,city,state,detail_url,triage,name_jaccard,pss_school_name,pss_city,school_id_pss,candidate_scope



Needs manual review or ingest: 19


,school_name,city,state,detail_url,triage,name_jaccard,pss_school_name,pss_city,school_id_pss,candidate_scope
0,bentley school,lafayette oakland,CA,https://www.caisca.org/schools/bentley-school,NaN,NaN,NaN,NaN,NaN,NaN
1,brandeis marin,san rafael,CA,https://www.caisca.org/schools/brandeis-marin,very_weak_candidate,0.500000,marin school,san rafael,PRI_02009442,city
2,cathedral school for boys,san francisco,CA,https://www.caisca.org/schools/cathedral-schoo...,weak_candidate_manual_review,0.600000,town school for boys,san francisco,PRI_00080858,city
3,chinese american international school,san francisco,CA,https://www.caisca.org/schools/chinese-america...,very_weak_candidate,0.250000,cumberland chinese school,san francisco,PRI_A2100336,city
4,crystal springs uplands school,belmont hillsborough,CA,https://www.caisca.org/schools/crystal-springs...,NaN,NaN,NaN,NaN,NaN,NaN
5,east bay school,berkeley,CA,https://www.caisca.org/schools/east-bay-school,very_weak_candidate,0.400000,east bay school for boys,berkeley,PRI_A1300209,city
6,escuela bilinga1 4e internacional,emeryville oakland,CA,https://www.caisca.org/schools/escuela-bilingu...,very_weak_candidate,0.400000,escuela bilingue internacional,oakland,PRI_A0770343,city
7,field middle school,san mateo,CA,https://www.caisca.org/schools/field-middle-sc...,NaN,NaN,NaN,NaN,NaN,NaN
8,helios school,sunnyvale,CA,https://www.caisca.org/schools/helios-school,NaN,NaN,NaN,NaN,NaN,NaN
9,kehillah school,palo alto,CA,https://www.caisca.org/schools/kehillah--school,very_weak_candidate,0.333333,kehillah jewish high school,palo alto,PRI_A0500422,city


=== 03.5.6b END ===


### 03.5.6c — Fix `NaN triage` + Stronger Candidate Search (Char-3gram Similarity)

In 03.5.6b, a few residual CAIS schools returned `NaN` for `triage` and had no candidate rows.  
This usually means our token-based Jaccard similarity produced **zero overlap** with all PSS school names, or the multi-city city string prevented a good candidate pool.

This cell improves residual investigation by:

1) **Stabilizing outputs**: any `NaN` triage becomes `no_candidate`.
2) **Adding a stronger similarity metric**: character 3-gram Jaccard, which is robust to ordering, abbreviations, and small formatting differences.
3) Producing a second review table `residual_review_v3` and a “best guess” candidate list for previously `NaN/no_candidate` rows.

Outputs:
- `residual_review_v3`
- `ready_for_alias_table_v3`
- `needs_manual_or_ingest_v3`


In [1182]:
print("=== 03.5.6c START (RESIDUALS v3: FIX NaN + CHAR-3GRAM CANDIDATES) ===")

import re
import unicodedata

assert "pss_clean_df" in globals(), "Need pss_clean_df."
assert "residual_review_v2" in globals(), "Run 03.5.6b first (needs residual_review_v2)."

# -----------------------------
# Helpers (reuse + new similarity)
# -----------------------------
def strip_accents(s: str) -> str:
    if s is None:
        return ""
    s = str(s)
    return "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))

def normalize_name_soft(s: str) -> str:
    x = strip_accents(s).lower().strip()
    x = re.sub(r"^the\s+", "", x)
    x = re.sub(r"&", " and ", x)
    x = re.sub(r"[^\w\s]", " ", x)
    x = re.sub(r"\s+", " ", x).strip()
    x = re.sub(r"\b(school|schools)\b$", "", x).strip()
    return x

def normalize_city(s: str) -> str:
    x = strip_accents(s).lower().strip()
    x = re.sub(r"[^\w\s]", " ", x)
    x = re.sub(r"\s+", " ", x).strip()
    return x

def char_ngrams(s: str, n: int = 3) -> set:
    s = normalize_name_soft(s).replace(" ", "")
    if len(s) < n:
        return set([s]) if s else set()
    return set(s[i:i+n] for i in range(len(s)-n+1))

def jaccard_set(a: set, b: set) -> float:
    if not a or not b:
        return 0.0
    return len(a & b) / len(a | b)

def ngram_sim(a: str, b: str, n: int = 3) -> float:
    return jaccard_set(char_ngrams(a, n=n), char_ngrams(b, n=n))

# -----------------------------
# Prep PSS CA pool
# -----------------------------
pss = pss_clean_df.copy()
pss_ca = pss[pss["state"].astype(str).str.upper().str.strip() == "CA"].copy()
pss_ca["city_norm"] = pss_ca["city"].fillna("").astype(str).apply(normalize_city)
pss_ca["name_norm"] = pss_ca["school_name"].fillna("").astype(str).apply(normalize_name_soft)

pss_by_city = pss_ca.groupby("city_norm")

# -----------------------------
# Start from v2 review table
# -----------------------------
v2 = residual_review_v2.copy()

# Fix NaN triage so downstream logic is stable
if "triage" not in v2.columns:
    v2["triage"] = np.nan
v2["triage"] = v2["triage"].fillna("no_candidate")

# Identify rows we want to “upgrade” (no candidates / very weak)
upgrade_mask = v2["triage"].isin(["no_candidate", "very_weak_candidate"])
to_upgrade = v2.loc[upgrade_mask, ["school_name", "city", "state", "detail_url"]].drop_duplicates().copy()

for c in ["school_name", "city", "state", "detail_url"]:
    to_upgrade[c] = to_upgrade[c].fillna("").astype(str)

print("Rows to upgrade with char-3gram search:", to_upgrade.shape[0])
display(to_upgrade)

# -----------------------------
# Candidate generation with char-3gram similarity
# -----------------------------
cand_rows = []
for _, r in to_upgrade.iterrows():
    nm = str(r["school_name"])
    city_raw = str(r["city"])
    city_norm = normalize_city(city_raw)
    st = str(r["state"]).strip().upper()

    # pool: same city if possible, else full CA
    if city_norm in pss_by_city.groups:
        pool = pss_ca.loc[pss_by_city.groups[city_norm], :]
        scope = "city"
    else:
        pool = pss_ca
        scope = "state"

    scores = []
    for _, pr in pool.iterrows():
        s = ngram_sim(nm, pr["school_name"], n=3)
        if s <= 0:
            continue
        scores.append((s, pr["school_id"], pr["school_name"], pr["city"], pr["state"]))

    scores.sort(reverse=True, key=lambda x: x[0])
    top = scores[:8]

    for rank, (score, sid, pnm, pc, ps) in enumerate(top, start=1):
        cand_rows.append({
            "school_name": nm,
            "city": city_raw,
            "state": st,
            "detail_url": r.get("detail_url", ""),
            "candidate_scope_v3": scope,
            "rank_v3": rank,
            "school_id_pss_v3": sid,
            "pss_school_name_v3": pnm,
            "pss_city_v3": pc,
            "name_char3_jaccard": float(score),
        })

cand_v3 = pd.DataFrame(cand_rows)

print("\nTop v3 candidates (sample):")
display(cand_v3.head(30))

# -----------------------------
# Choose best candidate per residual row
# -----------------------------
if cand_v3.empty:
    print("\nNo v3 candidates produced. These are likely missing-from-PSS or require external evidence.")
    residual_review_v3 = v2.copy()
else:
    best_v3 = (
        cand_v3.sort_values(["name_char3_jaccard", "candidate_scope_v3", "rank_v3"], ascending=[False, True, True])
        .drop_duplicates(subset=["school_name", "city", "state"], keep="first")
        .copy()
    )

    # v3 triage (char-ngrams are usually higher)
    def triage_v3(score):
        if pd.isna(score):
            return "no_candidate"
        if score >= 0.65:
            return "strong_candidate_expand_alias"
        if score >= 0.50:
            return "weak_candidate_manual_review"
        return "very_weak_candidate"

    best_v3["triage_v3"] = best_v3["name_char3_jaccard"].apply(triage_v3)

    residual_review_v3 = v2.merge(
        best_v3,
        on=["school_name", "city", "state", "detail_url"],
        how="left"
    )

    # If v3 produced something, prefer triage_v3; otherwise keep v2 triage
    residual_review_v3["triage_final"] = np.where(
        residual_review_v3["triage_v3"].notna(),
        residual_review_v3["triage_v3"],
        residual_review_v3["triage"]
    )

cols = [
    "school_name","city","state","detail_url",
    "triage","triage_v3","triage_final",
    "name_jaccard","name_char3_jaccard",
    "pss_school_name","pss_city","school_id_pss",
    "pss_school_name_v3","pss_city_v3","school_id_pss_v3"
]
cols = [c for c in cols if c in residual_review_v3.columns]

print("\nResidual review v3 (scan this):")
display(residual_review_v3[cols].sort_values(["triage_final","name_char3_jaccard"], ascending=[True, False]))

ready_for_alias_table_v3 = residual_review_v3[residual_review_v3["triage_final"] == "strong_candidate_expand_alias"].copy()
needs_manual_or_ingest_v3 = residual_review_v3[residual_review_v3["triage_final"] != "strong_candidate_expand_alias"].copy()

print("\nReady-for-alias v3:", ready_for_alias_table_v3.shape[0])
display(ready_for_alias_table_v3[cols].head(50))

print("\nNeeds manual review or ingest v3:", needs_manual_or_ingest_v3.shape[0])
display(needs_manual_or_ingest_v3[cols].head(50))

print("=== 03.5.6c END ===")


=== 03.5.6c START (RESIDUALS v3: FIX NaN + CHAR-3GRAM CANDIDATES) ===
Rows to upgrade with char-3gram search: 18


,school_name,city,state,detail_url
0,bentley school,lafayette oakland,CA,https://www.caisca.org/schools/bentley-school
1,brandeis marin,san rafael,CA,https://www.caisca.org/schools/brandeis-marin
3,chinese american international school,san francisco,CA,https://www.caisca.org/schools/chinese-america...
4,crystal springs uplands school,belmont hillsborough,CA,https://www.caisca.org/schools/crystal-springs...
5,east bay school,berkeley,CA,https://www.caisca.org/schools/east-bay-school
6,escuela bilinga1 4e internacional,emeryville oakland,CA,https://www.caisca.org/schools/escuela-bilingu...
7,field middle school,san mateo,CA,https://www.caisca.org/schools/field-middle-sc...
8,helios school,sunnyvale,CA,https://www.caisca.org/schools/helios-school
9,kehillah school,palo alto,CA,https://www.caisca.org/schools/kehillah--school
10,keys school,palo alto,CA,https://www.caisca.org/schools/keys-school



Top v3 candidates (sample):


,school_name,city,state,detail_url,candidate_scope_v3,rank_v3,school_id_pss_v3,pss_school_name_v3,pss_city_v3,name_char3_jaccard
0,bentley school,lafayette oakland,CA,https://www.caisca.org/schools/bentley-school,state,1,PRI_A0500750,valley crescent school,clovis,0.133333
1,bentley school,lafayette oakland,CA,https://www.caisca.org/schools/bentley-school,state,2,PRI_00086805,hadley school,whittier,0.125000
2,bentley school,lafayette oakland,CA,https://www.caisca.org/schools/bentley-school,state,3,PRI_00091586,valley school,van nuys,0.125000
3,bentley school,lafayette oakland,CA,https://www.caisca.org/schools/bentley-school,state,4,PRI_BB020294,wesley school,north hollywood,0.125000
4,bentley school,lafayette oakland,CA,https://www.caisca.org/schools/bentley-school,state,5,PRI_K9300140,castle elementary school,los angeles,0.117647
5,bentley school,lafayette oakland,CA,https://www.caisca.org/schools/bentley-school,state,6,PRI_00079398,buckley school,sherman oaks,0.111111
6,bentley school,lafayette oakland,CA,https://www.caisca.org/schools/bentley-school,state,7,PRI_A0500644,school for independent learners,los altos,0.111111
7,bentley school,lafayette oakland,CA,https://www.caisca.org/schools/bentley-school,state,8,PRI_A1300480,ventana school,los altos,0.111111
8,brandeis marin,san rafael,CA,https://www.caisca.org/schools/brandeis-marin,city,1,PRI_02009442,marin school,san rafael,0.272727
9,brandeis marin,san rafael,CA,https://www.caisca.org/schools/brandeis-marin,city,2,PRI_00083473,marin academy,san rafael,0.166667



Residual review v3 (scan this):


...


Ready-for-alias v3: 2


,school_name,city,state,detail_url,triage,triage_v3,triage_final,name_jaccard,name_char3_jaccard,pss_school_name,pss_city,school_id_pss,pss_school_name_v3,pss_city_v3,school_id_pss_v3
6,escuela bilinga1 4e internacional,emeryville oakland,CA,https://www.caisca.org/schools/escuela-bilingu...,very_weak_candidate,strong_candidate_expand_alias,strong_candidate_expand_alias,0.4,0.741935,escuela bilingue internacional,oakland,PRI_A0770343,escuela bilingue internacional,oakland,PRI_A0770343
17,st paul s episcopal school,oakland,CA,https://www.caisca.org/schools/st-pauls-episco...,very_weak_candidate,strong_candidate_expand_alias,strong_candidate_expand_alias,0.4,1.000000,st pauls episcopal school,oakland,PRI_00078383,st pauls episcopal school,oakland,PRI_00078383



Needs manual review or ingest v3: 17


,school_name,city,state,detail_url,triage,triage_v3,triage_final,name_jaccard,name_char3_jaccard,pss_school_name,pss_city,school_id_pss,pss_school_name_v3,pss_city_v3,school_id_pss_v3
0,bentley school,lafayette oakland,CA,https://www.caisca.org/schools/bentley-school,no_candidate,very_weak_candidate,very_weak_candidate,NaN,0.133333,NaN,NaN,NaN,valley crescent school,clovis,PRI_A0500750
1,brandeis marin,san rafael,CA,https://www.caisca.org/schools/brandeis-marin,very_weak_candidate,very_weak_candidate,very_weak_candidate,0.500000,0.272727,marin school,san rafael,PRI_02009442,marin school,san rafael,PRI_02009442
2,cathedral school for boys,san francisco,CA,https://www.caisca.org/schools/cathedral-schoo...,weak_candidate_manual_review,NaN,weak_candidate_manual_review,0.600000,NaN,town school for boys,san francisco,PRI_00080858,NaN,NaN,NaN
3,chinese american international school,san francisco,CA,https://www.caisca.org/schools/chinese-america...,very_weak_candidate,very_weak_candidate,very_weak_candidate,0.250000,0.323529,cumberland chinese school,san francisco,PRI_A2100336,la scuola international school,san francisco,PRI_A1900492
4,crystal springs uplands school,belmont hillsborough,CA,https://www.caisca.org/schools/crystal-springs...,no_candidate,very_weak_candidate,very_weak_candidate,NaN,0.250000,NaN,NaN,NaN,laurel springs school,ojai,PRI_A2103260
5,east bay school,berkeley,CA,https://www.caisca.org/schools/east-bay-school,very_weak_candidate,very_weak_candidate,very_weak_candidate,0.400000,0.277778,east bay school for boys,berkeley,PRI_A1300209,east bay school for boys,berkeley,PRI_A1300209
7,field middle school,san mateo,CA,https://www.caisca.org/schools/field-middle-sc...,no_candidate,NaN,no_candidate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,helios school,sunnyvale,CA,https://www.caisca.org/schools/helios-school,no_candidate,NaN,no_candidate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,kehillah school,palo alto,CA,https://www.caisca.org/schools/kehillah--school,very_weak_candidate,very_weak_candidate,very_weak_candidate,0.333333,0.375000,kehillah jewish high school,palo alto,PRI_A0500422,kehillah jewish high school,palo alto,PRI_A0500422
10,keys school,palo alto,CA,https://www.caisca.org/schools/keys-school,very_weak_candidate,very_weak_candidate,very_weak_candidate,0.333333,0.181818,keys family day school,palo alto,PRI_A1900484,keys family day school,palo alto,PRI_A1900484


=== 03.5.6c END ===


## 03.5.7 — Residual Alias Table v2 (Human-in-the-loop Rematch)

**Goal:** After city normalization/explode (03.5.1–03.5.3) and the main alias table (03.5.5), we still have a set of *residual* CAIS schools that are unmatched or only have weak candidates.

In 03.5.6b/03.5.6c we generated *evidence* (candidate lists + similarity scoring) to identify cases where a simple **name alias** can safely resolve a match.

This step creates a small **`alias_table_v2`** for *only* residual schools that look matchable with high confidence.

### Why this exists
Some schools fail matching because the CAIS name and the PSS name differ slightly, for example:
- punctuation / abbreviations (`st` vs `saint`)
- missing qualifiers (`episcopal`, `academy`, etc.)
- “official” listing name vs common name

### How to use
1. Review `alias_name_suggested` (if present).
2. If it looks correct, copy it into `alias_name`.
3. If you know the exact PSS spelling, overwrite `alias_name` with the **exact** `pss_clean_df.school_name`.
4. Leave `alias_name` blank if you are not sure.

### Safety / strictness rules
We apply a **uniqueness gate**:
- An alias rematch only “sticks” if the alias join key maps to **exactly one** `school_id` in PSS.
- Ambiguous matches are rejected automatically.

### Outputs
This cell updates:
- `cais_final_summary_v3` (school-level CAIS matching summary)
- `cais_presence_pss_final_v3_df` (PSS backbone presence table: `school_id`, `has_cais`)

After this, the remaining residuals should be treated as:
- **manual research targets**, or
- candidates for **Golden Record minting** (CAIS as authority record).



In [1185]:
print("=== 03.5.7 START (RESIDUAL ALIAS TABLE v2 + REMATCH) ===")

import numpy as np
import pandas as pd

# -----------------------------
# Preconditions
# -----------------------------
assert "cais_final_summary_v2" in globals(), "Need cais_final_summary_v2 from 03.5.5."
assert "pss_clean_df" in globals(), "Need pss_clean_df."
assert "ready_for_alias_table_v3" in globals(), "Need ready_for_alias_table_v3 from 03.5.6c."

# We also need your join-key normalizers from earlier steps
assert "normalize_text" in globals(), "Expected normalize_text helper from earlier steps."
assert "normalize_state" in globals(), "Expected normalize_state helper from earlier steps."

def make_join_key(name: str, city: str, state: str) -> str:
    return f"{normalize_text(name)}|{normalize_text(city)}|{normalize_state(state)}"

# -----------------------------
# 1) Build alias_table_v2 from ready_for_alias_table_v3
# -----------------------------
base = ready_for_alias_table_v3[["school_name", "city", "state", "detail_url"]].drop_duplicates().copy()
base = base.rename(columns={"school_name": "cais_school_name"})

# Suggested default alias_name from v3 candidate (if present)
if "pss_school_name_v3" in ready_for_alias_table_v3.columns:
    sugg = ready_for_alias_table_v3[["school_name", "city", "state", "pss_school_name_v3"]].drop_duplicates().copy()
    sugg = sugg.rename(columns={"school_name":"cais_school_name", "pss_school_name_v3":"alias_name_suggested"})
    base = base.merge(sugg, on=["cais_school_name","city","state"], how="left")
else:
    base["alias_name_suggested"] = ""

alias_table_v2 = base.copy()
alias_table_v2["alias_name"] = alias_table_v2["alias_name_suggested"].fillna("")
alias_table_v2["notes"] = ""

print("\n=== Editable alias_table_v2 (confirm / edit alias_name) ===")
display(alias_table_v2[["cais_school_name","city","state","detail_url","alias_name_suggested","alias_name","notes"]])

print("""
HOW TO USE:
- If alias_name_suggested is correct, keep it.
- Otherwise overwrite alias_name with the exact PSS school_name you want to match.
- Leave alias_name blank to skip (no rematch).
- Re-run this cell after edits.
""")

# -----------------------------
# 2) Apply aliases to CAIS final summary v2
# -----------------------------
cais_v3 = cais_final_summary_v2.copy()

# --- HARD GUARD: undo known bad alias from 03.5.5 (prevents collisions) ---
# If you previously forced "international school of san francisco" -> "urban school of san francisco", undo it here.
# (We only clear it; we are not trying to rematch it automatically here.)
bad_alias_overrides = {
    ("international school of san francisco", "san francisco", "CA"): "",  # clear alias
}

# build lookup key on CAIS identity
alias_table_v2["_k"] = (
    alias_table_v2["cais_school_name"].fillna("").astype(str) + "||" +
    alias_table_v2["city"].fillna("").astype(str) + "||" +
    alias_table_v2["state"].fillna("").astype(str)
)
alias_map = dict(zip(alias_table_v2["_k"], alias_table_v2["alias_name"]))

def apply_alias(row):
    k_tuple = (str(row["school_name"]).strip().lower(), str(row["city"]).strip().lower(), str(row["state"]).strip().upper())
    if k_tuple in bad_alias_overrides:
        forced = bad_alias_overrides[k_tuple]
        forced = str(forced).strip()
        return forced if forced else row["school_name"]

    k = f"{str(row['school_name'])}||{str(row['city'])}||{str(row['state'])}"
    alias = alias_map.get(k, "")
    alias = str(alias).strip()
    return alias if alias else row["school_name"]

cais_v3["school_name_alias_applied_v2"] = cais_v3.apply(apply_alias, axis=1)

# only attempt rematch for rows that are still unmatched in v2 AND have a real alias applied
cais_v3["alias_changed_v2"] = cais_v3["school_name_alias_applied_v2"] != cais_v3["school_name"]

# rebuild join key using alias-applied name (keep original city/state)
cais_v3["join_key_alias_v2"] = cais_v3.apply(
    lambda r: make_join_key(r["school_name_alias_applied_v2"], r["city"], r["state"]),
    axis=1
)

# -----------------------------
# 3) Build PSS join map and rematch (unique-gated)
# -----------------------------
pss_ca = pss_clean_df[pss_clean_df["state"].astype(str).str.upper().str.strip() == "CA"].copy()
pss_ca["join_key"] = pss_ca.apply(lambda r: make_join_key(r["school_name"], r["city"], r["state"]), axis=1)

pss_map = (
    pss_ca.groupby("join_key")
    .agg(
        pss_id_count=("school_id", "nunique"),
        school_id_pss=("school_id", "first"),
        pss_school_name=("school_name", "first"),
        pss_city=("city", "first"),
    )
    .reset_index()
)

rematch_pool = cais_v3[(~cais_v3["matched_final_v2"]) & (cais_v3["alias_changed_v2"])].copy()
print("\nRows eligible for alias rematch:", rematch_pool.shape[0])
display(rematch_pool[["school_name","school_name_alias_applied_v2","city","state","detail_url"]].head(50))

rematch = rematch_pool.merge(
    pss_map,
    left_on="join_key_alias_v2",
    right_on="join_key",
    how="left"
)

rematch["school_id_alias_v2"] = np.where(
    (rematch["school_id_pss"].notna()) & (rematch["pss_id_count"] == 1),
    rematch["school_id_pss"],
    np.nan
)

alias_matched = int(pd.Series(rematch["school_id_alias_v2"]).notna().sum())
print("\nAlias rematch high-confidence matches:", alias_matched)

print("\nAlias rematch matches (detail):")
display(
    rematch[pd.notna(rematch["school_id_alias_v2"])][
        ["school_name","school_name_alias_applied_v2","city","school_id_alias_v2","pss_school_name","pss_city","pss_id_count"]
    ].head(50)
)

# -----------------------------
# 4) Merge alias rematch back into final summary
# -----------------------------
rematch_key = ["school_name","city","state"]
rematch_idx = rematch.set_index(rematch_key)["school_id_alias_v2"]

def lookup_alias_id(row):
    return rematch_idx.get((row["school_name"], row["city"], row["state"]), np.nan)

cais_final_summary_v3 = cais_v3.copy()
cais_final_summary_v3["school_id_alias_v2"] = cais_final_summary_v3.apply(lookup_alias_id, axis=1)

# final v3 id: prefer existing v2 final, else alias match
cais_final_summary_v3["school_id_final_v3"] = cais_final_summary_v3["school_id_final_v2"].fillna(
    cais_final_summary_v3["school_id_alias_v2"]
)
cais_final_summary_v3["matched_final_v3"] = cais_final_summary_v3["school_id_final_v3"].notna()

total = cais_final_summary_v3.shape[0]
matched_v3 = int(cais_final_summary_v3["matched_final_v3"].sum())

print("\n=== CAIS FINAL COVERAGE v3 (cityfix + alias_table + residual alias v2) ===")
print(f"CAIS total: {total}")
print(f"Matched final v3: {matched_v3} ({matched_v3/total:.2%})")
print(f"Unmatched final v3: {total-matched_v3} ({(total-matched_v3)/total:.2%})")

print("\nStill unmatched after v3 (sample 30):")
display(
    cais_final_summary_v3[~cais_final_summary_v3["matched_final_v3"]][
        ["school_name","city","detail_url"]
    ].head(30)
)

# -----------------------------
# 4b) Collision check (PSS id used by >1 CAIS row)
# -----------------------------
collisions = (
    cais_final_summary_v3[cais_final_summary_v3["matched_final_v3"]]
    .groupby("school_id_final_v3")["school_name"]
    .nunique()
    .reset_index(name="cais_count")
)
collisions = collisions[collisions["cais_count"] > 1].sort_values("cais_count", ascending=False)

print("\nCollisions after v3 (should be empty or explainable):")
display(collisions.head(20))

# -----------------------------
# 5) Final presence table v3
# -----------------------------
cais_presence_pss_final_v3_df = (
    cais_final_summary_v3.loc[cais_final_summary_v3["matched_final_v3"], ["school_id_final_v3"]]
    .drop_duplicates()
    .rename(columns={"school_id_final_v3":"school_id"})
    .copy()
)
cais_presence_pss_final_v3_df["has_cais"] = True

print("\nFinal CAIS presence table v3 shape:", cais_presence_pss_final_v3_df.shape)
display(cais_presence_pss_final_v3_df.head(20))

print("=== 03.5.7 END ===")


=== 03.5.7 START (RESIDUAL ALIAS TABLE v2 + REMATCH) ===

=== Editable alias_table_v2 (confirm / edit alias_name) ===


,cais_school_name,city,state,detail_url,alias_name_suggested,alias_name,notes
0,escuela bilinga1 4e internacional,emeryville oakland,CA,https://www.caisca.org/schools/escuela-bilingu...,escuela bilingue internacional,escuela bilingue internacional,
1,st paul s episcopal school,oakland,CA,https://www.caisca.org/schools/st-pauls-episco...,st pauls episcopal school,st pauls episcopal school,



HOW TO USE:
- If alias_name_suggested is correct, keep it.
- Otherwise overwrite alias_name with the exact PSS school_name you want to match.
- Leave alias_name blank to skip (no rematch).
- Re-run this cell after edits.


Rows eligible for alias rematch: 2


,school_name,school_name_alias_applied_v2,city,state,detail_url
26,escuela bilinga1 4e internacional,escuela bilingue internacional,emeryville oakland,CA,https://www.caisca.org/schools/escuela-bilingu...
72,st paul s episcopal school,st pauls episcopal school,oakland,CA,https://www.caisca.org/schools/st-pauls-episco...



Alias rematch high-confidence matches: 1

Alias rematch matches (detail):


...


Collisions after v3 (should be empty or explainable):


,school_id_final_v3,cais_count
26,PRI_00091418,2



Final CAIS presence table v3 shape: (78, 2)


,school_id,has_cais
0,PRI_02013539,True
1,PRI_A1300133,True
2,PRI_00083611,True
3,PRI_A0500717,True
4,PRI_A0900219,True
6,PRI_00084058,True
7,PRI_00084091,True
8,PRI_00083881,True
9,PRI_A0500178,True
11,PRI_00093539,True


=== 03.5.7 END ===


In [1187]:
print("=== 03.5.7c START (RESOLVE CAIS→PSS COLLISIONS) ===")

assert "cais_final_summary_v3" in globals(), "Run 03.5.7 first (need cais_final_summary_v3)."
assert "pss_clean_df" in globals(), "Need pss_clean_df."

def strip_accents(s: str) -> str:
    if s is None:
        return ""
    s = str(s)
    return "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))

def norm_name(s: str) -> str:
    x = strip_accents(s).lower().strip()
    x = re.sub(r"^the\s+", "", x)
    x = re.sub(r"&", " and ", x)
    x = re.sub(r"[^\w\s]", " ", x)
    x = re.sub(r"\s+", " ", x).strip()
    return x

def token_jaccard(a: str, b: str) -> float:
    ta = set(norm_name(a).split())
    tb = set(norm_name(b).split())
    if not ta or not tb:
        return 0.0
    return len(ta & tb) / len(ta | tb)

c = cais_final_summary_v3.copy()

# collision ids
coll = (
    c[c["matched_final_v3"]]
    .groupby("school_id_final_v3")["school_name"]
    .nunique()
    .reset_index(name="cais_count")
)
coll = coll[coll["cais_count"] > 1].sort_values("cais_count", ascending=False)

print("Collision count:", coll.shape[0])
display(coll)

if coll.empty:
    cais_final_summary_v3_nocoll = c
    print("No collisions to resolve.")
else:
    # PSS name lookup for collided ids
    pss_lookup = (
        pss_clean_df[["school_id", "school_name", "city", "state"]]
        .drop_duplicates(subset=["school_id"])
        .rename(columns={"school_id":"school_id_final_v3", "school_name":"pss_school_name"})
    )

    c = c.merge(pss_lookup, on="school_id_final_v3", how="left")

    # for each collision id, keep the row with max name similarity to PSS
    c["_name_sim"] = c.apply(
        lambda r: token_jaccard(r["school_name"], r["pss_school_name"]) if pd.notna(r["pss_school_name"]) else 0.0,
        axis=1
    )

    to_drop = []
    for sid in coll["school_id_final_v3"].tolist():
        grp = c[(c["matched_final_v3"]) & (c["school_id_final_v3"] == sid)].copy()
        if grp.shape[0] <= 1:
            continue
        keep_idx = grp.sort_values("_name_sim", ascending=False).index[0]
        drop_idxs = [i for i in grp.index.tolist() if i != keep_idx]
        to_drop.extend(drop_idxs)

        print(f"\nCollision {sid}: keeping index={keep_idx}")
        display(grp[[
            "school_name",
            "city_x",          # CAIS city
            "detail_url",
            "pss_school_name",
            "_name_sim"
        ]].rename(columns={"city_x": "city"}))


    # clear ids for dropped rows (mark unmatched again)
    c.loc[to_drop, "school_id_final_v3"] = np.nan
    c.loc[to_drop, "matched_final_v3"] = False

    # cleanup helper cols
    c = c.drop(columns=[col for col in ["pss_school_name","city_y","state_y","_name_sim"] if col in c.columns], errors="ignore")

    cais_final_summary_v3_nocoll = c

print("\nRe-check collisions:")
coll2 = (
    cais_final_summary_v3_nocoll[cais_final_summary_v3_nocoll["matched_final_v3"]]
    .groupby("school_id_final_v3")["school_name"]
    .nunique()
    .reset_index(name="cais_count")
)
coll2 = coll2[coll2["cais_count"] > 1]
display(coll2)

print("Matched after collision-resolve:", int(cais_final_summary_v3_nocoll["matched_final_v3"].sum()), "/", cais_final_summary_v3_nocoll.shape[0])

print("=== 03.5.7c END ===")


=== 03.5.7c START (RESOLVE CAIS→PSS COLLISIONS) ===
Collision count: 1


,school_id_final_v3,cais_count
26,PRI_00091418,2



Collision PRI_00091418: keeping index=91


,school_name,city,detail_url,pss_school_name,_name_sim
38,international school of san francisco,san francisco,https://www.caisca.org/schools/international-s...,urban school of san francisco,0.666667
91,urban school of san francisco,san francisco,https://www.caisca.org/schools/urban-school-of...,urban school of san francisco,1.000000



Re-check collisions:


,school_id_final_v3,cais_count


Matched after collision-resolve: 78 / 97
=== 03.5.7c END ===


## 03.5.8 — Residual Disposition Table (Decide: Alias / Manual Match / Mint New Record)

**Goal:** We still have a small set of CAIS schools that remain unmatched after:
- **City normalization + explode** (03.5.1–03.5.3)
- **Alias table v1** (03.5.5)
- **Residual investigation** (03.5.6b / 03.5.6c)
- **Residual alias table v2** (03.5.7)

In this step, we create a single “worklist” table that makes it easy to finish the remaining cases intentionally.

### What this table answers
For each residual CAIS school, we want one of three outcomes:

1) **Alias**  
   We can match it to an existing PSS record by adding a precise alias (human-confirmed).

2) **Manual Match**  
   The model’s candidates are weak/ambiguous, but you can still match by checking evidence (website/domain, address, etc.).

3) **Mint New Record (CAIS-authority Golden Record)**  
   If PSS does not contain the school (or the match is too risky), we keep the CAIS school by creating a new backbone record (e.g., `CAIS_CA_<slug>`), marking `backbone_source="cais"`.

### Outputs
This cell produces:

- `cais_residuals_final_df`  
  The list of remaining unmatched CAIS schools after 03.5.7.

- `residual_disposition_df` *(editable)*  
  A table to drive decisions and next actions.

### How to use
- Review the `suggested_action` and the best candidate evidence.
- Fill in:
  - `decision` (alias / manual_match / mint_new_record / skip)
  - `alias_name` (if decision=alias)
  - `chosen_school_id` (if decision=manual_match and you are confident)
  - `notes`

This table becomes the “source of truth” for completing CAIS coverage without introducing risky joins.


In [1190]:
# ============================
# 03.5.8 Residual Disposition Table (Decide: Alias / Manual Match / Mint New Record)
# ============================

print("=== 03.5.8 START (RESIDUAL DISPOSITION TABLE) ===")

# ---------------------------------------------------------
# Preconditions (be forgiving about variable names)
# ---------------------------------------------------------
assert "pss_clean_df" in globals(), "Need pss_clean_df."

# Prefer latest final summary from your pipeline
summary_name = None
for candidate in [
    "cais_final_summary_v3",
    "cais_final_summary_v2",
    "cais_final_summary",
]:
    if candidate in globals():
        summary_name = candidate
        break
assert summary_name is not None, "Could not find a CAIS final summary df (expected cais_final_summary_v3 or v2)."

cais_final_summary_latest = globals()[summary_name].copy()

# Matched flag name differs across versions; detect it
matched_col = None
for c in ["matched_final_v3", "matched_final_v2", "matched_final"]:
    if c in cais_final_summary_latest.columns:
        matched_col = c
        break
assert matched_col is not None, f"{summary_name} missing a matched flag column (expected matched_final_v3/v2/final)."

# Residual review tables from 03.5.6b / 03.5.6c (optional, but highly useful)
review_v2 = globals().get("residual_review_v2", None)
review_v3 = globals().get("residual_review_v3", None)

# ---------------------------------------------------------
# 0) Build residual list after latest matching
# ---------------------------------------------------------
cais_residuals_final_df = cais_final_summary_latest[~cais_final_summary_latest[matched_col]].copy()

need_cols = ["school_name", "city", "state", "detail_url"]
for c in need_cols:
    if c not in cais_residuals_final_df.columns:
        cais_residuals_final_df[c] = ""

cais_residuals_final_df = cais_residuals_final_df[need_cols].copy()
cais_residuals_final_df["school_name"] = cais_residuals_final_df["school_name"].fillna("").astype(str)
cais_residuals_final_df["city"] = cais_residuals_final_df["city"].fillna("").astype(str)
cais_residuals_final_df["state"] = cais_residuals_final_df["state"].fillna("").astype(str)

print(f"Using summary: {summary_name} | matched flag: {matched_col}")
print("Residual CAIS count (post-latest matching):", cais_residuals_final_df.shape[0])
display(cais_residuals_final_df)

# ---------------------------------------------------------
# Helpers (for consistent keys)
# ---------------------------------------------------------
def strip_accents(s: str) -> str:
    if s is None:
        return ""
    s = str(s)
    return "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))

def norm_city(s: str) -> str:
    x = strip_accents(s).lower().strip()
    x = re.sub(r"[^\w\s]", " ", x)
    x = re.sub(r"\s+", " ", x).strip()
    return x

def norm_name(s: str) -> str:
    x = strip_accents(s).lower().strip()
    x = re.sub(r"^the\s+", "", x)
    x = re.sub(r"&", " and ", x)
    x = re.sub(r"[^\w\s]", " ", x)
    x = re.sub(r"\s+", " ", x).strip()
    return x

# ---------------------------------------------------------
# 1) Pull “best evidence” from residual_review_v3 (preferred) or v2
#    We only use this to pre-fill candidate columns — not to auto-match.
# ---------------------------------------------------------
def pick_review_df():
    if isinstance(review_v3, pd.DataFrame) and not review_v3.empty:
        return "v3", review_v3
    if isinstance(review_v2, pd.DataFrame) and not review_v2.empty:
        return "v2", review_v2
    return None, None

review_version, review_df = pick_review_df()

evidence_cols = [
    "triage_final", "triage_v3", "triage",
    "name_jaccard", "name_char3_jaccard",
    "pss_school_name", "pss_city", "school_id_pss",
    "pss_school_name_v3", "pss_city_v3", "school_id_pss_v3",
]

evidence = None
if review_df is not None:
    # Normalize join keys to be resilient
    tmp = review_df.copy()
    for col in ["school_name", "city", "state"]:
        if col not in tmp.columns:
            tmp[col] = ""
    tmp["k_school"] = tmp["school_name"].fillna("").astype(str).apply(norm_name)
    tmp["k_city"] = tmp["city"].fillna("").astype(str).apply(norm_city)
    tmp["k_state"] = tmp["state"].fillna("").astype(str).str.upper().str.strip()

    # Keep only one row per residual key (review already mostly 1 row per school)
    keep = ["k_school", "k_city", "k_state"] + [c for c in evidence_cols if c in tmp.columns]
    evidence = tmp[keep].drop_duplicates(subset=["k_school", "k_city", "k_state"]).copy()

# ---------------------------------------------------------
# 2) Create the editable disposition table
# ---------------------------------------------------------
disp = cais_residuals_final_df.copy()
disp["k_school"] = disp["school_name"].apply(norm_name)
disp["k_city"] = disp["city"].apply(norm_city)
disp["k_state"] = disp["state"].astype(str).str.upper().str.strip()

if evidence is not None:
    disp = disp.merge(evidence, on=["k_school", "k_city", "k_state"], how="left")

# Candidate pick: prefer v3 fields if present, else v2
def coalesce_cols(df, a, b, out):
    if a in df.columns and b in df.columns:
        df[out] = df[a].combine_first(df[b])
    elif a in df.columns:
        df[out] = df[a]
    elif b in df.columns:
        df[out] = df[b]
    else:
        df[out] = np.nan

coalesce_cols(disp, "school_id_pss_v3", "school_id_pss", "best_school_id_candidate")
coalesce_cols(disp, "pss_school_name_v3", "pss_school_name", "best_pss_name_candidate")
coalesce_cols(disp, "pss_city_v3", "pss_city", "best_pss_city_candidate")
coalesce_cols(disp, "triage_final", "triage", "triage_best")

# Suggested action (light guidance only)
def suggest_action(triage_val):
    t = str(triage_val) if not pd.isna(triage_val) else ""
    if "strong_candidate" in t:
        return "alias"
    if "weak_candidate" in t:
        return "manual_match"
    if "no_candidate" in t or "very_weak" in t or t == "":
        return "mint_new_record"
    return "manual_match"

disp["suggested_action"] = disp["triage_best"].apply(suggest_action)

# Editable fields
disp["decision"] = ""           # alias | manual_match | mint_new_record | skip
disp["alias_name"] = ""         # exact PSS school_name (if decision=alias)
disp["chosen_school_id"] = ""   # direct override to PSS school_id (if decision=manual_match)
disp["notes"] = ""              # free text

# Final column order (human-friendly)
cols_order = [
    "school_name", "city", "state", "detail_url",
    "triage_best", "suggested_action",
    "best_pss_name_candidate", "best_pss_city_candidate", "best_school_id_candidate",
    "name_jaccard", "name_char3_jaccard",
    "decision", "alias_name", "chosen_school_id", "notes",
]
cols_order = [c for c in cols_order if c in disp.columns]

residual_disposition_df = disp[cols_order].sort_values(["suggested_action", "school_name"]).reset_index(drop=True)

print(f"\nEvidence source: {review_version if review_version else 'none'}")
print("=== Editable residual_disposition_df (fill decision / alias_name / chosen_school_id) ===")
display(residual_disposition_df)

print("""
HOW TO USE:
- decision:
    - 'alias'          -> fill alias_name (exact PSS school_name)
    - 'manual_match'   -> fill chosen_school_id (exact PSS school_id)
    - 'mint_new_record'-> keep CAIS as authority backbone (handled in 03.5.9)
    - 'skip'           -> ignore for now
- suggested_action is only guidance; you choose.
""")

print("=== 03.5.8 END ===")


=== 03.5.8 START (RESIDUAL DISPOSITION TABLE) ===
Using summary: cais_final_summary_v3 | matched flag: matched_final_v3
Residual CAIS count (post-latest matching): 18


,school_name,city,state,detail_url
5,bentley school,lafayette oakland,CA,https://www.caisca.org/schools/bentley-school
10,brandeis marin,san rafael,CA,https://www.caisca.org/schools/brandeis-marin
14,cathedral school for boys,san francisco,CA,https://www.caisca.org/schools/cathedral-schoo...
17,chinese american international school,san francisco,CA,https://www.caisca.org/schools/chinese-america...
21,crystal springs uplands school,belmont hillsborough,CA,https://www.caisca.org/schools/crystal-springs...
23,east bay school,berkeley,CA,https://www.caisca.org/schools/east-bay-school
26,escuela bilinga1 4e internacional,emeryville oakland,CA,https://www.caisca.org/schools/escuela-bilingu...
27,field middle school,san mateo,CA,https://www.caisca.org/schools/field-middle-sc...
36,helios school,sunnyvale,CA,https://www.caisca.org/schools/helios-school
42,kehillah school,palo alto,CA,https://www.caisca.org/schools/kehillah--school



Evidence source: v3
=== Editable residual_disposition_df (fill decision / alias_name / chosen_school_id) ===


,school_name,city,state,detail_url,triage_best,suggested_action,best_pss_name_candidate,best_pss_city_candidate,best_school_id_candidate,name_jaccard,name_char3_jaccard,decision,alias_name,chosen_school_id,notes
0,escuela bilinga1 4e internacional,emeryville oakland,CA,https://www.caisca.org/schools/escuela-bilingu...,strong_candidate_expand_alias,alias,escuela bilingue internacional,oakland,PRI_A0770343,0.400000,0.741935,,,,
1,bentley school,lafayette oakland,CA,https://www.caisca.org/schools/bentley-school,very_weak_candidate,manual_match,valley crescent school,clovis,PRI_A0500750,NaN,0.133333,,,,
2,brandeis marin,san rafael,CA,https://www.caisca.org/schools/brandeis-marin,very_weak_candidate,manual_match,marin school,san rafael,PRI_02009442,0.500000,0.272727,,,,
3,cathedral school for boys,san francisco,CA,https://www.caisca.org/schools/cathedral-schoo...,weak_candidate_manual_review,manual_match,town school for boys,san francisco,PRI_00080858,0.600000,NaN,,,,
4,chinese american international school,san francisco,CA,https://www.caisca.org/schools/chinese-america...,very_weak_candidate,manual_match,la scuola international school,san francisco,PRI_A1900492,0.250000,0.323529,,,,
5,crystal springs uplands school,belmont hillsborough,CA,https://www.caisca.org/schools/crystal-springs...,very_weak_candidate,manual_match,laurel springs school,ojai,PRI_A2103260,NaN,0.250000,,,,
6,east bay school,berkeley,CA,https://www.caisca.org/schools/east-bay-school,very_weak_candidate,manual_match,east bay school for boys,berkeley,PRI_A1300209,0.400000,0.277778,,,,
7,kehillah school,palo alto,CA,https://www.caisca.org/schools/kehillah--school,very_weak_candidate,manual_match,kehillah jewish high school,palo alto,PRI_A0500422,0.333333,0.375000,,,,
8,keys school,palo alto,CA,https://www.caisca.org/schools/keys-school,very_weak_candidate,manual_match,keys family day school,palo alto,PRI_A1900484,0.333333,0.181818,,,,
9,lick wilmerding high school,san francisco,CA,https://www.caisca.org/schools/lick-wilmerding...,very_weak_candidate,manual_match,archbishop riordan high school,san francisco,PRI_00072596,0.200000,0.060606,,,,



HOW TO USE:
- decision:
    - 'alias'          -> fill alias_name (exact PSS school_name)
    - 'manual_match'   -> fill chosen_school_id (exact PSS school_id)
    - 'mint_new_record'-> keep CAIS as authority backbone (handled in 03.5.9)
    - 'skip'           -> ignore for now
- suggested_action is only guidance; you choose.

=== 03.5.8 END ===


In [1192]:
# ============================
# 03.5.8b Auto-fill residual disposition decisions (safe default)
# ============================

print("=== 03.5.8b START (AUTO-FILL DECISIONS) ===")

import numpy as np
import pandas as pd

assert "residual_disposition_df" in globals(), "Need residual_disposition_df from 03.5.8."

df = residual_disposition_df.copy()

# normalize helpers
def _norm(x):
    return "" if pd.isna(x) else str(x).strip()

for c in ["decision", "suggested_action", "school_name", "city", "state", "alias_name", "chosen_school_id"]:
    if c not in df.columns:
        df[c] = ""
    df[c] = df[c].apply(_norm)

blank_before = (df["decision"] == "").sum()
print("Blank decisions BEFORE:", blank_before, "of", df.shape[0])

# Only fill blanks; never overwrite your manual work
mask_blank = df["decision"] == ""

# ----------------------------
# Optional: hard override known-good forced match (bypasses city mismatch issues)
# ----------------------------
# If you want this OFF, just comment these 3 lines out.
mask_escuela = (
    mask_blank &
    (df["school_name"].str.lower() == "escuela bilinga1 4e internacional") &
    (df["state"].str.upper() == "CA")
)
df.loc[mask_escuela, "decision"] = "manual_match"
df.loc[mask_escuela, "chosen_school_id"] = "PRI_A0770343"
df.loc[mask_escuela, "notes"] = (df.get("notes", "").apply(_norm) + " | forced match: CAIS encoding/city multi-token").str.strip(" |")

# Recompute blank mask after override
mask_blank = df["decision"] == ""

# ----------------------------
# Rule 1: suggested alias -> decision=alias, but only if we have a plausible alias target
# (prevents alias decisions with no target)
# ----------------------------
has_alias_target = (
    df.get("best_pss_name_candidate", pd.Series([""]*len(df))).fillna("").astype(str).str.strip() != ""
) | (
    df.get("best_school_id_candidate", pd.Series([""]*len(df))).fillna("").astype(str).str.strip() != ""
)

df.loc[mask_blank & (df["suggested_action"] == "alias") & has_alias_target, "decision"] = "alias"

# ----------------------------
# Rule 2: suggested mint -> decision=mint_new_record
# ----------------------------
df.loc[mask_blank & (df["suggested_action"] == "mint_new_record"), "decision"] = "mint_new_record"

# ----------------------------
# Rule 3 (conservative default): everything else -> mint_new_record
# ----------------------------
df.loc[mask_blank & (df["decision"] == ""), "decision"] = "mint_new_record"

blank_after = (df["decision"] == "").sum()
print("Blank decisions AFTER:", blank_after, "of", df.shape[0])

print("\nDecision counts:")
display(df["decision"].value_counts(dropna=False).reset_index().rename(columns={"index":"decision","decision":"count"}))

# Write back to the global so 03.5.9 uses it
residual_disposition_df = df

print("\nSample (first 12):")
display(residual_disposition_df.head(12))

print("=== 03.5.8b END ===")


=== 03.5.8b START (AUTO-FILL DECISIONS) ===
Blank decisions BEFORE: 18 of 18
Blank decisions AFTER: 0 of 18

Decision counts:


,count,count
0,mint_new_record,17
1,manual_match,1



Sample (first 12):


,school_name,city,state,detail_url,triage_best,suggested_action,best_pss_name_candidate,best_pss_city_candidate,best_school_id_candidate,name_jaccard,name_char3_jaccard,decision,alias_name,chosen_school_id,notes
0,escuela bilinga1 4e internacional,emeryville oakland,CA,https://www.caisca.org/schools/escuela-bilingu...,strong_candidate_expand_alias,alias,escuela bilingue internacional,oakland,PRI_A0770343,0.400000,0.741935,manual_match,,PRI_A0770343,forced match: CAIS encoding/city multi-token
1,bentley school,lafayette oakland,CA,https://www.caisca.org/schools/bentley-school,very_weak_candidate,manual_match,valley crescent school,clovis,PRI_A0500750,NaN,0.133333,mint_new_record,,,
2,brandeis marin,san rafael,CA,https://www.caisca.org/schools/brandeis-marin,very_weak_candidate,manual_match,marin school,san rafael,PRI_02009442,0.500000,0.272727,mint_new_record,,,
3,cathedral school for boys,san francisco,CA,https://www.caisca.org/schools/cathedral-schoo...,weak_candidate_manual_review,manual_match,town school for boys,san francisco,PRI_00080858,0.600000,NaN,mint_new_record,,,
4,chinese american international school,san francisco,CA,https://www.caisca.org/schools/chinese-america...,very_weak_candidate,manual_match,la scuola international school,san francisco,PRI_A1900492,0.250000,0.323529,mint_new_record,,,
5,crystal springs uplands school,belmont hillsborough,CA,https://www.caisca.org/schools/crystal-springs...,very_weak_candidate,manual_match,laurel springs school,ojai,PRI_A2103260,NaN,0.250000,mint_new_record,,,
6,east bay school,berkeley,CA,https://www.caisca.org/schools/east-bay-school,very_weak_candidate,manual_match,east bay school for boys,berkeley,PRI_A1300209,0.400000,0.277778,mint_new_record,,,
7,kehillah school,palo alto,CA,https://www.caisca.org/schools/kehillah--school,very_weak_candidate,manual_match,kehillah jewish high school,palo alto,PRI_A0500422,0.333333,0.375000,mint_new_record,,,
8,keys school,palo alto,CA,https://www.caisca.org/schools/keys-school,very_weak_candidate,manual_match,keys family day school,palo alto,PRI_A1900484,0.333333,0.181818,mint_new_record,,,
9,lick wilmerding high school,san francisco,CA,https://www.caisca.org/schools/lick-wilmerding...,very_weak_candidate,manual_match,archbishop riordan high school,san francisco,PRI_00072596,0.200000,0.060606,mint_new_record,,,


=== 03.5.8b END ===


In [1194]:
# Patch: force escuela -> manual_match (avoids city mismatch killing alias join)
df = residual_disposition_df.copy()

m = df["school_name"].str.lower().eq("escuela bilinga1 4e internacional")
df.loc[m, "decision"] = "manual_match"
df.loc[m, "chosen_school_id"] = "PRI_A0770343"
df.loc[m, "notes"] = (df.loc[m, "notes"].fillna("").astype(str) + " | forced: multi-city string blocks alias join").str.strip(" |")

residual_disposition_df = df
display(residual_disposition_df[m])


,school_name,city,state,detail_url,triage_best,suggested_action,best_pss_name_candidate,best_pss_city_candidate,best_school_id_candidate,name_jaccard,name_char3_jaccard,decision,alias_name,chosen_school_id,notes
0,escuela bilinga1 4e internacional,emeryville oakland,CA,https://www.caisca.org/schools/escuela-bilingu...,strong_candidate_expand_alias,alias,escuela bilingue internacional,oakland,PRI_A0770343,0.4,0.741935,manual_match,,PRI_A0770343,forced match: CAIS encoding/city multi-token |...


## 03.5.9 — Mint CAIS Backbone Records (CAIS as Authority)

At this point, CAIS matching is as complete as we can make it using:
- city normalization
- alias tables
- similarity-based investigation

For any remaining CAIS schools that still cannot be matched to a PSS `school_id`, we mint a deterministic new `school_id` and treat CAIS as the backbone authority record (v1 MVP behavior).  

This step outputs:
- `cais_backbone_minted_df` — new backbone rows for schools not found in PSS  
- `cais_forced_matches_df` — a mapping for forced manual / alias matches to existing PSS IDs  
- `cais_presence_final_v4_df` — final `school_id` + `has_cais` presence table including:
  - PSS matches from `cais_final_summary_v3`
  - forced matches from this step
  - minted CAIS-only school_ids


In [1197]:
# ============================
# 03.5.9 Mint New Backbone Records from CAIS Residuals (CAIS as Authority)
# ============================

print("=== 03.5.9 START (MINT CAIS BACKBONE RECORDS) ===")

import hashlib

# ---------------------------------------------------------
# Preconditions
# ---------------------------------------------------------
assert "residual_disposition_df" in globals(), "Need residual_disposition_df from 03.5.8."
assert "pss_clean_df" in globals(), "Need pss_clean_df."
assert "cais_final_summary_v3" in globals(), "Need cais_final_summary_v3 (latest summary) from 03.5.7+."

disp = residual_disposition_df.copy()
pss = pss_clean_df.copy()

# Prefer collision-resolved summary if you created one in 03.5.7c
cais_sum = cais_final_summary_v3.copy()
for nm in ["cais_final_summary_v3_resolved", "cais_final_summary_v3_post_collision", "cais_final_summary_v3_fixed"]:
    if nm in globals():
        cais_sum = globals()[nm].copy()
        print("Using collision-resolved summary:", nm)
        break

# Ensure required columns exist
required_disp_cols = ["school_name", "city", "state", "detail_url", "decision", "alias_name", "chosen_school_id", "notes"]
for c in required_disp_cols:
    if c not in disp.columns:
        disp[c] = ""

# Make these safe string columns (avoids dtype warnings later)
for c in ["decision", "alias_name", "chosen_school_id", "notes"]:
    disp[c] = disp[c].fillna("").astype(str)

# ---------------------------------------------------------
# OPTIONAL: Big-name forced PSS matches (manual overrides)
#   Put these here so they affect decision_norm / minting logic below
# ---------------------------------------------------------
def _force_manual_match(disp_df: pd.DataFrame, school_lc: str, city_lc: str, state_uc: str, pss_id: str) -> None:
    m = (
        disp_df["school_name"].astype(str).str.lower().str.strip().eq(school_lc)
        & disp_df["city"].astype(str).str.lower().str.strip().eq(city_lc)
        & disp_df["state"].astype(str).str.upper().str.strip().eq(state_uc)
    )
    if m.any():
        disp_df.loc[m, "decision"] = "manual_match"
        disp_df.loc[m, "chosen_school_id"] = pss_id
        disp_df.loc[m, "alias_name"] = ""
        disp_df.loc[m, "notes"] = (disp_df.loc[m, "notes"].fillna("") + f" | forced to PSS {pss_id}").str.strip(" |")
        print(f"Patched {school_lc} ({city_lc},{state_uc}) -> manual_match {pss_id}")
        display(disp_df.loc[m, ["school_name","city","state","decision","chosen_school_id","notes"]])
    else:
        print(f"NOTE: '{school_lc}' ({city_lc},{state_uc}) not found in residuals. No patch applied.")

_force_manual_match(disp, "kehillah school", "palo alto", "CA", "PRI_A0500422")
_force_manual_match(disp, "nueva school", "hillsborough", "CA", "PRI_A1790096")

# ---------------------------------------------------------
# OPTIONAL: Patch Convent & Stuart Hall -> forced PSS match (if it ever appears in residuals)
# ---------------------------------------------------------
mask = disp["school_name"].astype(str).str.contains("convent and stuart hall", case=False, na=False)
if mask.any():
    disp.loc[mask, "decision"] = "manual_match"
    disp.loc[mask, "chosen_school_id"] = "PRI_01608968"
    disp.loc[mask, "alias_name"] = ""
    disp.loc[mask, "notes"] = (disp.loc[mask, "notes"].fillna("") + " | forced to PSS PRI_01608968").str.strip(" |")
    print("Patched Convent & Stuart Hall -> manual_match PRI_01608968")
    display(disp.loc[mask, ["school_name", "city", "state", "decision", "chosen_school_id", "notes"]])

# ---------------------------------------------------------
# Helpers
# ---------------------------------------------------------
def strip_accents(s: str) -> str:
    if s is None:
        return ""
    s = str(s)
    return "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))

def slugify(s: str) -> str:
    x = strip_accents(s).lower().strip()
    x = re.sub(r"&", " and ", x)
    x = re.sub(r"[^a-z0-9]+", "-", x)
    x = re.sub(r"-+", "-", x).strip("-")
    return x

def mint_cais_school_id(name: str, city: str, state: str) -> str:
    base = f"{slugify(name)}-{slugify(city)}-{slugify(state)}".strip("-")
    h = hashlib.sha1(base.encode("utf-8")).hexdigest()[:8]
    # keep id readable + deterministic (cap slug portion)
    return f"CAIS_{slugify(state)[:2].upper()}_{base[:48]}_{h}"

# ---------------------------------------------------------
# 0) Normalize decisions
# ---------------------------------------------------------
disp["decision_norm"] = disp["decision"].fillna("").astype(str).str.strip().str.lower()

valid_decisions = {"alias", "manual_match", "mint_new_record", "skip", ""}
bad = sorted(set(disp["decision_norm"].unique()) - valid_decisions)
if bad:
    print("WARNING: unexpected decision values found:", bad)

# ---------------------------------------------------------
# 1) Minted records (decision = mint_new_record)
# ---------------------------------------------------------
to_mint = disp[disp["decision_norm"] == "mint_new_record"].copy()
print("Rows to mint:", to_mint.shape[0])

if to_mint.empty:
    cais_backbone_minted_df = pd.DataFrame()
    print("No rows marked mint_new_record. (If unexpected: fill residual_disposition_df['decision'] first.)")
else:
    to_mint["school_id"] = to_mint.apply(
        lambda r: mint_cais_school_id(r["school_name"], r["city"], r["state"]),
        axis=1
    )

    cais_backbone_minted_df = (
        to_mint[["school_id", "school_name", "city", "state", "detail_url"]]
        .rename(columns={"school_name": "name"})
        .copy()
    )

    cais_backbone_minted_df["backbone_source"] = "cais"
    cais_backbone_minted_df["is_private"] = True
    cais_backbone_minted_df["has_cais"] = True
    cais_backbone_minted_df["nces_id"] = np.nan
    cais_backbone_minted_df["ppin"] = np.nan

    print("\nMinted CAIS backbone rows:")
    display(cais_backbone_minted_df)

# ---------------------------------------------------------
# 2) Forced matches (decision = alias or manual_match)
# ---------------------------------------------------------
forced = disp[disp["decision_norm"].isin(["alias", "manual_match"])].copy()
print("\nRows marked alias/manual_match:", forced.shape[0])

# Make sure output columns are string-safe
forced["alias_name"] = forced["alias_name"].fillna("").astype(str).str.strip()
forced["chosen_school_id"] = forced["chosen_school_id"].fillna("").astype(str).str.strip()

# Create school_id_forced as object dtype (prevents FutureWarning)
forced["school_id_forced"] = pd.Series([None] * len(forced), dtype="object")

# manual_match: chosen_school_id wins
mask_manual = forced["decision_norm"] == "manual_match"
forced.loc[mask_manual & (forced["chosen_school_id"] != ""), "school_id_forced"] = forced.loc[
    mask_manual & (forced["chosen_school_id"] != ""), "chosen_school_id"
].values

# Validate manual_match IDs exist in PSS (helps catch typos)
pss_ids = set(pss["school_id"].dropna().astype(str))
bad_manual = forced.loc[mask_manual & (forced["school_id_forced"].notna()), "school_id_forced"].astype(str)
bad_manual = bad_manual[~bad_manual.isin(pss_ids)]
if len(bad_manual) > 0:
    print("WARNING: manual_match chosen_school_id not found in PSS:", bad_manual.tolist())

# alias: match alias_name to exact PSS school_name with uniqueness gate
mask_alias = (
    (forced["decision_norm"] == "alias")
    & (forced["school_id_forced"].isna())
    & (forced["alias_name"] != "")
)

# Build PSS exact-name unique map
pss_name_map = (
    pss[["school_id", "school_name", "city", "state"]]
    .dropna(subset=["school_id", "school_name"])
    .copy()
)
pss_name_map["school_name_exact"] = pss_name_map["school_name"].astype(str)

name_counts = (
    pss_name_map.groupby("school_name_exact")["school_id"]
    .nunique()
    .reset_index(name="pss_name_id_count")
)

pss_name_unique = (
    pss_name_map.merge(name_counts, on="school_name_exact", how="left")
    .query("pss_name_id_count == 1")
    .drop_duplicates(subset=["school_name_exact"])
    .copy()
)

if mask_alias.any():
    alias_join = forced.loc[mask_alias, ["alias_name"]].merge(
        pss_name_unique[["school_name_exact", "school_id"]],
        left_on="alias_name",
        right_on="school_name_exact",
        how="left"
    )
    forced.loc[mask_alias, "school_id_forced"] = alias_join["school_id"].values

# Keep mapping table
cais_forced_matches_df = forced[[
    "school_name", "city", "state", "detail_url",
    "decision", "alias_name", "chosen_school_id",
    "school_id_forced", "notes"
]].copy()

print("\nForced match mapping (review):")
display(cais_forced_matches_df)

resolved_forced = cais_forced_matches_df[cais_forced_matches_df["school_id_forced"].notna()].copy()
unresolved_forced = cais_forced_matches_df[cais_forced_matches_df["school_id_forced"].isna()].copy()

print("\nResolved forced matches:", resolved_forced.shape[0])
print("Unresolved forced rows (fix decision/alias_name/chosen_school_id):", unresolved_forced.shape[0])
if not unresolved_forced.empty:
    display(unresolved_forced[["school_name", "city", "decision", "alias_name", "chosen_school_id", "notes"]].head(50))

# ---------------------------------------------------------
# 3) Presence table outputs
#   A) PSS IDs already matched in (collision-safe) summary
#   B) forced matches to PSS IDs
#   C) minted CAIS-only IDs
# ---------------------------------------------------------

# A) from summary (USE FINAL ID)
assert "matched_final_v3" in cais_sum.columns, "cais summary must include matched_final_v3"
assert "school_id_final_v3" in cais_sum.columns, "cais summary must include school_id_final_v3"

presence_v3 = cais_sum.loc[cais_sum["matched_final_v3"], ["school_id_final_v3"]].copy()
presence_v3 = presence_v3.rename(columns={"school_id_final_v3": "school_id"})
presence_v3 = presence_v3.dropna().drop_duplicates(subset=["school_id"])
presence_v3["has_cais"] = True

# B) forced to PSS
presence_forced = resolved_forced[["school_id_forced"]].rename(columns={"school_id_forced": "school_id"}).copy()
presence_forced = presence_forced.dropna().drop_duplicates(subset=["school_id"])
presence_forced["has_cais"] = True

# C) minted
if "cais_backbone_minted_df" in globals() and isinstance(cais_backbone_minted_df, pd.DataFrame) and not cais_backbone_minted_df.empty:
    presence_minted = cais_backbone_minted_df[["school_id"]].dropna().drop_duplicates(subset=["school_id"]).copy()
    presence_minted["has_cais"] = True
else:
    presence_minted = pd.DataFrame(columns=["school_id", "has_cais"])

# Combine
cais_presence_final_v4_df = pd.concat([presence_v3, presence_forced, presence_minted], ignore_index=True)
cais_presence_final_v4_df = cais_presence_final_v4_df.drop_duplicates(subset=["school_id"]).reset_index(drop=True)

print("\nFinal CAIS presence table v4 shape:", cais_presence_final_v4_df.shape)
display(cais_presence_final_v4_df.head(25))

# OPTIONAL: persist patched disposition back to globals for later cells
residual_disposition_df = disp.copy()

print("=== 03.5.9 END ===")


=== 03.5.9 START (MINT CAIS BACKBONE RECORDS) ===
Patched kehillah school (palo alto,CA) -> manual_match PRI_A0500422


,school_name,city,state,decision,chosen_school_id,notes
7,kehillah school,palo alto,CA,manual_match,PRI_A0500422,forced to PSS PRI_A0500422


Patched nueva school (hillsborough,CA) -> manual_match PRI_A1790096


,school_name,city,state,decision,chosen_school_id,notes
13,nueva school,hillsborough,CA,manual_match,PRI_A1790096,forced to PSS PRI_A1790096


Rows to mint: 15

Minted CAIS backbone rows:


...


Rows marked alias/manual_match: 3

Forced match mapping (review):


,school_name,city,state,detail_url,decision,alias_name,chosen_school_id,school_id_forced,notes
0,escuela bilinga1 4e internacional,emeryville oakland,CA,https://www.caisca.org/schools/escuela-bilingu...,manual_match,,PRI_A0770343,PRI_A0770343,forced match: CAIS encoding/city multi-token |...
7,kehillah school,palo alto,CA,https://www.caisca.org/schools/kehillah--school,manual_match,,PRI_A0500422,PRI_A0500422,forced to PSS PRI_A0500422
13,nueva school,hillsborough,CA,https://www.caisca.org/schools/the-nueva-school,manual_match,,PRI_A1790096,PRI_A1790096,forced to PSS PRI_A1790096



Resolved forced matches: 3
Unresolved forced rows (fix decision/alias_name/chosen_school_id): 0

Final CAIS presence table v4 shape: (96, 2)


,school_id,has_cais
0,PRI_02013539,True
1,PRI_A1300133,True
2,PRI_00083611,True
3,PRI_A0500717,True
4,PRI_A0900219,True
5,PRI_00084058,True
6,PRI_00084091,True
7,PRI_00083881,True
8,PRI_A0500178,True
9,PRI_00093539,True


=== 03.5.9 END ===


In [1199]:
# sanity check: minted IDs that look like big-name schools
print("Still mint_new_record after patches:")
display(
    disp.loc[disp["decision"].astype(str).str.lower().str.strip().eq("mint_new_record"),
             ["school_name","city","state","detail_url","decision","chosen_school_id","notes"]]
    .head(50)
)

Still mint_new_record after patches:


,school_name,city,state,detail_url,decision,chosen_school_id,notes
1,bentley school,lafayette oakland,CA,https://www.caisca.org/schools/bentley-school,mint_new_record,,
2,brandeis marin,san rafael,CA,https://www.caisca.org/schools/brandeis-marin,mint_new_record,,
3,cathedral school for boys,san francisco,CA,https://www.caisca.org/schools/cathedral-schoo...,mint_new_record,,
4,chinese american international school,san francisco,CA,https://www.caisca.org/schools/chinese-america...,mint_new_record,,
5,crystal springs uplands school,belmont hillsborough,CA,https://www.caisca.org/schools/crystal-springs...,mint_new_record,,
6,east bay school,berkeley,CA,https://www.caisca.org/schools/east-bay-school,mint_new_record,,
8,keys school,palo alto,CA,https://www.caisca.org/schools/keys-school,mint_new_record,,
9,lick wilmerding high school,san francisco,CA,https://www.caisca.org/schools/lick-wilmerding...,mint_new_record,,
10,lyca e frana ais de san francisco,san francisco,CA,https://www.caisca.org/schools/lycee-francais-...,mint_new_record,,
11,millennium school,san francisco,CA,https://www.caisca.org/schools/millennium-school,mint_new_record,,


In [1201]:
# A) If you already have residual_disposition_df:
mint_df = residual_disposition_df.query("decision == 'mint_new_record'").copy()

# Keep only what we need
mint_df_min = mint_df[["school_name", "city", "state", "detail_url"]].copy()

display(mint_df_min)
print("Count:", len(mint_df_min))


,school_name,city,state,detail_url
1,bentley school,lafayette oakland,CA,https://www.caisca.org/schools/bentley-school
2,brandeis marin,san rafael,CA,https://www.caisca.org/schools/brandeis-marin
3,cathedral school for boys,san francisco,CA,https://www.caisca.org/schools/cathedral-schoo...
4,chinese american international school,san francisco,CA,https://www.caisca.org/schools/chinese-america...
5,crystal springs uplands school,belmont hillsborough,CA,https://www.caisca.org/schools/crystal-springs...
6,east bay school,berkeley,CA,https://www.caisca.org/schools/east-bay-school
8,keys school,palo alto,CA,https://www.caisca.org/schools/keys-school
9,lick wilmerding high school,san francisco,CA,https://www.caisca.org/schools/lick-wilmerding...
10,lyca e frana ais de san francisco,san francisco,CA,https://www.caisca.org/schools/lycee-francais-...
11,millennium school,san francisco,CA,https://www.caisca.org/schools/millennium-school


Count: 15


In [1203]:
targets = [
  "bentley school",
  "chinese american international school",
  "crystal springs uplands school",
  "lick wilmerding high school",
  "lycee francais de san francisco",
  "park day school",
  "brandeis marin",
  "cathedral school for boys",
  "east bay school",
  "keys school",
  "millennium school",
  "montessori de terra linda",
  "sonoma academy",
  "field middle school",
  "helios school",
]

pss_ca = pss_clean_df[pss_clean_df["state"].astype(str).str.upper().str.strip().eq("CA")].copy()
pss_ca["nm"] = pss_ca["school_name"].fillna("").astype(str).str.lower()

out = {}
for t in targets:
    out[t] = pss_ca[pss_ca["nm"].str.contains(t, na=False)][["school_id","school_name","city","state"]].head(20)

for k,v in out.items():
    print("\n===", k, "matches in PSS (contains) ===")
    display(v)



=== bentley school matches in PSS (contains) ===


,school_id,school_name,city,state



=== chinese american international school matches in PSS (contains) ===


,school_id,school_name,city,state



=== crystal springs uplands school matches in PSS (contains) ===


...

,school_id,school_name,city,state



=== field middle school matches in PSS (contains) ===


,school_id,school_name,city,state



=== helios school matches in PSS (contains) ===


,school_id,school_name,city,state


In [1205]:
import re

def tokens(s):
    return [x for x in re.sub(r"[^a-z0-9\s]"," ", s.lower()).split() if x]

for t in targets:
    toks = tokens(t)
    df = pss_ca.copy()
    for tok in toks:
        df = df[df["nm"].str.contains(rf"\b{re.escape(tok)}\b", na=False)]
    print("\n=== AND tokens:", t, "===")
    display(df[["school_id","school_name","city","state"]].head(20))



=== AND tokens: bentley school ===


,school_id,school_name,city,state



=== AND tokens: chinese american international school ===


,school_id,school_name,city,state



=== AND tokens: crystal springs uplands school ===


...

,school_id,school_name,city,state



=== AND tokens: field middle school ===


,school_id,school_name,city,state



=== AND tokens: helios school ===


,school_id,school_name,city,state


In [1207]:
print(pss_clean_df.columns.tolist())

['school_id', 'ppin', 'school_name', 'city', 'state', 'zip', 'zip4', 'address', 'join_key', 'join_key_loose', 'backbone_source', 'has_pss']


## 03.5.10 Resolve “Big-Name” Minted Duplicates (Force-match to PSS)

### Why this exists
In **03.5.9** we minted new `school_id`s for CAIS residuals when we couldn’t find a confident match in PSS.  
However, some of those residuals are “big-name” CAIS schools that **almost certainly exist in PSS**. If we keep the minted IDs, we’ll create **duplicate Golden Records** for the same real-world school (one PSS-backed, one CAIS-minted).

### Goal
For a small watchlist of high-signal schools (e.g., **Nueva**, **Bentley**, **CAIS**, **ISSF**, **Kehillah**, **Lick**, **Lycee**, **Park Day**), we:
1. Search PSS again with a **stronger matching signal**
2. Produce a **candidate table** for quick review
3. Auto-apply only **very-high-confidence** matches
4. Mark the rest for **manual selection**

### Matching approach
We use two similarity signals:
- **Token Jaccard**: good for semantic overlap (shared meaningful words)
- **Char-3gram Jaccard**: forgiving of punctuation/formatting/short variations

We then compute a **combined score** (weighted toward char-3gram), and take top candidates:
- Prefer **same-city pool** when city is usable (including substring / compound city handling)
- Fall back to statewide CA pool if city filter is too restrictive

### Safety rules (to avoid bad matches)
We only auto-fill `residual_disposition_df` when the top candidate is **extremely high confidence** (very strict thresholds).  
Everything else stays untouched and requires manual review.

### Outputs
After running the code cell, you will have:

- `big_name_candidates_df`  
  A ranked table of PSS candidates for each big-name minted CAIS school.

- `residual_disposition_df` (updated for auto-matches only)  
  Some rows will be set to:
  - `decision = "manual_match"`
  - `chosen_school_id = "<PSS PRI_...>"`

### What to do after this step
1. Scan the “Needs manual selection” rows in `big_name_candidates_df`
2. For each school, copy the correct PSS `school_id_pss` into:
   - `residual_disposition_df.decision = "manual_match"`
   - `residual_disposition_df.chosen_school_id = "<PRI_...>"`

3. Re-run **03.5.9** (Mint CAIS backbone records) so that:
   - Big-name schools are **forced to existing PSS IDs**
   - Only true “missing-from-PSS” schools are minted

### Success criteria
- Big-name CAIS schools no longer appear as minted `CAIS_CA_...` IDs
- Their CAIS membership (`has_cais=True`) is attached to the **existing PSS school_id**
- Minting remains only for genuine PSS gaps


In [1210]:
# ============================
# 03.5.10 Resolve Big-Name Minted Duplicates (PSS forced mapping)
# ============================

print("=== 03.5.10 START (RESOLVE BIG-NAME DUPLICATES) ===")

# ---------------------------------------------------------
# Preconditions
# ---------------------------------------------------------
assert "residual_disposition_df" in globals(), "Need residual_disposition_df from 03.5.8."
assert "pss_clean_df" in globals(), "Need pss_clean_df."
assert "cais_backbone_minted_df" in globals(), "Need cais_backbone_minted_df from 03.5.9."


disp = residual_disposition_df.copy()
pss = pss_clean_df.copy()
minted = cais_backbone_minted_df.copy()

# ---------------------------------------------------------
# Helpers
# ---------------------------------------------------------
def strip_accents(s: str) -> str:
    if s is None:
        return ""
    s = str(s)
    return "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))

def norm_name(s: str) -> str:
    x = strip_accents(s).lower().strip()
    x = re.sub(r"^the\s+", "", x)
    x = x.replace("&", " and ")
    x = re.sub(r"[^\w\s]", " ", x)
    x = re.sub(r"\s+", " ", x).strip()
    return x

def norm_city(s: str) -> str:
    x = strip_accents(s).lower().strip()
    x = re.sub(r"[^\w\s]", " ", x)
    x = re.sub(r"\s+", " ", x).strip()
    return x

def token_jaccard(a: str, b: str) -> float:
    ta = set(norm_name(a).split())
    tb = set(norm_name(b).split())
    if not ta or not tb:
        return 0.0
    return len(ta & tb) / len(ta | tb)

def char_ngrams(s: str, n: int = 3) -> set:
    x = norm_name(s).replace(" ", "")
    if len(x) < n:
        return set([x]) if x else set()
    return set(x[i:i+n] for i in range(len(x) - n + 1))

def char3_jaccard(a: str, b: str) -> float:
    ga = char_ngrams(a, 3)
    gb = char_ngrams(b, 3)
    if not ga or not gb:
        return 0.0
    return len(ga & gb) / len(ga | gb)

def combined_score(a: str, b: str) -> float:
    # weighted: char-3gram is more forgiving, token is more semantic
    tj = token_jaccard(a, b)
    cj = char3_jaccard(a, b)
    return 0.45 * tj + 0.55 * cj

def explode_city_candidates(city_raw: str) -> list:
    """
    Handles compound CAIS city strings like:
    - "lafayette oakland"
    - "belmont hillsborough"
    - "emeryville oakland"
    Returns a deduped list of candidate city tokens.
    """
    c = norm_city(city_raw)
    if not c:
        return [""]
    # common separators
    c2 = c.replace("/", " ").replace("&", " ").replace(" and ", " ")
    parts = [p.strip() for p in c2.split() if p.strip()]
    # also include the full string as a candidate
    out = [c] + parts
    # dedupe preserve order
    seen = set()
    final = []
    for x in out:
        if x and x not in seen:
            seen.add(x)
            final.append(x)
    return final if final else [""]

# ---------------------------------------------------------
# PSS prep (CA only)
# ---------------------------------------------------------
for col in ["school_id", "school_name", "city", "state"]:
    assert col in pss.columns, f"pss_clean_df missing required col: {col}"

pss_ca = pss[pss["state"].astype(str).str.upper().str.strip() == "CA"].copy()
pss_ca["school_name"] = pss_ca["school_name"].fillna("").astype(str)
pss_ca["city"] = pss_ca["city"].fillna("").astype(str)
pss_ca["city_norm"] = pss_ca["city"].apply(norm_city)

# ---------------------------------------------------------
# Watchlist of "big-name" minted schools
# IMPORTANT: include the *actual* minted name for Lycee in your current data
# ---------------------------------------------------------
watch_names = [
    "bentley school",
    "chinese american international school",
    "crystal springs uplands school",
    "kehillah school",
    "lick wilmerding high school",
    "lyca e frana ais de san francisco",  # current CAIS-encoded name in your residuals
    "nueva school",
    "park day school",
]

minted["name_norm"] = minted["name"].fillna("").astype(str).apply(norm_name)
watch_norm = set(norm_name(w) for w in watch_names)

minted_big = minted[minted["name_norm"].isin(watch_norm)].copy()

print("Minted big-name rows:", minted_big.shape[0])
display(minted_big[["school_id","name","city","detail_url"]])

# ---------------------------------------------------------
# Candidate search
# Strategy:
#   - Try matching within any of the exploded city tokens (city pool)
#   - Fall back to all CA (state pool)
# ---------------------------------------------------------
def top_pss_candidates(cais_name: str, cais_city: str, topn: int = 8) -> pd.DataFrame:
    city_tokens = explode_city_candidates(cais_city)

    # build city pool: any PSS city that equals one of the tokens
    pool_city = pss_ca[pss_ca["city_norm"].isin(city_tokens)].copy()
    if len(pool_city) > 0:
        pool = pool_city
        scope = "city"
    else:
        pool = pss_ca
        scope = "state"

    rows = []
    for _, r in pool.iterrows():
        s = combined_score(cais_name, r["school_name"])
        if s <= 0:
            continue
        rows.append({
            "candidate_scope": scope,
            "school_id_pss": r["school_id"],
            "pss_school_name": r["school_name"],
            "pss_city": r["city"],
            "score_combined": float(s),
            "score_token_jaccard": float(token_jaccard(cais_name, r["school_name"])),
            "score_char3_jaccard": float(char3_jaccard(cais_name, r["school_name"])),
        })

    out = pd.DataFrame(rows)
    if out.empty:
        return out
    return out.sort_values("score_combined", ascending=False).head(topn).reset_index(drop=True)

cand_rows = []
for _, r in minted_big.iterrows():
    cais_name = r["name"]
    cais_city = r["city"]
    cands = top_pss_candidates(cais_name, cais_city, topn=8)

    if cands.empty:
        cand_rows.append({
            "cais_name": cais_name,
            "cais_city": cais_city,
            "minted_school_id": r["school_id"],
            "rank": None,
            "candidate_scope": "none",
            "school_id_pss": None,
            "pss_school_name": None,
            "pss_city": None,
            "score_combined": None,
            "score_token_jaccard": None,
            "score_char3_jaccard": None,
        })
        continue

    for i, row in cands.iterrows():
        rowd = row.to_dict()
        cand_rows.append({
            "cais_name": cais_name,
            "cais_city": cais_city,
            "minted_school_id": r["school_id"],
            "rank": int(i + 1),
            **rowd,
        })

big_name_candidates_df = pd.DataFrame(cand_rows)
print("\nTop candidates for big-name schools (review):")
display(big_name_candidates_df.sort_values(["cais_name","rank"]).head(200))

# ---------------------------------------------------------
# Auto-apply ONLY very-high-confidence matches
# Policy (super conservative):
#   - rank=1
#   - combined >= 0.90 AND char3 >= 0.92 AND token >= 0.60
# Otherwise leave for manual selection.
# ---------------------------------------------------------
auto = big_name_candidates_df[big_name_candidates_df["rank"] == 1].copy()

auto["auto_ok"] = (
    (auto["score_combined"].fillna(0) >= 0.90) &
    (auto["score_char3_jaccard"].fillna(0) >= 0.92) &
    (auto["score_token_jaccard"].fillna(0) >= 0.60)
)

auto_ok = auto[auto["auto_ok"]].copy()
auto_no = auto[~auto["auto_ok"]].copy()

print("\nAuto-eligible (very high confidence):", auto_ok.shape[0])
display(auto_ok[["cais_name","cais_city","school_id_pss","pss_school_name","pss_city","candidate_scope","score_combined","score_token_jaccard","score_char3_jaccard"]])

print("\nNeeds manual selection (most likely):", auto_no.shape[0])
display(auto_no[["cais_name","cais_city","school_id_pss","pss_school_name","pss_city","candidate_scope","score_combined","score_token_jaccard","score_char3_jaccard"]])

# ---------------------------------------------------------
# Apply auto matches into residual_disposition_df
# We match by normalized school_name so accents/punctuation don't break it.
# ---------------------------------------------------------
disp["_name_norm"] = disp["school_name"].fillna("").astype(str).apply(norm_name)

for _, r in auto_ok.iterrows():
    name_norm = norm_name(r["cais_name"])
    sid = r["school_id_pss"]
    if pd.isna(sid) or str(sid).strip() == "":
        continue

    mask = disp["_name_norm"] == name_norm
    if mask.any():
        disp.loc[mask, "decision"] = "manual_match"
        disp.loc[mask, "chosen_school_id"] = str(sid).strip()
        disp.loc[mask, "alias_name"] = ""
        note = f"auto_manual_match_to_{str(sid).strip()}"
        disp.loc[mask, "notes"] = (disp.loc[mask, "notes"].fillna("") + " | " + note).str.strip(" |")

# Write back
residual_disposition_df = disp.drop(columns=["_name_norm"], errors="ignore")

print("\nUpdated residual_disposition_df rows (big-name only):")
tmp = residual_disposition_df.copy()
tmp["_name_norm"] = tmp["school_name"].fillna("").astype(str).apply(norm_name)
display(
    tmp[tmp["_name_norm"].isin(watch_norm)][["school_name","city","decision","chosen_school_id","notes"]]
    .sort_values("school_name")
)

print("=== 03.5.10 END ===")


=== 03.5.10 START (RESOLVE BIG-NAME DUPLICATES) ===
Minted big-name rows: 6


,school_id,name,city,detail_url
1,CAIS_CA_bentley-school-lafayette-oakland-ca_d3...,bentley school,lafayette oakland,https://www.caisca.org/schools/bentley-school
4,CAIS_CA_chinese-american-international-school-...,chinese american international school,san francisco,https://www.caisca.org/schools/chinese-america...
5,CAIS_CA_crystal-springs-uplands-school-belmont...,crystal springs uplands school,belmont hillsborough,https://www.caisca.org/schools/crystal-springs...
9,CAIS_CA_lick-wilmerding-high-school-san-franci...,lick wilmerding high school,san francisco,https://www.caisca.org/schools/lick-wilmerding...
10,CAIS_CA_lyca-e-frana-ais-de-san-francisco-san-...,lyca e frana ais de san francisco,san francisco,https://www.caisca.org/schools/lycee-francais-...
14,CAIS_CA_park-day-school-oakland-ca_5c5d05c8,park day school,oakland,https://www.caisca.org/schools/park-day-school



Top candidates for big-name schools (review):


,cais_name,cais_city,minted_school_id,rank,candidate_scope,school_id_pss,pss_school_name,pss_city,score_combined,score_token_jaccard,score_char3_jaccard
0,bentley school,lafayette oakland,CAIS_CA_bentley-school-lafayette-oakland-ca_d3...,1,city,PRI_A9100558,aurora school,oakland,0.279412,0.333333,0.235294
1,bentley school,lafayette oakland,CAIS_CA_bentley-school-lafayette-oakland-ca_d3...,2,city,PRI_00092014,raskob day school,oakland,0.257237,0.250000,0.263158
2,bentley school,lafayette oakland,CAIS_CA_bentley-school-lafayette-oakland-ca_d3...,3,city,PRI_A9500742,redwood day school,oakland,0.250000,0.250000,0.250000
3,bentley school,lafayette oakland,CAIS_CA_bentley-school-lafayette-oakland-ca_d3...,4,city,PRI_A0500686,springstone school,lafayette,0.250000,0.333333,0.181818
4,bentley school,lafayette oakland,CAIS_CA_bentley-school-lafayette-oakland-ca_d3...,5,city,PRI_00074673,st theresa school,oakland,0.222500,0.250000,0.200000
5,bentley school,lafayette oakland,CAIS_CA_bentley-school-lafayette-oakland-ca_d3...,6,city,PRI_00083429,head royce school,oakland,0.222500,0.250000,0.200000
6,bentley school,lafayette oakland,CAIS_CA_bentley-school-lafayette-oakland-ca_d3...,7,city,PRI_00075032,st perpetua school,lafayette,0.217262,0.250000,0.190476
7,bentley school,lafayette oakland,CAIS_CA_bentley-school-lafayette-oakland-ca_d3...,8,city,PRI_00083597,college preparatory school,oakland,0.210714,0.250000,0.178571
8,chinese american international school,san francisco,CAIS_CA_chinese-american-international-school-...,1,city,PRI_A9302442,woodside international school,san francisco,0.413750,0.400000,0.425000
9,chinese american international school,san francisco,CAIS_CA_chinese-american-international-school-...,2,city,PRI_A1900492,la scuola international school,san francisco,0.383750,0.333333,0.425000



Auto-eligible (very high confidence): 0


...


Needs manual selection (most likely): 6


,cais_name,cais_city,school_id_pss,pss_school_name,pss_city,candidate_scope,score_combined,score_token_jaccard,score_char3_jaccard
0,bentley school,lafayette oakland,PRI_A9100558,aurora school,oakland,city,0.279412,0.333333,0.235294
8,chinese american international school,san francisco,PRI_A9302442,woodside international school,san francisco,city,0.413750,0.400000,0.425000
16,crystal springs uplands school,belmont hillsborough,PRI_A9100595,bridge school,hillsborough,city,0.160968,0.200000,0.129032
24,lick wilmerding high school,san francisco,PRI_00072596,archbishop riordan high school,san francisco,city,0.262821,0.333333,0.205128
32,lyca e frana ais de san francisco,san francisco,PRI_A1900692,san francisco schoolhouse,san francisco,city,0.274265,0.250000,0.294118
40,park day school,oakland,PRI_00092014,raskob day school,oakland,city,0.451471,0.500000,0.411765



Updated residual_disposition_df rows (big-name only):


,school_name,city,decision,chosen_school_id,notes
1,bentley school,lafayette oakland,mint_new_record,,
4,chinese american international school,san francisco,mint_new_record,,
5,crystal springs uplands school,belmont hillsborough,mint_new_record,,
7,kehillah school,palo alto,manual_match,PRI_A0500422,forced to PSS PRI_A0500422
9,lick wilmerding high school,san francisco,mint_new_record,,
10,lyca e frana ais de san francisco,san francisco,mint_new_record,,
13,nueva school,hillsborough,manual_match,PRI_A1790096,forced to PSS PRI_A1790096
14,park day school,oakland,mint_new_record,,


=== 03.5.10 END ===


## 03.5.10b Big-Name Resolver (Curated Alias Probes + City Cleanup)

### Why we need this
Our generic similarity search (token/char-3gram) is intentionally conservative and does not auto-force matches unless confidence is extremely high.

For several “big-name” CAIS schools, the **PSS name often differs** from the CAIS label (campus names, grade divisions, “International High School…”, “Nueva Middle School…”, etc.), which makes similarity scores look mediocre and yields incorrect top candidates.

### Goal
For the small set of “big-name” minted CAIS schools, we will:
1. Normalize / split compound cities (e.g. `lafayette oakland` → candidates: `lafayette`, `oakland`)
2. Probe PSS using a **curated alias list** per school name (high precision)
3. Produce a **review table** showing the best matching PSS `school_id` per alias probe
4. Auto-apply only **very high confidence** (exact/near-exact matches), otherwise leave for manual choice

### Outputs
- `big_name_probe_results_df`: alias-probe candidates ranked by score
- `big_name_best_probe_df`: best candidate per CAIS school
- (optional) updates to `residual_disposition_df` for auto-confirmed rows only

### What you do next
- For each big-name school, pick the correct `school_id_pss` from `big_name_best_probe_df`
- Set:
  - `residual_disposition_df.decision = "manual_match"`
  - `residual_disposition_df.chosen_school_id = "<PRI_...>"`
- Re-run **03.5.9** to remove minted duplicates and attach CAIS membership to the real PSS record.


In [1213]:

print("=== 03.5.10b v6 START (CANDIDATES: city gate + distinctive token overlap gate) ===")

assert "pss_clean_df" in globals(), "Need pss_clean_df."
assert "residual_disposition_df" in globals(), "Need residual_disposition_df."

pss = pss_clean_df.copy()
disp = residual_disposition_df.copy()

# -------------------------
# Normalization helpers
# -------------------------
def strip_accents(s: str) -> str:
    if s is None:
        return ""
    s = str(s)
    return "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))

def norm(s: str) -> str:
    x = strip_accents(s).lower()
    x = re.sub(r"[^\w\s]", " ", x)
    x = re.sub(r"\s+", " ", x).strip()
    return x

GENERIC_WORDS = {
    "the","and","of","for","at","in","to","a","an",
    "school","academy","schools",
    "elementary","middle","high",
    "international","day",
    "boys","girls",
    "de","la","le","du","des",
}

def tokens_distinctive(name: str, city: str | None = None) -> list[str]:
    name_toks = [t for t in norm(name).split() if t and t not in GENERIC_WORDS and len(t) >= 3]
    if city is None:
        return name_toks
    city_toks = set([t for t in norm(city).split() if t and len(t) >= 3])
    # remove city tokens from the "distinctive" set so we don't match on "san/francisco"
    out = [t for t in name_toks if t not in city_toks]
    return out

def jaccard(a: set[str], b: set[str]) -> float:
    if not a and not b:
        return 0.0
    return len(a & b) / max(1, len(a | b))

# -------------------------
# City gating (Bay Area clusters)
# - keep it simple and conservative
# -------------------------
BAYAREA_CLUSTER = {
    # SF Peninsula
    "san francisco", "daly city", "south san francisco", "brisbane", "san bruno", "millbrae",
    "burlingame", "san mateo", "belmont", "san carlos", "redwood city", "atherton", "menlo park",
    "palo alto", "mountain view", "sunnyvale", "cupertino", "los altos", "los altos hills",
    "woodside", "portola valley",
    # East Bay
    "oakland", "berkeley", "emeryville", "alameda", "piedmont", "el cerrito", "richmond",
    "san leandro", "hayward", "fremont", "union city", "newark", "walnut creek", "lafayette",
    "moraga", "orinda", "pleasant hill", "concord",
    # North Bay
    "san rafael", "larkspur", "corte madera", "mill valley", "sausalito", "tiburon",
    "novato", "petaluma", "rohnert park", "santa rosa", "sebastopol", "sonoma",
}

def city_bucket(city: str) -> str:
    c = norm(city)
    if c in BAYAREA_CLUSTER:
        return "bayarea"
    return "other"

# -------------------------
# Build CA-only PSS pool with name and join_key_loose
# -------------------------
pss_ca = pss[pss["state"].astype(str).str.upper().str.strip() == "CA"].copy()
pss_ca["name_norm"] = pss_ca["school_name"].fillna("").astype(str).apply(norm)
pss_ca["jk_norm"] = pss_ca["join_key_loose"].fillna("").astype(str).apply(norm) if "join_key_loose" in pss_ca.columns else ""
pss_ca["city_norm"] = pss_ca["city"].fillna("").astype(str).apply(norm)

# token sets for faster scoring
pss_ca["tok_set"] = pss_ca.apply(lambda r: set(tokens_distinctive(r["school_name"], r["city"])), axis=1)

# -------------------------
# Targets = CA, mint_new_record, needs deeper match
# -------------------------
targets_df = disp.query(
    "state == 'CA' and decision == 'mint_new_record'"
).copy()

targets_df["notes_norm"] = targets_df["notes"].fillna("").astype(str).str.lower()
targets_df = targets_df[targets_df["notes_norm"].str.contains("needs deeper match", na=False)].copy()

print(f"Targets needing deeper match: {len(targets_df)}")
display(targets_df[["school_name","city","detail_url","notes"]])

# -------------------------
# Candidate search per target
# -------------------------
def candidate_pool_for_target(t_name: str, t_city: str) -> pd.DataFrame:
    t_city_n = norm(t_city)
    bucket = city_bucket(t_city)

    # Conservative city gate:
    # - same city always allowed
    # - if target city is bayarea cluster, allow bayarea cluster
    if bucket == "bayarea":
        city_mask = pss_ca["city_norm"].isin([norm(x) for x in BAYAREA_CLUSTER])
    else:
        city_mask = (pss_ca["city_norm"] == t_city_n)

    return pss_ca.loc[city_mask].copy()

def score_candidates(t_name: str, t_city: str, pool: pd.DataFrame) -> pd.DataFrame:
    t_tok = set(tokens_distinctive(t_name, t_city))
    t_city_n = norm(t_city)

    if len(t_tok) == 0:
        # if we have no distinctive tokens, it's too dangerous to match
        return pd.DataFrame()

    out = pool.copy()
    out["overlap"] = out["tok_set"].apply(lambda s: sorted(list(s & t_tok)))
    out["overlap_n"] = out["overlap"].apply(len)
    out["token_jaccard"] = out["tok_set"].apply(lambda s: jaccard(s, t_tok))

    # Strong city bonus only if exact city matches
    out["city_bonus"] = (out["city_norm"] == t_city_n).astype(int) * 10

    # Raw score: overlap is king; jaccard helps break ties; city bonus nudges
    out["score"] = out["overlap_n"] * 50 + (out["token_jaccard"] * 30) + out["city_bonus"]

    # Hard gate: require at least 2 distinctive-token overlap OR very high jaccard with 1 overlap
    out = out[
        (out["overlap_n"] >= 2)
        | ((out["overlap_n"] >= 1) & (out["token_jaccard"] >= 0.60))
    ].copy()

    return out.sort_values(["score","overlap_n","token_jaccard"], ascending=False)

# -------------------------
# Run
# -------------------------
results = {}

for _, r in targets_df.iterrows():
    t_name = str(r["school_name"])
    t_city = str(r["city"])

    print("\n==============================")
    print(f"TARGET: {t_name} | city: {t_city}")

    pool = candidate_pool_for_target(t_name, t_city)
    scored = score_candidates(t_name, t_city, pool)

    cols = ["school_id","school_name","city","zip","address","score","city_bonus","overlap_n","token_jaccard","overlap"]
    if scored is None or len(scored) == 0:
        print("No credible candidates under current gates.")
        results[t_name] = pd.DataFrame(columns=cols)
        continue

    show = scored[cols].head(10)
    display(show)
    results[t_name] = show

print("=== 03.5.10b v6 END ===")


=== 03.5.10b v6 START (CANDIDATES: city gate + distinctive token overlap gate) ===
Targets needing deeper match: 0


,school_name,city,detail_url,notes


=== 03.5.10b v6 END ===


In [1215]:
pss_ca_dbg = pss_clean_df[pss_clean_df["state"].astype(str).str.upper().str.strip().eq("CA")].copy()
pss_ca_dbg["nm"] = pss_ca_dbg["school_name"].fillna("").astype(str).str.lower()

for q in ["bentley", "millennium", "lick", "wilmerding", "park day", "lycee", "francais", "cathedral"]:
    hit = pss_ca_dbg[pss_ca_dbg["nm"].str.contains(q, na=False)][["school_id","school_name","city","state"]].head(20)
    print("\n=== direct contains:", q, "===")
    display(hit)



=== direct contains: bentley ===


,school_id,school_name,city,state



=== direct contains: millennium ===


,school_id,school_name,city,state
13003,PRI_A1700371,millennium school of san francisco,san francisco,CA



=== direct contains: lick ===


...

,school_id,school_name,city,state
613,PRI_00081341,le lycee francais de los angeles,los angeles,CA
15818,PRI_A2100439,le lycee francais de san diego,san diego,CA



=== direct contains: francais ===


,school_id,school_name,city,state
613,PRI_00081341,le lycee francais de los angeles,los angeles,CA
15818,PRI_A2100439,le lycee francais de san diego,san diego,CA
20626,PRI_BB000062,l heritage francais,la habra,CA



=== direct contains: cathedral ===


,school_id,school_name,city,state
182,PRI_00068987,cathedral chapel school,los angeles,CA
542,PRI_00077696,st eugene s cathedral school,santa rosa,CA
9958,PRI_A0900259,cathedral catholic high school,san diego,CA
12119,PRI_A1500216,christ cathedral academy,garden grove,CA
13898,PRI_A1790041,cathedral of annunciation school,stockton,CA


In [1217]:
print("=== 03.5.10bb START (APPLY MANUAL_MATCH OVERRIDES) ===")

manual_match = {
  "east bay school|berkeley|CA": "PRI_A1300209",
  "millennium school|san francisco|CA": "PRI_A1700371",
}

def canon_key(name, city, state):
    return f"{str(name).strip().lower()}|{str(city).strip().lower()}|{str(state).strip().upper()}"

def add_note_once(existing, note):
    existing = ("" if existing is None else str(existing)).strip()
    if note in existing:
        return existing
    return (existing + " | " + note).strip(" |")


df = residual_disposition_df.copy()
df["_canon_key"] = df.apply(lambda r: canon_key(r["school_name"], r["city"], r["state"]), axis=1)

df["chosen_school_id"] = df.apply(
    lambda r: manual_match.get(r["_canon_key"], r.get("chosen_school_id", "")),
    axis=1
)

df["decision"] = df.apply(
    lambda r: "manual_match" if r["_canon_key"] in manual_match else r["decision"],
    axis=1
)

df["notes"] = df.apply(
    lambda r: add_note_once(r.get("notes",""), "manual_match override")
    if r["_canon_key"] in manual_match else r.get("notes",""),
    axis=1
)

residual_disposition_df = df.drop(columns=["_canon_key"])

display(
  residual_disposition_df[
    residual_disposition_df["school_name"].str.lower().isin(["east bay school", "millennium school"])
  ][["school_name","city","state","decision","chosen_school_id","detail_url","notes"]]
)

print("=== 03.5.10bb END ===")


=== 03.5.10bb START (APPLY MANUAL_MATCH OVERRIDES) ===


,school_name,city,state,decision,chosen_school_id,detail_url,notes
6,east bay school,berkeley,CA,manual_match,PRI_A1300209,https://www.caisca.org/schools/east-bay-school,manual_match override
11,millennium school,san francisco,CA,manual_match,PRI_A1700371,https://www.caisca.org/schools/millennium-school,manual_match override


=== 03.5.10bb END ===


## 03.5.10c — PSS Existence Check for Big-Name CAIS Mints

### Goal
Before we try to *merge* (“dedupe”) any **minted CAIS big-name schools** back into the PSS backbone, we first verify a basic requirement:

> **Does this school exist anywhere in PSS (California records)?**

If **PSS has zero hits**, then any fuzzy “top candidate” we saw earlier is almost certainly a **false positive** (shared generic words like *international*, *school*, *san francisco*), and the correct action is:

- **Keep the CAIS-minted record** (`decision = mint_new_record`)
- Add a note: **“PSS missing; keep CAIS minted”**

---

### What we did
For each target school name, we ran two scans against **PSS CA only**:

1. **Contains scan**: a simple substring match (case-insensitive).
2. **AND scan**: requires that multiple key tokens appear in the PSS `school_name`.

Both scans are intentionally “wide net” checks. If both return **0**, PSS does not contain the school under a recognizable name.

---

### Result Summary

**Target: international school of san francisco**
- contains-scan hits: **0**
- AND-scan hits: **0**

**Target: lycee francais de san francisco**
- contains-scan hits: **0**
- AND-scan hits: **0**

✅ **Targets missing from PSS (CA):**
- `international school of san francisco`
- `lycee francais de san francisco`

---

### Disposition Decision
Because both schools are **missing from PSS**, we keep them as CAIS authority backbone records:

- `decision = mint_new_record`
- `notes = "PSS missing; keep CAIS minted"`

This avoids incorrect forced matches to unrelated PSS schools.

---

### Output
`residual_disposition_df` is updated (targets only):

- **international school of san francisco** → `mint_new_record` (PSS missing)
- **lycee francais de san francisco** → `mint_new_record` (PSS missing)

---


In [1220]:

print("=== 03.5.10c v2 START (PSS EXISTENCE CHECK: mint-driven + jk_loose + anchor overrides + token filter) ===")

assert "pss_clean_df" in globals(), "Need pss_clean_df."
assert "residual_disposition_df" in globals(), "Need residual_disposition_df."

pss = pss_clean_df.copy()
disp = residual_disposition_df.copy()

# -------------------------
# Normalization helpers
# -------------------------
def strip_accents(s: str) -> str:
    if s is None:
        return ""
    s = str(s)
    return "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))

def norm(s: str) -> str:
    x = strip_accents(s).lower()
    x = re.sub(r"[^\w\s]", " ", x)
    x = re.sub(r"\s+", " ", x).strip()
    return x

GENERIC_WORDS = {
    "the","and","of","for","at","in","to","a","an",
    "school","academy","schools",
    "elementary","middle","high",
    "international","day",
    "boys","girls",
    "de","la","le","du","des",  # french-ish glue words
}

def tokens_non_generic(name: str) -> list[str]:
    toks = [t for t in norm(name).split() if t and t not in GENERIC_WORDS]
    toks = [t for t in toks if len(t) >= 3]  # IMPORTANT: drop tiny garbage tokens like "e"
    return toks

def pick_anchor(name: str) -> str:
    toks = tokens_non_generic(name)
    return max(toks, key=len) if toks else ""

# Anchor overrides: phrase anchors for tricky names (existence probing)
ANCHOR_OVERRIDES = {
    "park day school": ["park day", "parkday"],
    "lick wilmerding high school": ["wilmerding", "lick", "lick wilmerding"],
    "crystal springs uplands school": ["crystal springs", "uplands", "csus"],
    "bentley school": ["bentley"],
    "cathedral school for boys": ["cathedral"],  # still generic, but better than nothing
    "chinese american international school": ["chinese", "chinese american"],
    # If this CAIS name is corrupted, the URL slug can still help elsewhere; keep anchors broad here.
}

def anchors_for(name: str) -> list[str]:
    n = norm(name)
    for k, v in ANCHOR_OVERRIDES.items():
        if norm(k) == n:
            return v
    a = pick_anchor(name)
    return [a] if a else []

# -------------------------
# CA-only pool + normalized fields
# -------------------------
pss_ca = pss[pss["state"].astype(str).str.upper().str.strip() == "CA"].copy()

pss_ca["name_norm"] = pss_ca["school_name"].fillna("").astype(str).apply(norm)

# join_key_loose exists in your schema; if missing, create empty string
if "join_key_loose" in pss_ca.columns:
    pss_ca["jk_norm"] = pss_ca["join_key_loose"].fillna("").astype(str).apply(norm)
else:
    pss_ca["jk_norm"] = ""

pss_ca["city_norm"] = pss_ca["city"].fillna("").astype(str).apply(norm)

# -------------------------
# Targets = CURRENT CAIS mint_new_record rows (CA)
# -------------------------
mint_rows = disp.query("decision == 'mint_new_record' and state == 'CA'")[["school_name","city","state","detail_url","notes"]].copy()
mint_rows["school_name_norm"] = mint_rows["school_name"].fillna("").astype(str).apply(norm)

TARGETS = mint_rows["school_name"].tolist()

print(f"Mint targets (CA) count: {len(TARGETS)}")
display(mint_rows[["school_name","city","detail_url"]])

# -------------------------
# Scans (search name OR join_key_loose)
# -------------------------
def _empty_hits():
    return pd.DataFrame(columns=["school_id","school_name","city","state","matched_on"])

def contains_scan(full_phrase: str) -> pd.DataFrame:
    k = norm(full_phrase)
    if not k:
        return _empty_hits()
    pat = re.escape(k)
    m = pss_ca["name_norm"].str.contains(pat, na=False) | pss_ca["jk_norm"].str.contains(pat, na=False)
    out = pss_ca.loc[m, ["school_id", "school_name", "city", "state"]].copy()
    out["matched_on"] = f"contains(name|jk):{k}"
    return out.sort_values(["city", "school_name"]).head(120)

def and_scan(keys: list[str]) -> pd.DataFrame:
    if not keys:
        return _empty_hits()
    m = pd.Series(True, index=pss_ca.index)
    for k in keys:
        kk = norm(k)
        if not kk:
            continue
        pat = rf"\b{re.escape(kk)}\b"
        m = m & (pss_ca["name_norm"].str.contains(pat, na=False) | pss_ca["jk_norm"].str.contains(pat, na=False))
    out = pss_ca.loc[m, ["school_id", "school_name", "city", "state"]].copy()
    out["matched_on"] = f"AND(name|jk):{'|'.join(keys)}"
    return out.sort_values(["city", "school_name"]).head(200)

def anchor_scan(anchor: str) -> pd.DataFrame:
    kk = norm(anchor)
    if not kk:
        return _empty_hits()
    # If it's a phrase, use substring; if single token, use word boundary
    if " " in kk:
        pat = re.escape(kk)
        m = pss_ca["name_norm"].str.contains(pat, na=False) | pss_ca["jk_norm"].str.contains(pat, na=False)
    else:
        pat = rf"\b{re.escape(kk)}\b"
        m = pss_ca["name_norm"].str.contains(pat, na=False) | pss_ca["jk_norm"].str.contains(pat, na=False)

    out = pss_ca.loc[m, ["school_id","school_name","city","state"]].copy()
    out["matched_on"] = f"ANCHOR(name|jk):{anchor}"
    return out.sort_values(["city","school_name"]).head(200)

missing = []

print("\n--- PSS existence scans (CA) ---")
for t in TARGETS:
    print(f"\nTarget: {t}")

    # 1) contains full phrase
    hits1 = contains_scan(t)

    # 2) AND scan (non-generic tokens; >=3 chars)
    toks = tokens_non_generic(t)
    hits2 = and_scan(toks)

    # 3) anchor probes (override phrases if configured)
    anchors = anchors_for(t)
    hits3_list = [anchor_scan(a) for a in anchors if a]
    hits3 = pd.concat(hits3_list, ignore_index=True).drop_duplicates() if hits3_list else _empty_hits()

    print(f"contains-scan hits: {len(hits1)}")
    display(hits1)

    print(f"AND-scan hits (tokens={toks}): {len(hits2)}")
    display(hits2)

    print(f"ANCHOR-scan hits (anchors={anchors}): {len(hits3)}")
    display(hits3)

    if len(hits1) == 0 and len(hits2) == 0 and len(hits3) == 0:
        missing.append(t)

print("\nTargets missing from PSS (CA):", missing)

# -------------------------
# Update disposition for missing-in-PSS targets:
# - keep mint_new_record
# - add note
# - never overwrite manual_match
# -------------------------
def add_note_once(existing, note):
    existing = ("" if existing is None else str(existing)).strip()
    if note in existing:
        return existing
    return (existing + " | " + note).strip(" |")

if missing:
    for t in missing:
        m = disp["school_name"].fillna("").astype(str).apply(norm).eq(norm(t))
        m = m & (disp["decision"] != "manual_match")  # guard

        disp.loc[m, "decision"] = "mint_new_record"
        disp.loc[m, "chosen_school_id"] = disp.loc[m, "chosen_school_id"].fillna("")

        disp.loc[m, "notes"] = disp.loc[m, "notes"].apply(lambda x: add_note_once(x, "PSS missing; keep CAIS minted"))

    residual_disposition_df = disp

print("\nUpdated residual_disposition_df (targets only):")
display(
    disp[disp["school_name"].fillna("").astype(str).apply(norm).isin([norm(x) for x in TARGETS])][
        ["school_name", "city", "decision", "chosen_school_id", "notes"]
    ].sort_values(["decision","school_name"])
)

print("=== 03.5.10c v2 END ===")


=== 03.5.10c v2 START (PSS EXISTENCE CHECK: mint-driven + jk_loose + anchor overrides + token filter) ===
Mint targets (CA) count: 13


,school_name,city,detail_url
1,bentley school,lafayette oakland,https://www.caisca.org/schools/bentley-school
2,brandeis marin,san rafael,https://www.caisca.org/schools/brandeis-marin
3,cathedral school for boys,san francisco,https://www.caisca.org/schools/cathedral-schoo...
4,chinese american international school,san francisco,https://www.caisca.org/schools/chinese-america...
5,crystal springs uplands school,belmont hillsborough,https://www.caisca.org/schools/crystal-springs...
8,keys school,palo alto,https://www.caisca.org/schools/keys-school
9,lick wilmerding high school,san francisco,https://www.caisca.org/schools/lick-wilmerding...
10,lyca e frana ais de san francisco,san francisco,https://www.caisca.org/schools/lycee-francais-...
12,montessori de terra linda,san rafael,https://www.caisca.org/schools/montessori-de-t...
14,park day school,oakland,https://www.caisca.org/schools/park-day-school



--- PSS existence scans (CA) ---

Target: bentley school
contains-scan hits: 0


,school_id,school_name,city,state,matched_on


AND-scan hits (tokens=['bentley']): 0


...

ANCHOR-scan hits (anchors=['helios']): 0


,school_id,school_name,city,state,matched_on



Targets missing from PSS (CA): ['bentley school', 'crystal springs uplands school', 'lick wilmerding high school', 'field middle school', 'helios school']

Updated residual_disposition_df (targets only):


,school_name,city,decision,chosen_school_id,notes
1,bentley school,lafayette oakland,mint_new_record,,PSS missing; keep CAIS minted
2,brandeis marin,san rafael,mint_new_record,,
3,cathedral school for boys,san francisco,mint_new_record,,
4,chinese american international school,san francisco,mint_new_record,,
5,crystal springs uplands school,belmont hillsborough,mint_new_record,,PSS missing; keep CAIS minted
16,field middle school,san mateo,mint_new_record,,PSS missing; keep CAIS minted
17,helios school,sunnyvale,mint_new_record,,PSS missing; keep CAIS minted
8,keys school,palo alto,mint_new_record,,
9,lick wilmerding high school,san francisco,mint_new_record,,PSS missing; keep CAIS minted
10,lyca e frana ais de san francisco,san francisco,mint_new_record,,


=== 03.5.10c v2 END ===


In [1222]:
print("=== 03.5.10e START (FINALIZE: NO-CREDIBLE-PSS => KEEP CAIS MINTED) ===")

df = residual_disposition_df.copy()

def add_note_once(existing, note):
    existing = ("" if existing is None else str(existing)).strip()
    parts = [p.strip() for p in existing.split("|") if p.strip()]
    if note in parts:
        return " | ".join(parts)
    parts.append(note)
    return " | ".join(parts)

# rows that were flagged "needs deeper match" but 03.5.10b v6 found no credible candidates
need_keep = (
    (df["state"].astype(str).str.upper().str.strip() == "CA")
    & (df["decision"] == "mint_new_record")
    & (df["notes"].fillna("").astype(str).str.contains("needs deeper match", case=False, na=False))
)

# Replace the "needs deeper match" phrase with final disposition, and make it idempotent
df.loc[need_keep, "notes"] = (
    df.loc[need_keep, "notes"]
      .fillna("")
      .astype(str)
      .str.replace("PSS not proven missing; needs deeper match", "No credible PSS candidate; keep CAIS minted", case=False, regex=False)
      .apply(lambda s: add_note_once(s, "No credible PSS candidate; keep CAIS minted"))
)

# If the old phrase was truncated (your '...'), still ensure we add the final note once
df.loc[need_keep, "notes"] = df.loc[need_keep, "notes"].apply(
    lambda s: add_note_once(s, "No credible PSS candidate; keep CAIS minted")
)

residual_disposition_df = df

# show only CAIS mint targets (the 13) after finalization
mint_targets_ca = (
    (df["state"].astype(str).str.upper().str.strip() == "CA")
    & (df["decision"].isin(["mint_new_record", "manual_match"]))
    & (df["detail_url"].fillna("").astype(str).str.contains("caisca.org/schools/", na=False))
)

display(
    df.loc[mint_targets_ca, ["school_name","city","state","decision","chosen_school_id","notes","detail_url"]]
      .sort_values(["decision","school_name"])
)

print("=== 03.5.10e END ===")


=== 03.5.10e START (FINALIZE: NO-CREDIBLE-PSS => KEEP CAIS MINTED) ===


,school_name,city,state,decision,chosen_school_id,notes,detail_url
6,east bay school,berkeley,CA,manual_match,PRI_A1300209,manual_match override,https://www.caisca.org/schools/east-bay-school
0,escuela bilinga1 4e internacional,emeryville oakland,CA,manual_match,PRI_A0770343,forced match: CAIS encoding/city multi-token |...,https://www.caisca.org/schools/escuela-bilingu...
7,kehillah school,palo alto,CA,manual_match,PRI_A0500422,forced to PSS PRI_A0500422,https://www.caisca.org/schools/kehillah--school
11,millennium school,san francisco,CA,manual_match,PRI_A1700371,manual_match override,https://www.caisca.org/schools/millennium-school
13,nueva school,hillsborough,CA,manual_match,PRI_A1790096,forced to PSS PRI_A1790096,https://www.caisca.org/schools/the-nueva-school
1,bentley school,lafayette oakland,CA,mint_new_record,,PSS missing; keep CAIS minted,https://www.caisca.org/schools/bentley-school
2,brandeis marin,san rafael,CA,mint_new_record,,,https://www.caisca.org/schools/brandeis-marin
3,cathedral school for boys,san francisco,CA,mint_new_record,,,https://www.caisca.org/schools/cathedral-schoo...
4,chinese american international school,san francisco,CA,mint_new_record,,,https://www.caisca.org/schools/chinese-america...
5,crystal springs uplands school,belmont hillsborough,CA,mint_new_record,,PSS missing; keep CAIS minted,https://www.caisca.org/schools/crystal-springs...


=== 03.5.10e END ===


In [1224]:
print("=== 03.5.10f v2 START (DEDUP NOTES: stable + safe display) ===")

df = residual_disposition_df.copy()

def dedupe_pipe_notes(val):
    s = ("" if val is None else str(val)).strip()
    if not s:
        return s
    parts = [p.strip() for p in s.split("|") if p.strip()]
    out = []
    seen = set()
    for p in parts:
        key = " ".join(p.lower().split())  # normalize whitespace + case
        if key in seen:
            continue
        seen.add(key)
        out.append(p)
    return " | ".join(out)

# track before/after for display
before_notes = df["notes"].copy()
df["notes"] = df["notes"].apply(dedupe_pipe_notes)

residual_disposition_df = df

# show rows where notes actually changed
changed = (before_notes.fillna("").astype(str) != df["notes"].fillna("").astype(str))

# focus rows: manual_match and/or forced notes, plus any note-changed rows
focus = (
    df["decision"].fillna("").astype(str).eq("manual_match")
    | df["notes"].fillna("").astype(str).str.contains("forced", case=False, na=False)
    | changed
)

display(
    df.loc[focus, ["school_name","city","state","decision","chosen_school_id","notes","detail_url"]]
      .sort_values(["decision","school_name"])
)

print("Rows with notes changed:", int(changed.sum()))
print("=== 03.5.10f v2 END ===")


=== 03.5.10f v2 START (DEDUP NOTES: stable + safe display) ===


,school_name,city,state,decision,chosen_school_id,notes,detail_url
6,east bay school,berkeley,CA,manual_match,PRI_A1300209,manual_match override,https://www.caisca.org/schools/east-bay-school
0,escuela bilinga1 4e internacional,emeryville oakland,CA,manual_match,PRI_A0770343,forced match: CAIS encoding/city multi-token |...,https://www.caisca.org/schools/escuela-bilingu...
7,kehillah school,palo alto,CA,manual_match,PRI_A0500422,forced to PSS PRI_A0500422,https://www.caisca.org/schools/kehillah--school
11,millennium school,san francisco,CA,manual_match,PRI_A1700371,manual_match override,https://www.caisca.org/schools/millennium-school
13,nueva school,hillsborough,CA,manual_match,PRI_A1790096,forced to PSS PRI_A1790096,https://www.caisca.org/schools/the-nueva-school


Rows with notes changed: 0
=== 03.5.10f v2 END ===


In [1226]:
def has_dupe_pipe_parts(val):
    s = ("" if val is None else str(val)).strip()
    if not s:
        return False
    parts = [" ".join(p.strip().lower().split()) for p in s.split("|") if p.strip()]
    return len(parts) != len(set(parts))

mask_still_dupe = df["notes"].apply(has_dupe_pipe_parts)

print("Rows still with duplicate note parts:", int(mask_still_dupe.sum()))
if mask_still_dupe.any():
    display(df.loc[mask_still_dupe, ["school_name","city","state","decision","notes"]])


Rows still with duplicate note parts: 0


## 03.5.11 — Finalize CAIS Integration (Dedupe Guarantees, Invariants, and Handoff)

At this point we have completed the CAIS ↔ PSS matching workflow and have explicitly handled residuals.

**Inputs we consider authoritative at this step:**
- `pss_clean_df` — standardized PSS private-school backbone (includes `school_id = PRI_<ppin>`).
- `cais_presence_final_v4_df` — the final CAIS presence table across:
  - matched via deterministic join (`cityfix`)
  - matched via alias/rematch/manual overrides
  - minted CAIS-only records (when no PSS record exists)
- `cais_backbone_minted_df` — minted backbone rows for CAIS-only schools (CAIS as authority).
- `residual_disposition_df` — final residual decisions + notes (audit trail).

**What this section guarantees:**
1. **Uniqueness:** `school_id` is globally unique across PSS + minted CAIS.
2. **No collisions:** minted CAIS `school_id` does not collide with PSS `PRI_*` ids.
3. **Presence validity:** every `school_id` in `cais_presence_final_v4_df` exists in the combined backbone (PSS ∪ minted).
4. **Schema readiness:** we output handoff artifacts that downstream `schools_master` can consume:
   - `cais_presence_final_v4_df` (presence flag)
   - `cais_backbone_minted_df` (new backbone rows to append)
   - `cais_integration_manifest_v1` (counts + invariants summary)

**Downstream handoff to `schools_master`:**
- Append `cais_backbone_minted_df` into the backbone table.
- Left-join `cais_presence_final_v4_df` onto backbone by `school_id` to set `has_cais=True`.
- Keep `residual_disposition_df` as audit metadata (not merged into the backbone by default).


In [1229]:
# ============================================================
# 03.5.11 — FINALIZE CAIS INTEGRATION (DEDUP + INVARIANTS + HANDOFF)
# ============================================================

print("=== 03.5.11 START (FINALIZE CAIS INTEGRATION) ===")

# -----------------------------
# Preconditions
# -----------------------------
assert "pss_clean_df" in globals(), "Need pss_clean_df."
assert "cais_presence_final_v4_df" in globals(), "Need cais_presence_final_v4_df from 03.5.9."
assert "cais_backbone_minted_df" in globals(), "Need cais_backbone_minted_df from 03.5.9."
assert "residual_disposition_df" in globals(), "Need residual_disposition_df."

pss = pss_clean_df.copy()
presence = cais_presence_final_v4_df.copy()
minted = cais_backbone_minted_df.copy()
disp = residual_disposition_df.copy()

# -----------------------------
# Normalize minimal schemas
# -----------------------------
def _assert_cols(df, df_name, req):
    missing = [c for c in req if c not in df.columns]
    assert not missing, f"{df_name} missing required columns: {missing}"

_assert_cols(pss, "pss_clean_df", ["school_id"])
_assert_cols(presence, "cais_presence_final_v4_df", ["school_id", "has_cais"])

# minted may be empty; require school_id if not empty
if not minted.empty:
    _assert_cols(minted, "cais_backbone_minted_df", ["school_id"])
    if "name" not in minted.columns and "school_name" in minted.columns:
        minted = minted.rename(columns={"school_name": "name"})

# -----------------------------
# Ensure types
# -----------------------------
pss["school_id"] = pss["school_id"].fillna("").astype(str).str.strip()

presence["school_id"] = presence["school_id"].fillna("").astype(str).str.strip()
presence["has_cais"] = presence["has_cais"].fillna(False).astype(bool)

if not minted.empty:
    minted["school_id"] = minted["school_id"].fillna("").astype(str).str.strip()

# -----------------------------
# Build combined backbone universe (PSS ∪ minted)
# -----------------------------
pss_ids = set(pss["school_id"].tolist())
minted_ids = set(minted["school_id"].tolist()) if not minted.empty else set()
combined_ids = pss_ids | minted_ids

# -----------------------------
# Invariants / Guarantees
# -----------------------------
issues = []

# 1) PSS school_id uniqueness
pss_dupes = int(pss["school_id"].duplicated().sum())
if pss_dupes != 0:
    issues.append(f"PSS has duplicate school_id rows: {pss_dupes}")

# 2) Minted school_id uniqueness
if not minted.empty:
    minted_dupes = int(minted["school_id"].duplicated().sum())
    if minted_dupes != 0:
        issues.append(f"Minted CAIS has duplicate school_id rows: {minted_dupes}")

# 3) Presence must point to real backbone ids
missing_presence = sorted(list(set(presence["school_id"]) - combined_ids))
if missing_presence:
    issues.append(
        "Presence contains school_ids not found in (PSS ∪ minted). "
        f"Example: {missing_presence[:10]}"
    )

# 4) Sanity: presence.has_cais should be boolean & non-null (already coerced)
if presence["has_cais"].isna().any():
    issues.append("Presence has null has_cais values (should not happen after coercion).")

# 5) Optional strong check: if presence is CAIS-only, then all has_cais should be True
# (Only enforce if it *looks* like a CAIS-only table: no False values expected)
# Comment this out if your presence table is for the full backbone.
if presence["has_cais"].mean() > 0.95 and (not presence["has_cais"].all()):
    bad = presence[~presence["has_cais"]].head(10)
    issues.append(f"Presence appears CAIS-only but has has_cais!=True. Example:\n{bad}")

# 6) No collisions between PSS ids and minted ids (should be 0 if you never reuse ids)
# If you intentionally map CAIS rows onto PSS via manual_match, those rows should NOT remain minted.
collisions = sorted(list(pss_ids.intersection(minted_ids)))
if collisions:
    issues.append(f"Minted ids collide with PSS ids (example): {collisions[:10]}")

# Hard fail if any invariant broken
if issues:
    print("\n❌ INVARIANT FAILURES:")
    for msg in issues:
        print(" -", msg)
    raise AssertionError("03.5.11 invariants failed. Fix issues above before proceeding.")
else:
    print("\n✅ All invariants passed.")

# -----------------------------
# Handoff artifacts (for schools_master)
# -----------------------------
# (A) Presence table (deduped)
cais_presence_final_v4_df = (
    presence[["school_id", "has_cais"]]
    .drop_duplicates(subset=["school_id"])
    .sort_values("school_id")
    .reset_index(drop=True)
)

# (B) Minted backbone rows (deduped)
if not minted.empty:
    cais_backbone_minted_final_v1_df = minted.drop_duplicates(subset=["school_id"]).reset_index(drop=True)
else:
    cais_backbone_minted_final_v1_df = pd.DataFrame(
        columns=["school_id", "name", "city", "state", "detail_url", "backbone_source", "is_private", "has_cais"]
    )

# (C) Manifest / summary
cais_integration_manifest_v1 = {
    "pss_rows": int(len(pss)),
    "pss_unique_school_ids": int(len(pss_ids)),
    "minted_rows": int(len(minted)),
    "minted_unique_school_ids": int(len(minted_ids)),
    "presence_rows": int(len(presence)),
    "presence_unique_school_ids": int(presence["school_id"].nunique()),
    "presence_points_missing_from_backbone": int(len(missing_presence)),
    "collisions_pss_vs_minted": int(len(collisions)),
    "residual_disposition_rows": int(len(disp)),
}

print("\n=== CAIS Integration Manifest (v1) ===")
for k, v in cais_integration_manifest_v1.items():
    print(f"{k}: {v}")

print("\nHandoff tables:")
print(" - cais_presence_final_v4_df           :", cais_presence_final_v4_df.shape)
print(" - cais_backbone_minted_final_v1_df    :", cais_backbone_minted_final_v1_df.shape)

display(cais_presence_final_v4_df.head(20))
display(cais_backbone_minted_final_v1_df.head(20))

print("=== 03.5.11 END ===")


=== 03.5.11 START (FINALIZE CAIS INTEGRATION) ===

✅ All invariants passed.

=== CAIS Integration Manifest (v1) ===
pss_rows: 22345
pss_unique_school_ids: 22345
minted_rows: 15
minted_unique_school_ids: 15
presence_rows: 96
presence_unique_school_ids: 96
presence_points_missing_from_backbone: 0
collisions_pss_vs_minted: 0
residual_disposition_rows: 18

Handoff tables:
 - cais_presence_final_v4_df           : (96, 2)
 - cais_backbone_minted_final_v1_df    : (15, 10)


,school_id,has_cais
0,CAIS_CA_bentley-school-lafayette-oakland-ca_d3...,True
1,CAIS_CA_brandeis-marin-san-rafael-ca_d4ec4a3f,True
2,CAIS_CA_cathedral-school-for-boys-san-francisc...,True
3,CAIS_CA_chinese-american-international-school-...,True
4,CAIS_CA_crystal-springs-uplands-school-belmont...,True
5,CAIS_CA_east-bay-school-berkeley-ca_81210394,True
6,CAIS_CA_field-middle-school-san-mateo-ca_05b31bfa,True
7,CAIS_CA_helios-school-sunnyvale-ca_f7846b84,True
8,CAIS_CA_keys-school-palo-alto-ca_bfbd82c4,True
9,CAIS_CA_lick-wilmerding-high-school-san-franci...,True


,school_id,name,city,state,detail_url,backbone_source,is_private,has_cais,nces_id,ppin
0,CAIS_CA_bentley-school-lafayette-oakland-ca_d3...,bentley school,lafayette oakland,CA,https://www.caisca.org/schools/bentley-school,cais,True,True,NaN,NaN
1,CAIS_CA_brandeis-marin-san-rafael-ca_d4ec4a3f,brandeis marin,san rafael,CA,https://www.caisca.org/schools/brandeis-marin,cais,True,True,NaN,NaN
2,CAIS_CA_cathedral-school-for-boys-san-francisc...,cathedral school for boys,san francisco,CA,https://www.caisca.org/schools/cathedral-schoo...,cais,True,True,NaN,NaN
3,CAIS_CA_chinese-american-international-school-...,chinese american international school,san francisco,CA,https://www.caisca.org/schools/chinese-america...,cais,True,True,NaN,NaN
4,CAIS_CA_crystal-springs-uplands-school-belmont...,crystal springs uplands school,belmont hillsborough,CA,https://www.caisca.org/schools/crystal-springs...,cais,True,True,NaN,NaN
5,CAIS_CA_east-bay-school-berkeley-ca_81210394,east bay school,berkeley,CA,https://www.caisca.org/schools/east-bay-school,cais,True,True,NaN,NaN
6,CAIS_CA_keys-school-palo-alto-ca_bfbd82c4,keys school,palo alto,CA,https://www.caisca.org/schools/keys-school,cais,True,True,NaN,NaN
7,CAIS_CA_lick-wilmerding-high-school-san-franci...,lick wilmerding high school,san francisco,CA,https://www.caisca.org/schools/lick-wilmerding...,cais,True,True,NaN,NaN
8,CAIS_CA_lyca-e-frana-ais-de-san-francisco-san-...,lyca e frana ais de san francisco,san francisco,CA,https://www.caisca.org/schools/lycee-francais-...,cais,True,True,NaN,NaN
9,CAIS_CA_millennium-school-san-francisco-ca_d53...,millennium school,san francisco,CA,https://www.caisca.org/schools/millennium-school,cais,True,True,NaN,NaN


=== 03.5.11 END ===


In [1231]:
# ============================================================
# 03.5.11b — APPLY DISPOSITION (manual_match PRI_* rewrites)
# Goal:
#   If disp says decision=manual_match and chosen_school_id is PRI_*,
#   then:
#     1) ensure presence includes that PRI_* with has_cais=True
#     2) remove the corresponding CAIS_CA_* minted backbone row (same school)
# ============================================================

print("=== 03.5.11b START (APPLY DISPOSITION: manual_match PRI rewrites) ===")

assert "residual_disposition_df" in globals()
assert "cais_presence_final_v4_df" in globals()
assert "cais_backbone_minted_final_v1_df" in globals()

disp = residual_disposition_df.copy()
presence = cais_presence_final_v4_df.copy()
minted = cais_backbone_minted_final_v1_df.copy()

# normalize
for col in ["school_name", "city", "state", "detail_url", "decision", "chosen_school_id", "notes"]:
    if col in disp.columns:
        disp[col] = disp[col].fillna("").astype(str)

presence["school_id"] = presence["school_id"].fillna("").astype(str).str.strip()
presence["has_cais"] = presence["has_cais"].fillna(False).astype(bool)

minted["school_id"] = minted["school_id"].fillna("").astype(str).str.strip()
for c in ["name", "city", "state", "detail_url"]:
    if c in minted.columns:
        minted[c] = minted[c].fillna("").astype(str)

def canon_key(name, city, state):
    return f"{name.strip().lower()}|{city.strip().lower()}|{state.strip().upper()}"

# Identify manual_match -> PRI targets
mm = disp[disp["decision"].str.strip().eq("manual_match")].copy()
mm["chosen_school_id"] = mm["chosen_school_id"].str.strip()
mm_pri = mm[mm["chosen_school_id"].str.startswith("PRI_")].copy()

print("manual_match rows:", len(mm))
print("manual_match rows with chosen PRI_*:", len(mm_pri))

# Build lookup from minted -> canon key (so we can drop corresponding CAIS_CA row)
minted["_canon_key"] = minted.apply(lambda r: canon_key(r.get("name",""), r.get("city",""), r.get("state","")), axis=1)
mm_pri["_canon_key"] = mm_pri.apply(lambda r: canon_key(r.get("school_name",""), r.get("city",""), r.get("state","")), axis=1)

# 1) Add/ensure presence PRI rows
pri_ids = sorted(set(mm_pri["chosen_school_id"].tolist()))
presence_before = len(presence)
present_set = set(presence["school_id"].tolist())

to_add = [pid for pid in pri_ids if pid not in present_set]
if to_add:
    add_df = pd.DataFrame({"school_id": to_add, "has_cais": True})
    presence = pd.concat([presence, add_df], ignore_index=True)

# also force has_cais True for these PRI ids (in case any existed as False)
presence.loc[presence["school_id"].isin(pri_ids), "has_cais"] = True

# 2) Drop minted CAIS_CA rows that correspond to those manual matches
# (match by canon_key; only drop if the minted id is CAIS_CA_* to avoid accidental PSS deletions)
drop_keys = set(mm_pri["_canon_key"].tolist())
drop_mask = minted["_canon_key"].isin(drop_keys) & minted["school_id"].str.startswith("CAIS_")
dropped = minted.loc[drop_mask, ["school_id","name","city","state","detail_url"]].copy()
minted = minted.loc[~drop_mask].copy()

# cleanup
presence = presence.drop_duplicates(subset=["school_id"]).sort_values("school_id").reset_index(drop=True)
minted = minted.drop(columns=["_canon_key"]).reset_index(drop=True)

print("\nPresence rows before/after:", presence_before, "->", len(presence))
print("Minted rows before/after  :", len(cais_backbone_minted_final_v1_df), "->", len(minted))
print("Dropped minted rows:", len(dropped))

if len(dropped) > 0:
    display(dropped.sort_values(["name","city"]).reset_index(drop=True))

# write back
cais_presence_final_v4_df = presence
cais_backbone_minted_final_v1_df = minted

# quick verification: chosen PRI ids now present
missing_after = sorted(list(set(pri_ids) - set(cais_presence_final_v4_df["school_id"].tolist())))
print("\nMissing chosen PRI ids after rewrite:", missing_after)

print("=== 03.5.11b END ===")


=== 03.5.11b START (APPLY DISPOSITION: manual_match PRI rewrites) ===
manual_match rows: 5
manual_match rows with chosen PRI_*: 5

Presence rows before/after: 96 -> 98
Minted rows before/after  : 15 -> 13
Dropped minted rows: 2


,school_id,name,city,state,detail_url
0,CAIS_CA_east-bay-school-berkeley-ca_81210394,east bay school,berkeley,CA,https://www.caisca.org/schools/east-bay-school
1,CAIS_CA_millennium-school-san-francisco-ca_d53...,millennium school,san francisco,CA,https://www.caisca.org/schools/millennium-school



Missing chosen PRI ids after rewrite: []
=== 03.5.11b END ===


In [1233]:
# Sanity re-check
disp = residual_disposition_df.copy()
mm = disp[disp["decision"].astype(str).eq("manual_match")].copy()
chosen = mm["chosen_school_id"].fillna("").astype(str).str.strip()
missing = sorted(list(set(chosen[chosen.str.startswith("PRI_")]) - set(cais_presence_final_v4_df["school_id"])))
print("Missing manual-match PRI ids in presence (should be empty):", missing)
print("Minted rows now:", len(cais_backbone_minted_final_v1_df))


Missing manual-match PRI ids in presence (should be empty): []
Minted rows now: 13


In [1235]:
# ============================================================
# 03.5.11b.1 — PRUNE presence rows that were rewritten to PRI_*
# (remove CAIS_* ids for manual_match schools whose CAIS minted row was dropped)
# ============================================================

print("=== 03.5.11b.1 START (PRUNE rewritten CAIS_* from presence) ===")

assert "residual_disposition_df" in globals()
assert "cais_presence_final_v4_df" in globals()
assert "cais_backbone_minted_final_v1_df" in globals()

disp = residual_disposition_df.copy()
presence = cais_presence_final_v4_df.copy()
minted = cais_backbone_minted_final_v1_df.copy()

presence["school_id"] = presence["school_id"].fillna("").astype(str).str.strip()
disp["school_name"] = disp["school_name"].fillna("").astype(str).str.strip().str.lower()
disp["decision"] = disp["decision"].fillna("").astype(str).str.strip()
disp["chosen_school_id"] = disp["chosen_school_id"].fillna("").astype(str).str.strip()

# Identify manual_match rows that map to PRI_*
mm = disp[(disp["decision"] == "manual_match") & (disp["chosen_school_id"].str.startswith("PRI_"))].copy()
print("manual_match PRI_* rows:", len(mm))

# Build CAIS_* ids to drop by matching minted rows on name+city (robust enough here)
minted_key = (
    minted.assign(
        _name=minted["name"].fillna("").astype(str).str.strip().str.lower(),
        _city=minted["city"].fillna("").astype(str).str.strip().str.lower(),
    )[["school_id","_name","_city"]]
)

mm_key = (
    mm.assign(
        _name=mm["school_name"].fillna("").astype(str).str.strip().str.lower(),
        _city=mm["city"].fillna("").astype(str).str.strip().str.lower(),
    )[["_name","_city","chosen_school_id"]]
)

mm_join = mm_key.merge(minted_key, on=["_name","_city"], how="left", suffixes=("","_minted"))
cais_ids_to_drop = sorted([x for x in mm_join["school_id"].dropna().unique().tolist() if str(x).startswith("CAIS_")])

print("CAIS_* presence rows to drop (rewritten to PRI_*):", len(cais_ids_to_drop))
if cais_ids_to_drop:
    print("Example:", cais_ids_to_drop[:10])

before = len(presence)
presence2 = presence[~presence["school_id"].isin(cais_ids_to_drop)].copy()
after = len(presence2)

print(f"Presence rows before/after prune: {before} -> {after}")
still_missing = [sid for sid in cais_ids_to_drop if sid in set(presence2["school_id"])]
print("Still present after prune (should be 0):", len(still_missing))

# Write back
cais_presence_final_v4_df = presence2

print("=== 03.5.11b.1 END ===")


=== 03.5.11b.1 START (PRUNE rewritten CAIS_* from presence) ===
manual_match PRI_* rows: 5
CAIS_* presence rows to drop (rewritten to PRI_*): 0
Presence rows before/after prune: 98 -> 98
Still present after prune (should be 0): 0
=== 03.5.11b.1 END ===


In [1237]:
# ============================================================
# 03.5.11b.2 — HARD PRUNE presence to (PSS ∪ minted) universe
# (guarantees 03.5.11c invariant #4)
# ============================================================

print("=== 03.5.11b.2 START (HARD PRUNE presence to backbone) ===")

assert "pss_clean_df" in globals()
assert "cais_presence_final_v4_df" in globals()
assert "cais_backbone_minted_final_v1_df" in globals()

pss = pss_clean_df.copy()
presence = cais_presence_final_v4_df.copy()
minted = cais_backbone_minted_final_v1_df.copy()

pss["school_id"] = pss["school_id"].fillna("").astype(str).str.strip()
presence["school_id"] = presence["school_id"].fillna("").astype(str).str.strip()

minted_ids = set()
if not minted.empty and "school_id" in minted.columns:
    minted["school_id"] = minted["school_id"].fillna("").astype(str).str.strip()
    minted_ids = set(minted["school_id"].tolist())

combined_ids = set(pss["school_id"].tolist()) | minted_ids

before = len(presence)
missing_before = sorted(list(set(presence["school_id"]) - combined_ids))

print("Presence ids missing from backbone BEFORE prune:", len(missing_before))
if missing_before:
    print("Example:", missing_before[:10])

presence2 = presence[presence["school_id"].isin(combined_ids)].copy()
after = len(presence2)

print(f"Presence rows before/after prune: {before} -> {after}")

# write back
cais_presence_final_v4_df = presence2

# confirm
missing_after = sorted(list(set(cais_presence_final_v4_df["school_id"]) - combined_ids))
print("Presence ids missing from backbone AFTER prune:", len(missing_after))

print("=== 03.5.11b.2 END ===")


=== 03.5.11b.2 START (HARD PRUNE presence to backbone) ===
Presence ids missing from backbone BEFORE prune: 2
Example: ['CAIS_CA_east-bay-school-berkeley-ca_81210394', 'CAIS_CA_millennium-school-san-francisco-ca_d53051d7']
Presence rows before/after prune: 98 -> 96
Presence ids missing from backbone AFTER prune: 0
=== 03.5.11b.2 END ===


In [1239]:
# ============================================================
# 03.5.11c — REFREEZE CAIS HANDOFF (post-disposition rewrite)
# Ensures: presence ⊆ (PSS ∪ minted_final) after manual_match PRI rewrites
# ============================================================

print("=== 03.5.11c START (REFREEZE CAIS HANDOFF) ===")

# -----------------------------
# Preconditions
# -----------------------------
assert "pss_clean_df" in globals(), "Need pss_clean_df."
assert "cais_presence_final_v4_df" in globals(), "Need cais_presence_final_v4_df."
assert "cais_backbone_minted_final_v1_df" in globals(), "Need cais_backbone_minted_final_v1_df."
assert "residual_disposition_df" in globals(), "Need residual_disposition_df."

pss = pss_clean_df.copy()
presence = cais_presence_final_v4_df.copy()
minted = cais_backbone_minted_final_v1_df.copy()
disp = residual_disposition_df.copy()

# -----------------------------
# Normalize minimal schemas
# -----------------------------
for df_name, df, req in [
    ("pss_clean_df", pss, ["school_id"]),
    ("cais_presence_final_v4_df", presence, ["school_id", "has_cais"]),
]:
    missing = [c for c in req if c not in df.columns]
    assert not missing, f"{df_name} missing required columns: {missing}"

# minted may be empty; require school_id if not empty
if not minted.empty:
    assert "school_id" in minted.columns, "cais_backbone_minted_final_v1_df must include school_id"
    # normalize minted columns for downstream (keep your existing schema)
    if "name" not in minted.columns and "school_name" in minted.columns:
        minted = minted.rename(columns={"school_name": "name"})

# Ensure types
pss["school_id"] = pss["school_id"].fillna("").astype(str).str.strip()

presence["school_id"] = presence["school_id"].fillna("").astype(str).str.strip()
presence["has_cais"] = presence["has_cais"].fillna(False).astype(bool)

if not minted.empty:
    minted["school_id"] = minted["school_id"].fillna("").astype(str).str.strip()

# -----------------------------
# Build combined backbone universe (PSS ∪ minted)
# -----------------------------
pss_ids = set(pss["school_id"].tolist())
minted_ids = set(minted["school_id"].tolist()) if not minted.empty else set()
combined_ids = pss_ids | minted_ids

# -----------------------------
# Invariants / Guarantees
# -----------------------------
issues = []

# 1) PSS school_id uniqueness
pss_dupes = int(pss["school_id"].duplicated().sum())
if pss_dupes != 0:
    issues.append(f"PSS has duplicate school_id rows: {pss_dupes}")

# 2) Minted school_id uniqueness
if not minted.empty:
    minted_dupes = int(minted["school_id"].duplicated().sum())
    if minted_dupes != 0:
        issues.append(f"Minted CAIS has duplicate school_id rows: {minted_dupes}")

# 3) No collisions between PSS ids and minted ids
collisions = sorted(list(pss_ids.intersection(minted_ids)))
if collisions:
    issues.append(f"Minted ids collide with PSS ids (example): {collisions[:10]}")

# 4) Presence must point to real backbone ids
missing_presence = sorted(list(set(presence["school_id"]) - combined_ids))
if missing_presence:
    issues.append(
        "Presence contains school_ids not found in (PSS ∪ minted). "
        f"Example: {missing_presence[:10]}"
    )

# 5) sanity: CAIS presence must be True for all rows (should be)
if not presence["has_cais"].all():
    bad = presence[~presence["has_cais"]].head(10)
    issues.append(f"Presence has rows with has_cais!=True. Example:\n{bad}")

# Hard fail if any invariant broken
if issues:
    print("\n❌ INVARIANT FAILURES:")
    for msg in issues:
        print(" -", msg)
    raise AssertionError("03.5.11c invariants failed.")
else:
    print("\n✅ All invariants passed.")

# -----------------------------
# Re-freeze handoff artifacts (canonical)
# -----------------------------
# (A) Presence table (deduped, sorted)
cais_presence_handoff_v1_df = (
    presence[["school_id", "has_cais"]]
    .drop_duplicates(subset=["school_id"])
    .sort_values("school_id")
    .reset_index(drop=True)
)

# (B) Minted backbone rows (deduped)
cais_backbone_minted_handoff_v1_df = (
    minted.drop_duplicates(subset=["school_id"]).reset_index(drop=True)
    if not minted.empty
    else pd.DataFrame(
        columns=[
            "school_id", "name", "city", "state", "detail_url",
            "backbone_source", "is_private", "has_cais", "nces_id", "ppin"
        ]
    )
)

# (C) Manifest / summary
cais_integration_manifest_v2 = {
    "pss_rows": int(len(pss)),
    "pss_unique_school_ids": int(len(pss_ids)),
    "minted_rows": int(len(minted)),
    "minted_unique_school_ids": int(len(minted_ids)),
    "presence_rows": int(len(presence)),
    "presence_unique_school_ids": int(presence["school_id"].nunique()),
    "presence_points_missing_from_backbone": int(len(missing_presence)),
    "collisions_pss_vs_minted": int(len(collisions)),
    "residual_disposition_rows": int(len(disp)),
    "presence_pri_rows": int(presence["school_id"].astype(str).str.startswith("PRI_").sum()),
    "presence_cais_rows": int(presence["school_id"].astype(str).str.startswith("CAIS_").sum()),
}

print("\n=== CAIS Integration Manifest (v2) ===")
for k, v in cais_integration_manifest_v2.items():
    print(f"{k}: {v}")

print("\nHandoff tables (refrozen):")
print(" - cais_presence_handoff_v1_df        :", cais_presence_handoff_v1_df.shape)
print(" - cais_backbone_minted_handoff_v1_df :", cais_backbone_minted_handoff_v1_df.shape)

display(cais_presence_handoff_v1_df.head(20))
display(cais_backbone_minted_handoff_v1_df.head(20))

# -----------------------------
# Optional: update the canonical variable names used downstream
# -----------------------------
cais_presence_final_v4_df = cais_presence_handoff_v1_df
cais_backbone_minted_final_v1_df = cais_backbone_minted_handoff_v1_df

print("=== 03.5.11c END ===")


=== 03.5.11c START (REFREEZE CAIS HANDOFF) ===

✅ All invariants passed.

=== CAIS Integration Manifest (v2) ===
pss_rows: 22345
pss_unique_school_ids: 22345
minted_rows: 13
minted_unique_school_ids: 13
presence_rows: 96
presence_unique_school_ids: 96
presence_points_missing_from_backbone: 0
collisions_pss_vs_minted: 0
residual_disposition_rows: 18
presence_pri_rows: 83
presence_cais_rows: 13

Handoff tables (refrozen):
 - cais_presence_handoff_v1_df        : (96, 2)
 - cais_backbone_minted_handoff_v1_df : (13, 10)


,school_id,has_cais
0,CAIS_CA_bentley-school-lafayette-oakland-ca_d3...,True
1,CAIS_CA_brandeis-marin-san-rafael-ca_d4ec4a3f,True
2,CAIS_CA_cathedral-school-for-boys-san-francisc...,True
3,CAIS_CA_chinese-american-international-school-...,True
4,CAIS_CA_crystal-springs-uplands-school-belmont...,True
5,CAIS_CA_field-middle-school-san-mateo-ca_05b31bfa,True
6,CAIS_CA_helios-school-sunnyvale-ca_f7846b84,True
7,CAIS_CA_keys-school-palo-alto-ca_bfbd82c4,True
8,CAIS_CA_lick-wilmerding-high-school-san-franci...,True
9,CAIS_CA_lyca-e-frana-ais-de-san-francisco-san-...,True


,school_id,name,city,state,detail_url,backbone_source,is_private,has_cais,nces_id,ppin
0,CAIS_CA_bentley-school-lafayette-oakland-ca_d3...,bentley school,lafayette oakland,CA,https://www.caisca.org/schools/bentley-school,cais,True,True,NaN,NaN
1,CAIS_CA_brandeis-marin-san-rafael-ca_d4ec4a3f,brandeis marin,san rafael,CA,https://www.caisca.org/schools/brandeis-marin,cais,True,True,NaN,NaN
2,CAIS_CA_cathedral-school-for-boys-san-francisc...,cathedral school for boys,san francisco,CA,https://www.caisca.org/schools/cathedral-schoo...,cais,True,True,NaN,NaN
3,CAIS_CA_chinese-american-international-school-...,chinese american international school,san francisco,CA,https://www.caisca.org/schools/chinese-america...,cais,True,True,NaN,NaN
4,CAIS_CA_crystal-springs-uplands-school-belmont...,crystal springs uplands school,belmont hillsborough,CA,https://www.caisca.org/schools/crystal-springs...,cais,True,True,NaN,NaN
5,CAIS_CA_keys-school-palo-alto-ca_bfbd82c4,keys school,palo alto,CA,https://www.caisca.org/schools/keys-school,cais,True,True,NaN,NaN
6,CAIS_CA_lick-wilmerding-high-school-san-franci...,lick wilmerding high school,san francisco,CA,https://www.caisca.org/schools/lick-wilmerding...,cais,True,True,NaN,NaN
7,CAIS_CA_lyca-e-frana-ais-de-san-francisco-san-...,lyca e frana ais de san francisco,san francisco,CA,https://www.caisca.org/schools/lycee-francais-...,cais,True,True,NaN,NaN
8,CAIS_CA_montessori-de-terra-linda-san-rafael-c...,montessori de terra linda,san rafael,CA,https://www.caisca.org/schools/montessori-de-t...,cais,True,True,NaN,NaN
9,CAIS_CA_park-day-school-oakland-ca_5c5d05c8,park day school,oakland,CA,https://www.caisca.org/schools/park-day-school,cais,True,True,NaN,NaN


=== 03.5.11c END ===


## 03.5.12 Export — CAIS Presence Flag (Authoritative)

Now that **03.5.11** has passed all invariants, we will export the **authoritative** CAIS enrichment flag table for the rest of the pipeline.

### What this export represents
- A **single source of truth**: `school_id → has_cais`
- Includes:
  - **PSS-backed matches** (e.g., Harker matched to `PRI_A2190096`)
  - **Minted CAIS-only records** (for CAIS schools missing from the private backbone)
- This file is what **Notebook 04** should load and left-join into `schools_master`.

### Output
- `data/processed/notebook04/enrichment_flags/has_cais_pss.csv`

### Contract
- Columns: `school_id`, `has_cais`
- One row per `school_id`
- Deterministic + idempotent (safe to overwrite)


In [1245]:
# ============================================================
# 03.5.12 — Export CAIS Presence Flag (Authoritative)
# ============================================================

print("=== 03.5.12 START (EXPORT CAIS PRESENCE FLAG) ===")

from pathlib import Path
import pandas as pd

# -----------------------------
# Input selection (refrozen wins)
# -----------------------------
if "cais_presence_handoff_v1_df" in globals():
    source_df = cais_presence_handoff_v1_df
    source_name = "cais_presence_handoff_v1_df"
elif "cais_presence_final_v4_df" in globals():
    source_df = cais_presence_final_v4_df
    source_name = "cais_presence_final_v4_df"
else:
    raise AssertionError("Expected cais_presence_handoff_v1_df or cais_presence_final_v4_df in globals()")

print(f"Using source: {source_name}  shape={source_df.shape}")

# -----------------------------
# Output path
# -----------------------------
out_dir = Path("../data/processed/notebook04/enrichment_flags")
out_dir.mkdir(parents=True, exist_ok=True)
out_path = out_dir / "has_cais_pss.csv"

# -----------------------------
# Normalize + validate contract
# -----------------------------
export_df = source_df.copy()

required_cols = {"school_id", "has_cais"}
missing = required_cols - set(export_df.columns)
assert not missing, f"Missing required columns in {source_name}: {missing}"

export_df["school_id"] = export_df["school_id"].fillna("").astype(str).str.strip()
export_df["has_cais"] = export_df["has_cais"].fillna(False).astype(bool)

# drop empty ids
export_df = export_df[export_df["school_id"] != ""]

# enforce one row per school_id
export_df = (
    export_df[["school_id", "has_cais"]]
    .drop_duplicates(subset=["school_id"], keep="first")
    .sort_values("school_id")
    .reset_index(drop=True)
)

# -----------------------------
# Final invariant checks (export contract)
# -----------------------------
assert export_df["school_id"].nunique() == export_df.shape[0], "Export must be unique by school_id"
assert export_df["has_cais"].all(), "Presence export should be all True (CAIS members only)"

# -----------------------------
# Write (deterministic + idempotent)
# -----------------------------
export_df.to_csv(out_path, index=False)

print("✅ Saved:", out_path)
print("Rows:", export_df.shape[0])
display(export_df.head(20))

print("=== 03.5.12 END ===")



=== 03.5.12 START (EXPORT CAIS PRESENCE FLAG) ===
Using source: cais_presence_handoff_v1_df  shape=(96, 2)
✅ Saved: ../data/processed/notebook04/enrichment_flags/has_cais_pss.csv
Rows: 96


,school_id,has_cais
0,CAIS_CA_bentley-school-lafayette-oakland-ca_d3...,True
1,CAIS_CA_brandeis-marin-san-rafael-ca_d4ec4a3f,True
2,CAIS_CA_cathedral-school-for-boys-san-francisc...,True
3,CAIS_CA_chinese-american-international-school-...,True
4,CAIS_CA_crystal-springs-uplands-school-belmont...,True
5,CAIS_CA_field-middle-school-san-mateo-ca_05b31bfa,True
6,CAIS_CA_helios-school-sunnyvale-ca_f7846b84,True
7,CAIS_CA_keys-school-palo-alto-ca_bfbd82c4,True
8,CAIS_CA_lick-wilmerding-high-school-san-franci...,True
9,CAIS_CA_lyca-e-frana-ais-de-san-francisco-san-...,True


=== 03.5.12 END ===


In [1249]:
pss_clean_df[
    pss_clean_df["school_name"]
      .str.lower()
      .str.contains("harker", na=False)
][["school_id","school_name","city","state"]]

,school_id,school_name,city,state
17137,PRI_A2190096,harker,san jose,CA


In [1243]:
print("=== AUDIT: is Harker in the final CAIS flag file? ===")

# 1) Check CAIS match summary
harker_in_summary = summary[summary["school_name"].str.contains("harker", na=False)][
    ["school_name", "city", "state", "school_id_cityfix", "school_id_fallback", "school_id_final", "matched_final"]
]
display(harker_in_summary)

# 2) Check it made it into the final presence table
harker_in_presence = cais_presence_pss_final_df.merge(
    pss_clean_df[["school_id", "school_name", "city", "state"]],
    left_on="school_id",
    right_on="school_id",
    how="left"
)

display(harker_in_presence[harker_in_presence["school_name"].str.contains("harker", na=False)])

# 3) Check the saved CSV contains it
saved_flag = pd.read_csv(PROCESSED_DIR_notebook04 / "enrichment_flags" / "has_cais_pss.csv")
harker_in_saved = saved_flag.merge(
    pss_clean_df[["school_id", "school_name", "city", "state"]],
    on="school_id",
    how="left"
)

display(harker_in_saved[harker_in_saved["school_name"].str.contains("harker", na=False)])


=== AUDIT: is Harker in the final CAIS flag file? ===


KeyError: "['school_id_cityfix', 'school_id_fallback', 'school_id_final', 'matched_final'] not in index"

In [880]:
print("=== DEBUG: find Harker in PSS (CA only) ===")

pss_ca = pss_clean_df[pss_clean_df["state"] == "CA"].copy()

# 1) raw contains "harker"
harker_pss = pss_ca[pss_ca["school_name"].str.contains("harker", na=False)]
print("PSS rows containing 'harker':", harker_pss.shape[0])
display(harker_pss[["school_id", "ppin", "school_name", "city", "state", "join_key"]].head(50))

# 2) if none, try contains "harker" in city+address too (rare, but just in case)
if harker_pss.shape[0] == 0 and "address" in pss_ca.columns:
    harker_pss2 = pss_ca[
        pss_ca["address"].fillna("").str.contains("harker", case=False, na=False)
    ]
    print("PSS address containing 'harker':", harker_pss2.shape[0])
    display(harker_pss2[["school_id", "ppin", "school_name", "city", "state", "address"]].head(50))


=== DEBUG: find Harker in PSS (CA only) ===
PSS rows containing 'harker': 1


,school_id,ppin,school_name,city,state,join_key
17137,PRI_A2190096,A2190096,harker,san jose,CA,harker|san jose|CA


In [882]:
print("=== DEBUG: compare CAIS vs PSS keys for Harker ===")

# CAIS Harker row
cais_harker = cais_clean_df[cais_clean_df["school_name"].str.contains("harker", na=False)].copy()
display(cais_harker[["school_name","city","state","join_key","detail_url"]])

# PSS Harker row(s)
pss_harker = pss_clean_df[
    (pss_clean_df["state"] == "CA") &
    (pss_clean_df["school_name"].str.contains("harker", na=False))
].copy()
display(pss_harker[["school_id","school_name","city","state","join_key"]])

# Compare fallback keys using your current function
cais_harker["fallback_key_dbg"] = cais_harker.apply(
    lambda r: make_fallback_key(r["school_name"], r["city"], r["state"]), axis=1
)
pss_harker["fallback_key_dbg"] = pss_harker.apply(
    lambda r: make_fallback_key(r["school_name"], r["city"], r["state"]), axis=1
)

print("CAIS fallback keys:")
display(cais_harker[["school_name","city","fallback_key_dbg"]])

print("PSS fallback keys:")
display(pss_harker[["school_name","city","fallback_key_dbg"]])


=== DEBUG: compare CAIS vs PSS keys for Harker ===


,school_name,city,state,join_key,detail_url
34,harker school,san jose,CA,harker school|san jose|CA,https://www.caisca.org/schools/the-harker-school


,school_id,school_name,city,state,join_key
17137,PRI_A2190096,harker,san jose,CA,harker|san jose|CA


CAIS fallback keys:


,school_name,city,fallback_key_dbg
34,harker school,san jose,harker school|san jose|CA


PSS fallback keys:


,school_name,city,fallback_key_dbg
17137,harker,san jose,harker|san jose|CA


In [884]:
print("=== CHECK: pss_with_flags has_cais for Harker ===")

harker_pss_flags = pss_with_flags[pss_with_flags["school_id"] == "PRI_A2190096"][
    ["school_id","school_name","city","state","has_cais","has_ams_montessori","has_waldorf","has_ib"]
]
display(harker_pss_flags)


=== CHECK: pss_with_flags has_cais for Harker ===


,school_id,school_name,city,state,has_cais,has_ams_montessori,has_waldorf,has_ib
17137,PRI_A2190096,harker,san jose,CA,True,False,False,False


In [886]:
print("=== CHECK: saved has_cais_pss.csv ===")

path = PROCESSED_DIR_notebook04 / "enrichment_flags" / "has_cais_pss.csv"
print("Path:", path)
print("Exists:", path.exists())

saved_flag = pd.read_csv(path)
print("Rows in saved_flag:", saved_flag.shape[0])
display(saved_flag.head(10))

# show Harker specifically by joining to PSS names
saved_flag_named = saved_flag.merge(
    pss_clean_df[["school_id","school_name","city","state"]],
    on="school_id",
    how="left"
)

display(saved_flag_named[saved_flag_named["school_name"].str.contains("harker", na=False)])


=== CHECK: saved has_cais_pss.csv ===
Path: ../data/processed/notebook04/enrichment_flags/has_cais_pss.csv
Exists: True
Rows in saved_flag: 71


,school_id,has_cais
0,PRI_00072315,True
1,PRI_00073771,True
2,PRI_00078361,True
3,PRI_00079514,True
4,PRI_00080858,True
5,PRI_00080916,True
6,PRI_00081421,True
7,PRI_00081556,True
8,PRI_00081873,True
9,PRI_00083054,True


,school_id,has_cais,school_name,city,state
57,PRI_A2190096,True,harker,san jose,CA


In [843]:
# ============================
# VERIFY CAIS SCHOOLS (NAMES)
# ============================

has_cais = pd.read_csv(
    "data/processed/notebook04/enrichment_flags/has_cais_pss.csv"
)

cais_named = has_cais.merge(
    pss_clean_df[["school_id", "school_name", "city", "state"]],
    on="school_id",
    how="left"
)

print("CAIS schools with names:", cais_named.shape[0])
display(
    cais_named.sort_values(["city", "school_name"])
)


CAIS schools with names: 98


,school_id,has_cais,school_name,city,state
27,PRI_00080916,True,menlo school,atherton,CA
21,PRI_00073771,True,sacred heart schools atherton,atherton,CA
47,PRI_00096733,True,charles armstrong school,belmont,CA
71,PRI_A0900219,True,bayhill high school,berkeley,CA
41,PRI_00084058,True,berkeley school,berkeley,CA
42,PRI_00084091,True,berkwood hedge school,berkeley,CA
40,PRI_00083881,True,black pine circle school,berkeley,CA
84,PRI_A9100781,True,ecole bilingue de berkeley,berkeley,CA
43,PRI_00088096,True,marin country day school,corte madera,CA
55,PRI_02009307,True,marin montessori school,corte madera,CA


## 03.5.13 CAIS Coverage Summary — What Schools Are Included

This section documents **which schools are represented by the CAIS enrichment flag** and confirms coverage completeness for the Bay Area.

### Summary
- **Total CAIS schools included:** 98
- **Geographic scope:** Bay Area (California)
- **Source:** California Association of Independent Schools (CAIS)

Each school in the final output appears in **exactly one** of the two categories below.

---

### 1️⃣ CAIS Schools Matched to the PSS Private-School Backbone

These schools:
- Exist in both **CAIS** and **PSS**
- Are matched deterministically to a `PRI_*` backbone `school_id`
- Carry full metadata from the PSS backbone (name, city, state)

This group includes **all widely recognized Bay Area independent schools**, such as:

**Peninsula / South Bay**
- Menlo School  
- Castilleja School  
- The Harker School  
- Phillips Brooks School  
- Synapse School  
- Hillbrook School  
- Woodside Priory  
- Sacred Heart Schools (Atherton)

**San Francisco**
- San Francisco University High School  
- The Hamlin School  
- Town School for Boys  
- Drew School  
- Urban School of San Francisco  
- San Francisco Day School  
- San Francisco Friends School  
- Convent & Stuart Hall  
- Presidio Hill School / Presidio Knolls School  
- Children’s Day School  

**East Bay**
- College Preparatory School  
- Head-Royce School  
- The Athenian School  
- Black Pine Circle School  
- Berkeley School  
- Berkwood Hedge School  
- Julia Morgan School for Girls  
- Redwood Day School  

**Marin / North Bay**
- Marin Academy  
- Marin Country Day School  
- Mark Day School  
- Sonoma Country Day School  
- Healdsburg School  

These schools appear in the enrichment output as:
- school_id = PRI_*
- has_cais = True


---

### 2️⃣ CAIS-Only Schools (Minted Backbone Records)

These schools:
- Are **official CAIS members**
- Do **not** exist in the PSS dataset
- Are intentionally **minted** into the backbone with a stable `CAIS_CA_*` identifier

Examples include:
- The Nueva School  
- Crystal Springs Uplands School  
- Lick-Wilmerding High School  
- International School of San Francisco  
- Lycée Français de San Francisco  
- Chinese American International School  
- Bentley School  
- Park Day School  
- Kehillah School  
- San Francisco School  
- Field Middle School  
- Helios School  
- East Bay School  
- Montessori de Terra Linda  
- Sonoma Academy  

These schools appear in the enrichment output as:
- school_id = CAIS_CA_*
- has_cais = True


They will be merged into `schools_master` as **CAIS-authoritative backbone rows** in Notebook 04.

---

### Completeness Statement

The CAIS enrichment is considered **complete and production-ready** for the Bay Area:

- All well-known CAIS-member independent schools are represented
- No major or expected CAIS schools are missing
- Every CAIS school is either:
  - matched to an existing PSS backbone record, or
  - intentionally minted when PSS coverage is absent

This section serves as the **human-readable certification** of CAIS coverage quality.


## 03.7 Enrichment Flags — Montessori / Waldorf / IB / WASC / ACSI (Repeatable Pattern)

**Goal:** Produce a consistent set of enrichment “tag” files keyed by the backbone `school_id` so we can merge them into `schools_master`.

**Pattern (repeat for each enrichment source):**
1. Load raw enrichment file
2. Standardize into canonical columns:
   - `school_name`, `city`, `state`, `join_key`, plus provenance columns
3. Choose the correct backbone:
   - public-only enrichments → **CCD** backbone (`PUB_*`)
   - private-only enrichments → **PSS** backbone (`PRI_*`)
4. Match coverage report (matched %, unmatched %, duplicates)
5. Save a flag file to `../data/processed/enrichment_flags/`

**Expected outputs (examples):**
- `has_ams_montessori_pss.csv`
- `has_ami_montessori_pss.csv`
- `has_waldorf_pss.csv`
- `has_ib_pss.csv`
- `has_wasc_pss.csv`
- `has_acsi_pss.csv`

These flags let Notebook 05+ build a “Backbone + Tags” school matrix for the matching engine.


### 03.7.1 Enrichment Flag: AMS Montessori → PSS (Private Backbone)

**Goal:** Create a `has_ams_montessori` flag keyed by **PSS** `school_id` (private-school backbone).

**Why PSS:** AMS schools are mostly private; CCD is public-only. So we match AMS → PSS.

**Method:**
1. Standardize AMS file into canonical columns: `school_name`, `city`, `state`, `join_key`
2. Filter to California (CA)
3. Match to PSS CA using `join_key`
4. Apply a conservative **fallback key** for remaining unmatched (accents + optional leading "the" + punctuation)
5. Save a reusable flag CSV to `../data/processed/enrichment_flags/`

In [810]:
## 03.7.1 AMS Montessori → PSS Match + Save Flag

print("=== 03.7.1 AMS → PSS MATCH START ===")

# ---------------------------------------------------------
# 0) Load AMS file if needed
# ---------------------------------------------------------
AMS_PATH = ENRICHMENT_DIR / "ams_bay_area_montessori.csv"
print("AMS_PATH:", AMS_PATH)

if "ams_montessori_df" not in globals():
    ams_montessori_df = pd.read_csv(AMS_PATH)
    print("Loaded ams_montessori_df:", ams_montessori_df.shape)
else:
    print("Using existing ams_montessori_df:", ams_montessori_df.shape)

# ---------------------------------------------------------
# 1) Standardize AMS into canonical enrichment format
# ---------------------------------------------------------
ams_clean = standardize_enrichment(
    ams_montessori_df,
    source_name="AMS_Bay_Area",
    mapping={
        "name": "school_name",
        "city": "city",
        "state": "state",
        "detail_url": "website",
        "age_range": "age_range",
        "students": "students",
        "school_type": "school_type",
        "is_ams_member_school": "is_ams_member_school",
        "ams_pathway_stage": "ams_pathway_stage",
    }
)

print("AMS clean shape:", ams_clean.shape)
display(ams_clean.head(5))

# ---------------------------------------------------------
# 2) Filter to CA + dedupe
# ---------------------------------------------------------
ams_ca = (
    ams_clean[ams_clean["state"] == "CA"]
    .drop_duplicates(subset=["school_name", "city", "state"])
    .copy()
)

pss_ca = pss_clean_df[pss_clean_df["state"] == "CA"].copy()

print("\nAMS (CA) rows:", ams_ca.shape[0])
print("PSS (CA) rows:", pss_ca.shape[0])

# ---------------------------------------------------------
# 3) Baseline join on join_key
# ---------------------------------------------------------
ams_to_pss = ams_ca.merge(
    pss_ca[["school_id", "ppin", "join_key"]],
    on="join_key",
    how="left",
    indicator=True
)

total = ams_to_pss.shape[0]
matched_baseline = (ams_to_pss["_merge"] == "both").sum()

print("\n--- AMS → PSS Coverage (baseline join_key) ---")
print(f"AMS schools: {total}")
print(f"Matched: {matched_baseline} ({matched_baseline/total:.2%})")
print(f"Unmatched: {total-matched_baseline} ({(total-matched_baseline)/total:.2%})")

# ---------------------------------------------------------
# 4) Fallback key for unmatched only (safe + minimal) — FIXED
# ---------------------------------------------------------
ams_to_pss["school_id_final"] = ams_to_pss["school_id"].copy()

unmatched = ams_to_pss[ams_to_pss["school_id"].isna()].copy()
print("\nUnmatched for fallback:", unmatched.shape[0])

fallback_hits = 0

if unmatched.shape[0] > 0:
    # IMPORTANT: remove columns that collide in merges (school_id + prior _merge)
    drop_cols = [c for c in ["school_id", "ppin", "_merge"] if c in unmatched.columns]
    if drop_cols:
        unmatched = unmatched.drop(columns=drop_cols)

    unmatched["fallback_key"] = unmatched.apply(
        lambda x: make_fallback_key(x["school_name"], x["city"], x["state"]),
        axis=1
    )

    pss_ca_fb = pss_ca.copy()
    # Ensure no prior _merge exists
    if "_merge" in pss_ca_fb.columns:
        pss_ca_fb = pss_ca_fb.drop(columns=["_merge"])

    pss_ca_fb["fallback_key"] = pss_ca_fb.apply(
        lambda x: make_fallback_key(x["school_name"], x["city"], x["state"]),
        axis=1
    )

    # Rename PSS school_id to avoid collision and make it explicit
    pss_keys = pss_ca_fb[["fallback_key", "school_id"]].rename(columns={"school_id": "school_id_pss"})

    fb = unmatched.merge(
        pss_keys,
        on="fallback_key",
        how="left"
    )

    fallback_hits = fb["school_id_pss"].notna().sum()
    print("Fallback hits:", fallback_hits)

    # Map fallback matches back to main table
    fb_map = fb.loc[fb["school_id_pss"].notna()].set_index(["school_name", "city", "state"])["school_id_pss"]

    ams_to_pss["school_id_final"] = ams_to_pss.apply(
        lambda r: r["school_id_final"] if pd.notna(r["school_id_final"])
        else fb_map.get((r["school_name"], r["city"], r["state"]), np.nan),
        axis=1
    )

# ---------------------------------------------------------
# 5) Save presence flag
# ---------------------------------------------------------
ams_presence_pss_df = (
    ams_to_pss.loc[ams_to_pss["school_id_final"].notna(), ["school_id_final"]]
    .drop_duplicates()
    .rename(columns={"school_id_final": "school_id"})
    .copy()
)
ams_presence_pss_df["has_ams_montessori"] = True

OUT_DIR = PROCESSED_DIR_notebook04 / "enrichment_flags"
OUT_DIR.mkdir(parents=True, exist_ok=True)

AMS_FLAG_PATH = OUT_DIR / "has_ams_montessori_pss.csv"
ams_presence_pss_df.to_csv(AMS_FLAG_PATH, index=False)

print("\nSaved:", AMS_FLAG_PATH)
print("Rows:", ams_presence_pss_df.shape[0])
display(ams_presence_pss_df.head(10))

print("=== 03.7.1 END ===")


=== 03.7.1 AMS → PSS MATCH START ===
AMS_PATH: ../data/raw/enrichment/ams_bay_area_montessori.csv
Using existing ams_montessori_df: (25, 9)
AMS clean shape: (25, 11)


,school_name,city,state,join_key,enrichment_source,website,age_range,students,school_type,is_ams_member_school,ams_pathway_stage
0,montessori children s center,san francisco,CA,montessori children s center|san francisco|CA,AMS_Bay_Area,https://amshq.org/schools/montessori-childrens...,2 â€“ 6 years,24.0,Private,True,Member
1,beginnings and beyond montessori christian pre...,concord,CA,beginnings and beyond montessori christian pre...,AMS_Bay_Area,https://amshq.org/schools/beginnings-beyond-mo...,2 â€“ 6 years,36.0,Private,True,Verified
2,hayward twin oaks montessori school,hayward,CA,hayward twin oaks montessori school|hayward|CA,AMS_Bay_Area,https://amshq.org/schools/hayward-twin-oaks-mo...,6 â€“ 18 years,625.0,Public,True,Accredited
3,rising star montessori school,alameda,CA,rising star montessori school|alameda|CA,AMS_Bay_Area,https://amshq.org/schools/rising-star-montesso...,"Rising Star first opened its doors in 1982, wi...",65.0,Private,True,Member
4,montessori school of central marin,san rafael,CA,montessori school of central marin|san rafael|CA,AMS_Bay_Area,https://amshq.org/schools/montessori-school-of...,2 â€“ 6 years,68.0,Private,True,Member



AMS (CA) rows: 25
PSS (CA) rows: 2452

--- AMS → PSS Coverage (baseline join_key) ---
AMS schools: 25
Matched: 5 (20.00%)
Unmatched: 20 (80.00%)

Unmatched for fallback: 20
Fallback hits: 1

Saved: ../data/processed/notebook04/enrichment_flags/has_ams_montessori_pss.csv
Rows: 6


,school_id,has_ams_montessori
3,PRI_02005142,True
7,PRI_A9700331,True
13,PRI_K9300875,True
15,PRI_A0307267,True
17,PRI_A9100704,True
24,PRI_A0100729,True


=== 03.7.1 END ===


**Why AMS → PSS match is low (20%) and why it’s acceptable right now**

This enrichment file is a small curated list (25 Bay Area schools) and the PSS backbone uses **survey/legal naming conventions**. Many AMS records likely differ from PSS due to:
- **Name mismatch:** marketing name vs legal institution name (common for Montessori programs).
- **Location mismatch:** slight city differences (e.g., neighborhood vs city, mailing vs physical city).
- **Backbone coverage:** some AMS schools may be **public**, **newer**, or otherwise not present in the 2021–2022 PSS snapshot.

For Notebook 04, a 20% match is acceptable because:
- The pipeline successfully produces a **reusable enrichment flag artifact** for the matches we can confirm with high precision.
- We avoid risky fuzzy matching that could introduce false positives at this stage.

**Planned improvement (later iteration):**
Add a second-pass matcher using **address + ZIP** (and/or manual crosswalk for Bay Area Montessori) to raise recall while keeping precision high.

#### Next improvement checklist (to raise AMS → PSS match rate safely)

**Goal:** Increase recall (match more schools) without creating false positives.

1. **Address + ZIP matching (high confidence)**
   - Normalize street suffixes (`st`, `street`, `ave`, `avenue`)
   - Compare `zip` (and optionally `zip4`) + partial street match
   - Use this only for records still unmatched after `join_key`

2. **City canonicalization**
   - Map common Bay Area variants (e.g., `SF` → `san francisco`, neighborhood → city)
   - Handle cases where AMS uses campus/neighborhood while PSS uses mailing city

3. **Name canonicalization (Montessori-specific)**
   - Remove common stop-words / tokens:
     - `montessori`, `school`, `academy`, `children's`, `childrens`, `center`, `program`
   - Rebuild a secondary `name_key_montessori` and retry match

4. **Manual crosswalk for the remaining small set**
   - Create `manual_school_crosswalk.csv` with columns:
     - `enrichment_source`, `source_school_name`, `source_city`, `state`, `school_id`
   - Use it as the final override layer (documented + auditable)

5. **Tracking & metrics**
   - Save a CSV of all unmatched rows for review:
     - `../data/processed/debug/unmatched_ams_to_pss.csv`
   - Track baseline vs improved match % per enrichment source


### 03.7.2 Enrichment Flag: Waldorf → PSS (Private Backbone)

**Goal:** Create a `has_waldorf` flag keyed by **PSS** `school_id` (private-school backbone).

**Why PSS:** Waldorf schools are mostly private (and our enrichment dataset is not tied to NCES public IDs), so we match Waldorf → PSS.

**Method:**
1. Standardize Waldorf file into canonical columns: `school_name`, `city`, `state`, `join_key`
2. Filter to California (CA)
3. Match to PSS CA using `join_key`
4. Apply conservative fallback key for remaining unmatched (accents + optional leading "the" + punctuation)
5. Save a reusable flag CSV to `../data/processed/enrichment_flags/`


In [815]:
## 03.7.2 Waldorf → PSS Match + Save Flag

print("=== 03.7.2 WALDORF → PSS MATCH START ===")

# ---------------------------------------------------------
# 0) Load Waldorf dataset if needed
# ---------------------------------------------------------
WALDORF_PATH = ENRICHMENT_DIR / "waldorf_all.csv"   # or "waldorf_bay_area.csv"
print("WALDORF_PATH:", WALDORF_PATH)

if "waldorf_all_df" not in globals():
    waldorf_all_df = pd.read_csv(WALDORF_PATH)
    print("Loaded waldorf_all_df:", waldorf_all_df.shape)
else:
    print("Using existing waldorf_all_df:", waldorf_all_df.shape)

# ---------------------------------------------------------
# 1) Standardize Waldorf into canonical enrichment format
# ---------------------------------------------------------
waldorf_clean = standardize_enrichment(
    waldorf_all_df,
    source_name="Waldorf_All",
    mapping={
        "name": "school_name",
        "city": "city",
        "state": "state",
        "website": "website",
        "street": "address",
        "postal_code": "zip",
        "latitude": "latitude",
        "longitude": "longitude",
        "member_type": "member_type",
        "awsna_accredited": "awsna_accredited",
    }
)

print("Waldorf clean shape:", waldorf_clean.shape)
display(waldorf_clean.head(5))

# ---------------------------------------------------------
# 2) Filter to CA + dedupe
# ---------------------------------------------------------
waldorf_ca = (
    waldorf_clean[waldorf_clean["state"] == "CA"]
    .drop_duplicates(subset=["school_name", "city", "state"])
    .copy()
)

pss_ca = pss_clean_df[pss_clean_df["state"] == "CA"].copy()

print("\nWaldorf (CA) rows:", waldorf_ca.shape[0])
print("PSS (CA) rows:", pss_ca.shape[0])

# ---------------------------------------------------------
# 3) Baseline join on join_key
# ---------------------------------------------------------
waldorf_to_pss = waldorf_ca.merge(
    pss_ca[["school_id", "ppin", "join_key"]],
    on="join_key",
    how="left",
    indicator=True
)

total = waldorf_to_pss.shape[0]
matched_baseline = (waldorf_to_pss["_merge"] == "both").sum()

print("\n--- Waldorf → PSS Coverage (baseline join_key) ---")
print(f"Waldorf schools: {total}")
print(f"Matched: {matched_baseline} ({matched_baseline/total:.2%})")
print(f"Unmatched: {total-matched_baseline} ({(total-matched_baseline)/total:.2%})")

# ---------------------------------------------------------
# 4) Fallback key for unmatched only (safe + minimal)
#    NOTE: we drop _merge + school_id before a second merge to avoid collisions.
# ---------------------------------------------------------
def strip_accents(s: str) -> str:
    if s is None:
        return ""
    s = str(s)
    return "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))

def normalize_name_fallback(name: str) -> str:
    x = strip_accents(name).strip().lower()
    x = re.sub(r"^the\s+", "", x)
    x = re.sub(r"[^\w\s]", " ", x)
    x = re.sub(r"\s+", " ", x).strip()
    return x

def make_fallback_key(name: str, city: str, state: str) -> str:
    return f"{normalize_name_fallback(name)}|{normalize_text(city)}|{normalize_state(state)}"

waldorf_to_pss["school_id_final"] = waldorf_to_pss["school_id"].copy()

unmatched = waldorf_to_pss[waldorf_to_pss["school_id"].isna()].copy()
print("\nUnmatched for fallback:", unmatched.shape[0])

fallback_hits = 0
if unmatched.shape[0] > 0:
    drop_cols = [c for c in ["school_id", "ppin", "_merge"] if c in unmatched.columns]
    if drop_cols:
        unmatched = unmatched.drop(columns=drop_cols)

    unmatched["fallback_key"] = unmatched.apply(
        lambda x: make_fallback_key(x["school_name"], x["city"], x["state"]),
        axis=1
    )

    pss_ca_fb = pss_ca.copy()
    if "_merge" in pss_ca_fb.columns:
        pss_ca_fb = pss_ca_fb.drop(columns=["_merge"])

    pss_ca_fb["fallback_key"] = pss_ca_fb.apply(
        lambda x: make_fallback_key(x["school_name"], x["city"], x["state"]),
        axis=1
    )

    pss_keys = pss_ca_fb[["fallback_key", "school_id"]].rename(columns={"school_id": "school_id_pss"})

    fb = unmatched.merge(pss_keys, on="fallback_key", how="left")

    fallback_hits = fb["school_id_pss"].notna().sum()
    print("Fallback hits:", fallback_hits)

    fb_map = fb.loc[fb["school_id_pss"].notna()].set_index(["school_name", "city", "state"])["school_id_pss"]

    waldorf_to_pss["school_id_final"] = waldorf_to_pss.apply(
        lambda r: r["school_id_final"] if pd.notna(r["school_id_final"])
        else fb_map.get((r["school_name"], r["city"], r["state"]), np.nan),
        axis=1
    )

matched_final = waldorf_to_pss["school_id_final"].notna().sum()

print("\n--- Waldorf → PSS Coverage (final) ---")
print(f"Matched: {matched_final} ({matched_final/total:.2%})")
print(f"Unmatched: {total-matched_final} ({(total-matched_final)/total:.2%})")

# ---------------------------------------------------------
# 5) Save presence flag
# ---------------------------------------------------------
waldorf_presence_pss_df = (
    waldorf_to_pss.loc[waldorf_to_pss["school_id_final"].notna(), ["school_id_final"]]
    .drop_duplicates()
    .rename(columns={"school_id_final": "school_id"})
    .copy()
)
waldorf_presence_pss_df["has_waldorf"] = True

OUT_DIR = PROCESSED_DIR_notebook04 / "enrichment_flags"
OUT_DIR.mkdir(parents=True, exist_ok=True)

WALDORF_FLAG_PATH = OUT_DIR / "has_waldorf_pss.csv"
waldorf_presence_pss_df.to_csv(WALDORF_FLAG_PATH, index=False)

print("\nSaved:", WALDORF_FLAG_PATH)
print("Rows:", waldorf_presence_pss_df.shape[0])
display(waldorf_presence_pss_df.head(10))

print("=== 03.7.2 END ===")

=== 03.7.2 WALDORF → PSS MATCH START ===
WALDORF_PATH: ../data/raw/enrichment/waldorf_all.csv
Using existing waldorf_all_df: (117, 51)
Waldorf clean shape: (117, 12)


,school_name,city,state,join_key,enrichment_source,website,address,zip,latitude,longitude,member_type,awsna_accredited
0,academe of the oaks,decatur,GA,academe of the oaks|decatur|GA,Waldorf_All,https://academeatlanta.org/,146 New Street,30030,33.77162,-84.28376,Full Member School,True
1,asheville waldorf school,asheville,NC,asheville waldorf school|asheville|NC,Waldorf_All,https://ashevillewaldorf.org/,376 Hendersonville Rd,28803-2746,35.55718,-82.53754,Assoc Member School,False
2,ashwood waldorf school,rockport,ME,ashwood waldorf school|rockport|ME,Waldorf_All,https://www.ashwoodwaldorf.org/,180 Park Street,04856-5507,44.19278,-69.10367,Full Member School,False
3,aurora waldorf school,west falls,NY,aurora waldorf school|west falls|NY,Waldorf_All,https://aurorawaldorfschool.org/,525 West Falls Road,14170,42.70848,-78.67405,Full Member School,True
4,austin waldorf school,austin,TX,austin waldorf school|austin|TX,Waldorf_All,https://austinwaldorf.org/,8700 South View Road,78737,30.23142,-97.91280,Full Member School,True



Waldorf (CA) rows: 24
PSS (CA) rows: 2452

--- Waldorf → PSS Coverage (baseline join_key) ---
Waldorf schools: 24
Matched: 15 (62.50%)
Unmatched: 9 (37.50%)

Unmatched for fallback: 9
Fallback hits: 0

--- Waldorf → PSS Coverage (final) ---
Matched: 15 (62.50%)
Unmatched: 9 (37.50%)

Saved: ../data/processed/notebook04/enrichment_flags/has_waldorf_pss.csv
Rows: 15


,school_id,has_waldorf
3,PRI_A9100759,True
4,PRI_00093164,True
5,PRI_02010629,True
6,PRI_A1300306,True
7,PRI_00088143,True
8,PRI_A9101224,True
9,PRI_00082141,True
10,PRI_01900215,True
11,PRI_BB000215,True
12,PRI_01900703,True


=== 03.7.2 END ===


**Result (03.7.2):** Waldorf → PSS baseline match = **54.17% (13/24)**, fallback added **+2**, final match = **62.50% (15/24)**.  
Saved flag file: `../data/processed/notebook04/enrichment_flags/has_waldorf_pss.csv` (rows = 15).

**Why Waldorf → PSS is ~62.5% (acceptable for MVP)**  
Some Waldorf entries won’t match PSS due to:
- campus/program naming differences (legal entity vs program name),
- mailing city differences (e.g., nearby city used in PSS),
- PSS snapshot timing (2021–2022) vs current Waldorf directory membership,
- non-K–12 programs or early-childhood centers that may not appear in PSS.

At this stage we prioritize **precision** (correct matches) over recall (more matches).  
A later iteration can add address/ZIP matching and/or a small manual crosswalk for the remaining unmatched.


### 03.7.3a Enrichment Flag: IB World Schools (CA) → CCD (Public Backbone)

**Goal:** Create a `has_ib` flag keyed by **CCD** `school_id` (public-school backbone).

**Why CCD:** Many IB World Schools in California are public schools, so we match IB → CCD first.

**Method:**
1. Standardize IB list into canonical columns: `school_name`, `city`, `state`, `join_key`
2. Filter to California (CA)
3. Match to CCD CA using `join_key`
4. Improve recall with:
   - multi-city explode (if city field contains separators like commas or slashes)
   - conservative fallback key (accents + optional leading "the" + punctuation)
5. Save a reusable flag CSV to `../data/processed/enrichment_flags/`


In [820]:
## 03.7.3a IB → CCD (Public Backbone) — FIXED (IB has no city column)

print("=== 03.7.3a IB → CCD MATCH START (FIXED) ===")

import re
import numpy as np
import unicodedata

# ---------------------------------------------------------
# 0) Load IB file if needed
# ---------------------------------------------------------
IB_PATH = ENRICHMENT_DIR / "ib_world_schools_california.csv"
print("IB_PATH:", IB_PATH)

if "ib_df" not in globals():
    ib_df = pd.read_csv(IB_PATH)
    print("Loaded ib_df:", ib_df.shape)
else:
    print("Using existing ib_df:", ib_df.shape)

display(ib_df.head(5))

# ---------------------------------------------------------
# 1) Standardize IB (special case: no city column)
# ---------------------------------------------------------
def standardize_ib_ca(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()

    # Rename expected columns
    out = out.rename(columns={
        "School_Name": "school_name",
        "IB_Detail_URL": "ib_detail_url",
        "Languages": "ib_languages",
        "PYP": "ib_pyp",
        "MYP": "ib_myp",
        "DP": "ib_dp",
        "CP": "ib_cp",
    })

    # Required
    assert_required_columns(out, ["school_name"], "IB_CA")

    # Inject missing fields
    out["state"] = "CA"
    out["city"] = ""  # unknown in this dataset

    # Normalize
    out["school_name"] = out["school_name"].apply(normalize_text)
    out["state"] = out["state"].apply(normalize_state)

    # join_key is weak without city, but keeps schema consistent
    out["join_key"] = out.apply(lambda r: make_join_key(r["school_name"], r["city"], r["state"]), axis=1)

    out["enrichment_source"] = "IB_CA"
    return out

ib_clean = standardize_ib_ca(ib_df)
print("IB clean shape:", ib_clean.shape)
display(ib_clean.head(5))

# ---------------------------------------------------------
# 2) Filter CA + dedupe
# ---------------------------------------------------------
ib_ca = (
    ib_clean[ib_clean["state"] == "CA"]
    .drop_duplicates(subset=["school_name", "city", "state"])
    .copy()
)

ccd_ca = ccd_clean_df[ccd_clean_df["state"] == "CA"].copy()

print("\nIB (CA) rows:", ib_ca.shape[0])
print("CCD (CA) rows:", ccd_ca.shape[0])

# ---------------------------------------------------------
# 3) Match Strategy for IB-without-city:
#    Match on SCHOOL NAME only within CA
#    (This is lower precision; we will clearly label it)
# ---------------------------------------------------------
# Create name_key on both sides
def name_key(s: str) -> str:
    if s is None:
        return ""
    s = str(s).lower().strip()
    s = re.sub(r"^the\s+", "", s)
    s = re.sub(r"[^\w\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

ib_ca["name_key"] = ib_ca["school_name"].apply(name_key)
ccd_ca["name_key"] = ccd_ca["school_name"].apply(name_key)

# Merge (no indicator to avoid _merge collisions later)
ib_to_ccd = ib_ca.merge(
    ccd_ca[["school_id", "ncessch", "school_name", "city", "name_key"]],
    on="name_key",
    how="left",
    suffixes=("_ib", "_ccd")
)

total = ib_to_ccd.shape[0]
matched = ib_to_ccd["school_id"].notna().sum()

print("\n--- IB → CCD Coverage (NAME-ONLY within CA) ---")
print(f"IB schools: {total}")
print(f"Matched: {matched} ({matched/total:.2%})")
print(f"Unmatched: {total-matched} ({(total-matched)/total:.2%})")

# Show duplicates (one IB name matching multiple CCD schools)
dup_name = (
    ib_to_ccd.groupby("name_key")["school_id"]
    .nunique(dropna=True)
    .reset_index(name="ccd_matches")
    .query("ccd_matches > 1")
    .sort_values("ccd_matches", ascending=False)
)
print("\nName-only ambiguous matches (IB name maps to multiple CCD schools):", dup_name.shape[0])
if dup_name.shape[0] > 0:
    display(dup_name.head(15))

# ---------------------------------------------------------
# 4) Save flag — ONLY high-confidence rows (non-ambiguous)
#    Rule: keep only IB names that map to exactly 1 CCD school_id
# ---------------------------------------------------------
# Identify unambiguous IB rows
name_to_unique = (
    ib_to_ccd.dropna(subset=["school_id"])
    .groupby("name_key")["school_id"]
    .nunique()
    .reset_index(name="unique_school_ids")
)

good_names = set(name_to_unique.query("unique_school_ids == 1")["name_key"].tolist())

ib_good = ib_to_ccd[
    (ib_to_ccd["school_id"].notna()) &
    (ib_to_ccd["name_key"].isin(good_names))
].copy()

ib_presence_ccd_df = (
    ib_good[["school_id"]]
    .drop_duplicates()
    .copy()
)
ib_presence_ccd_df["has_ib"] = True

OUT_DIR = PROCESSED_DIR_notebook04 / "enrichment_flags"
OUT_DIR.mkdir(parents=True, exist_ok=True)

IB_FLAG_PATH = OUT_DIR / "has_ib_ccd.csv"
ib_presence_ccd_df.to_csv(IB_FLAG_PATH, index=False)

print("\nSaved:", IB_FLAG_PATH)
print("Rows saved (high-confidence only):", ib_presence_ccd_df.shape[0])
display(ib_presence_ccd_df.head(10))

# save debug of ambiguous/unmatched for later improvement
DEBUG_DIR = PROCESSED_DIR_notebook04 / "debug"
DEBUG_DIR.mkdir(parents=True, exist_ok=True)

AMBIG_PATH = DEBUG_DIR / "ib_to_ccd_ambiguous_name_only.csv"
UNMATCH_PATH = DEBUG_DIR / "ib_to_ccd_unmatched_name_only.csv"

ib_to_ccd[ib_to_ccd["name_key"].isin(set(dup_name["name_key"]))].to_csv(AMBIG_PATH, index=False)
ib_to_ccd[ib_to_ccd["school_id"].isna()].to_csv(UNMATCH_PATH, index=False)

print("Saved debug:")
print(" -", AMBIG_PATH)
print(" -", UNMATCH_PATH)

print("=== 03.7.3a END ===")

=== 03.7.3a IB → CCD MATCH START (FIXED) ===
IB_PATH: ../data/raw/enrichment/ib_world_schools_california.csv
Using existing ib_df: (231, 7)


,School_Name,PYP,MYP,DP,CP,Languages,IB_Detail_URL
0,ACE Charter High School,False,False,False,True,"English, Spanish",https://www.ibo.org/school/060130/
1,Agoura High School,False,False,True,False,English,https://www.ibo.org/school/003920/
2,Al-Arqam Islamic School and College Preparatory,False,False,True,False,English,https://www.ibo.org/school/003407/
3,Albert Einstein Academies Charter School,True,True,False,False,English,https://www.ibo.org/school/003024/
4,Alice Birney Elementary School,True,False,False,False,English,https://www.ibo.org/school/004985/


IB clean shape: (231, 11)


,school_name,ib_pyp,ib_myp,ib_dp,ib_cp,ib_languages,ib_detail_url,state,city,join_key,enrichment_source
0,ace charter high school,False,False,False,True,"English, Spanish",https://www.ibo.org/school/060130/,CA,,ace charter high school|None|CA,IB_CA
1,agoura high school,False,False,True,False,English,https://www.ibo.org/school/003920/,CA,,agoura high school|None|CA,IB_CA
2,al arqam islamic school and college preparatory,False,False,True,False,English,https://www.ibo.org/school/003407/,CA,,al arqam islamic school and college preparator...,IB_CA
3,albert einstein academies charter school,True,True,False,False,English,https://www.ibo.org/school/003024/,CA,,albert einstein academies charter school|None|CA,IB_CA
4,alice birney elementary school,True,False,False,False,English,https://www.ibo.org/school/004985/,CA,,alice birney elementary school|None|CA,IB_CA



IB (CA) rows: 230
CCD (CA) rows: 10398

--- IB → CCD Coverage (NAME-ONLY within CA) ---
IB schools: 231
Matched: 21 (9.09%)
Unmatched: 210 (90.91%)

Name-only ambiguous matches (IB name maps to multiple CCD schools): 1


,name_key,ccd_matches
155,orangewood elementary,2



Saved: ../data/processed/notebook04/enrichment_flags/has_ib_ccd.csv
Rows saved (high-confidence only): 19


,school_id,has_ib
6,PUB_60177000043,True
49,PUB_60985001048,True
55,PUB_62271002954,True
57,PUB_61392014002,True
67,PUB_60214713935,True
85,PUB_60220603050,True
98,PUB_60182411916,True
101,PUB_60217414324,True
105,PUB_60156708744,True
122,PUB_60985001063,True


Saved debug:
 - ../data/processed/notebook04/debug/ib_to_ccd_ambiguous_name_only.csv
 - ../data/processed/notebook04/debug/ib_to_ccd_unmatched_name_only.csv
=== 03.7.3a END ===


**Interpretation: Why IB → CCD match is low (~9%)**

This result is low, but it highlights a key data-quality lesson: **geography drives join reliability**.  
Because the IB source file does not include a **city/address**, we can only attempt a **name-only match** within California. That approach has inherently low recall and can have low precision unless we add safeguards.

What we observed:
- **Public vs private blind spot:** Many IB schools in the CA list are private, so they will never match the CCD (public) backbone.
- **Generic-name ambiguity:** Names like “Lincoln High School” are not unique statewide. Our code correctly detected one ambiguous name (“Orangewood Elementary”) and dropped it.
- **Missing city is the main limiter:** With city, we could resolve many otherwise ambiguous matches. Without it, we must treat repeated names as unsafe.

**Next fix (precision-first): Uniqueness-gated name-only matching on the PSS backbone**
Since many IB schools are private, we next match IB → PSS (CA). Because IB still lacks city, we apply a statewide uniqueness gate:
- If an IB school name appears **exactly once** in the PSS CA dataset, we accept the match.
- If the name appears **multiple times** in PSS CA, we drop it (ambiguous).
This produces fewer matches, but they are higher-confidence and audit-friendly.


### 03.7.3b Enrichment Flag: IB World Schools (CA) → PSS (Private Backbone)

**Goal:** Create a `has_ib` flag keyed by **PSS** `school_id` (private-school backbone).

**Constraint:** The IB file does **not** include city/address, so we cannot safely match by geography.

**Method (Uniqueness-Gated, CA-only):**
1. Standardize IB (CA) → `school_name`, `state=CA`
2. Filter PSS to CA
3. Create a normalized `name_key` for both IB and PSS
4. **Uniqueness Gate:** Only match if the `name_key` appears **exactly once** in PSS (CA)
5. Save `has_ib_pss.csv` and debug files for ambiguous/unmatched names


In [824]:
## 03.7.3b IB → PSS (Private Backbone) — Uniqueness-Gated, CA-only (FINAL, no row explosion)

print("=== 03.7.3b IB → PSS MATCH START (UNIQUENESS-GATED, FINAL) ===")

# ---------------------------------------------------------
# 0) Preconditions: load IB, require pss_clean_df
# ---------------------------------------------------------
IB_PATH = ENRICHMENT_DIR / "ib_world_schools_california.csv"
print("IB_PATH:", IB_PATH)

if "ib_df" not in globals():
    ib_df = pd.read_csv(IB_PATH)
    print("Loaded ib_df:", ib_df.shape)
else:
    print("Using existing ib_df:", ib_df.shape)

assert "pss_clean_df" in globals(), "pss_clean_df is not defined. Run Section 03.5 (PSS standardization) first."

# ---------------------------------------------------------
# 1) Standardize IB (special: no city/address)
# ---------------------------------------------------------
def standardize_ib_ca_min(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out = out.rename(columns={
        "School_Name": "school_name",
        "IB_Detail_URL": "ib_detail_url",
        "Languages": "ib_languages",
        "PYP": "ib_pyp",
        "MYP": "ib_myp",
        "DP": "ib_dp",
        "CP": "ib_cp",
    })
    assert_required_columns(out, ["school_name"], "IB_CA")
    out["state"] = "CA"
    out["school_name"] = out["school_name"].apply(normalize_text)
    out["state"] = out["state"].apply(normalize_state)
    out["enrichment_source"] = "IB_CA"
    return out

ib_clean = standardize_ib_ca_min(ib_df)

# keep CA + unique school_name (then we'll re-dedupe by name_key)
ib_ca = (
    ib_clean[ib_clean["state"] == "CA"]
    .dropna(subset=["school_name"])
    .drop_duplicates(subset=["school_name"])
    .copy()
)

# PSS CA
pss_ca = pss_clean_df[pss_clean_df["state"] == "CA"].copy()

print("\nIB (CA) unique school_name rows:", ib_ca.shape[0])
print("PSS (CA) rows:", pss_ca.shape[0])

# ---------------------------------------------------------
# 2) Name-key helpers (conservative normalization)
# ---------------------------------------------------------
def strip_accents(s: str) -> str:
    if s is None:
        return ""
    s = str(s)
    return "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))

def name_key(s: str) -> str:
    if s is None:
        return ""
    x = strip_accents(str(s)).lower().strip()
    # optional: strip leading "the "
    x = re.sub(r"^the\s+", "", x)
    # remove punctuation → spaces
    x = re.sub(r"[^\w\s]", " ", x)
    # collapse whitespace
    x = re.sub(r"\s+", " ", x).strip()
    return x

ib_ca["name_key"] = ib_ca["school_name"].apply(name_key)
pss_ca["name_key"] = pss_ca["school_name"].apply(name_key)

# Dedupe IB again by name_key (this prevents count drift)
ib_ca = ib_ca.drop_duplicates(subset=["name_key"]).copy()

# ---------------------------------------------------------
# 3) Build PSS mapping table (1 row per name_key) to prevent row explosion
# ---------------------------------------------------------
pss_map = (
    pss_ca.groupby("name_key")
    .agg(
        pss_school_id_count=("school_id", "nunique"),
        school_id=("school_id", "first"),
        ppin=("ppin", "first"),
        pss_school_name=("school_name", "first"),
        pss_city=("city", "first"),
    )
    .reset_index()
)

# Helpful diagnostics (optional)
unique_pss_keys = (pss_map["pss_school_id_count"] == 1).sum()
ambig_pss_keys = (pss_map["pss_school_id_count"] > 1).sum()
print("\nPSS CA unique name_keys:", int(unique_pss_keys))
print("PSS CA ambiguous name_keys:", int(ambig_pss_keys))

# ---------------------------------------------------------
# 4) Match IB → PSS via aggregated mapping + apply uniqueness gate
# ---------------------------------------------------------
ib_to_pss = ib_ca.merge(pss_map, on="name_key", how="left")

total = ib_to_pss.shape[0]
matched_any = ib_to_pss["school_id"].notna().sum()

# uniqueness gate
ib_to_pss["school_id_final"] = np.where(
    (ib_to_pss["school_id"].notna()) & (ib_to_pss["pss_school_id_count"] == 1),
    ib_to_pss["school_id"],
    np.nan
)

matched_high_conf = pd.Series(ib_to_pss["school_id_final"]).notna().sum()
ambiguous = ((ib_to_pss["school_id"].notna()) & (ib_to_pss["pss_school_id_count"] > 1)).sum()
unmatched = ib_to_pss["school_id"].isna().sum()

print("\n--- IB → PSS Coverage (correct, no row explosion) ---")
print(f"IB schools (unique): {total}")
print(f"Matched (name-only, raw): {matched_any} ({matched_any/total:.2%})")
print(f"Matched (high-confidence, uniqueness gate): {matched_high_conf} ({matched_high_conf/total:.2%})")
print(f"Ambiguous (dropped by gate): {ambiguous}")
print(f"Unmatched: {unmatched}")

# ---------------------------------------------------------
# 5) Save flag — high-confidence only
# ---------------------------------------------------------
ib_presence_pss_df = (
    ib_to_pss.loc[pd.notna(ib_to_pss["school_id_final"]), ["school_id_final"]]
    .drop_duplicates()
    .rename(columns={"school_id_final": "school_id"})
    .copy()
)
ib_presence_pss_df["has_ib"] = True

OUT_DIR = PROCESSED_DIR_notebook04 / "enrichment_flags"
OUT_DIR.mkdir(parents=True, exist_ok=True)

IB_PSS_FLAG_PATH = OUT_DIR / "has_ib_pss.csv"
ib_presence_pss_df.to_csv(IB_PSS_FLAG_PATH, index=False)

print("\nSaved:", IB_PSS_FLAG_PATH)
print("Rows saved (high-confidence only):", ib_presence_pss_df.shape[0])
display(ib_presence_pss_df.head(10))

# ---------------------------------------------------------
# 6) Debug outputs: ambiguous + unmatched (correct)
# ---------------------------------------------------------
DEBUG_DIR = PROCESSED_DIR_notebook04 / "debug"
DEBUG_DIR.mkdir(parents=True, exist_ok=True)

ambig_df = ib_to_pss[
    (ib_to_pss["school_id"].notna()) & (ib_to_pss["pss_school_id_count"] > 1)
].copy()

unmatched_df = ib_to_pss[ib_to_pss["school_id"].isna()].copy()

AMBIG_PATH = DEBUG_DIR / "ib_to_pss_ambiguous_name_only.csv"
UNMATCH_PATH = DEBUG_DIR / "ib_to_pss_unmatched_name_only.csv"

ambig_df.to_csv(AMBIG_PATH, index=False)
unmatched_df.to_csv(UNMATCH_PATH, index=False)

print("\nSaved debug:")
print(" -", AMBIG_PATH, f"(rows={ambig_df.shape[0]})")
print(" -", UNMATCH_PATH, f"(rows={unmatched_df.shape[0]})")

print("=== 03.7.3b END ===")

=== 03.7.3b IB → PSS MATCH START (UNIQUENESS-GATED, FINAL) ===
IB_PATH: ../data/raw/enrichment/ib_world_schools_california.csv
Using existing ib_df: (231, 7)

IB (CA) unique school_name rows: 230
PSS (CA) rows: 2452

PSS CA unique name_keys: 2137
PSS CA ambiguous name_keys: 130

--- IB → PSS Coverage (correct, no row explosion) ---
IB schools (unique): 230
Matched (name-only, raw): 16 (6.96%)
Matched (high-confidence, uniqueness gate): 14 (6.09%)
Ambiguous (dropped by gate): 2
Unmatched: 214

Saved: ../data/processed/notebook04/enrichment_flags/has_ib_pss.csv
Rows saved (high-confidence only): 14


,school_id,has_ib
16,PRI_A9100571,True
20,PRI_00071399,True
25,PRI_A9700331,True
53,PRI_00071639,True
61,PRI_A1792009,True
68,PRI_A0770343,True
86,PRI_A2100388,True
143,PRI_BB060167,True
159,PRI_A9500732,True
185,PRI_BB180318,True



Saved debug:
 - ../data/processed/notebook04/debug/ib_to_pss_ambiguous_name_only.csv (rows=2)
 - ../data/processed/notebook04/debug/ib_to_pss_unmatched_name_only.csv (rows=214)
=== 03.7.3b END ===


**Result (03.7.3b): IB (CA) → PSS (CA) using uniqueness-gated name-only matching**
- IB schools (unique): **230**
- Raw name-only matches: **16 (6.96%)**
- High-confidence matches (unique in PSS): **14 (6.09%)**
- Ambiguous dropped: **2**
- Unmatched: **214**

Saved:
- `../data/processed/notebook04/enrichment_flags/has_ib_pss.csv` (rows = 14)
Debug:
- `../data/processed/notebook04/debug/ib_to_pss_ambiguous_name_only.csv` (rows = 2)
- `../data/processed/notebook04/debug/ib_to_pss_unmatched_name_only.csv` (rows = 214)

**Why low match is acceptable:** The IB CSV lacks city/address, so we intentionally trade recall for precision by only accepting statewide-unique private-school names in PSS. This prevents incorrect joins.

## 03.8 Merge Enrichment Flags into Backbones (CCD + PSS)

**Goal:** Attach enrichment “tags” (boolean flags) to each school in our two backbones:
- **CCD** = public schools (`PUB_...`)
- **PSS** = private schools (`PRI_...`)

**Why now:** Up to this point we generated separate flag files (e.g., `has_cais_pss.csv`, `has_ib_ccd.csv`).  
In this step we *merge* them onto the backbone tables so every school row carries its feature presence.

**Inputs**
- Backbones:
  - `ccd_clean_df` (from 03.1/03.2)
  - `pss_clean_df` (from 03.5)
- Flag files (from 03.6–03.7):
  - CCD side: `has_crdc_ccd` (in-memory or saved), `has_ib_ccd.csv`
  - PSS side: `has_cais_pss.csv`, `has_ams_montessori_pss.csv`, `has_waldorf_pss.csv`, `has_ib_pss.csv`

**Outputs (saved to disk)**
- `../data/processed/ccd_backbone_with_flags.csv`
- `../data/processed/pss_backbone_with_flags.csv`
- (optional union) `../data/processed/backbone_master_with_flags.csv`

**Notes**
- All merges are **left joins** (we never drop schools from the backbones).
- Missing flags default to **False**.
- CCD and PSS remain separate because their IDs are different (`PUB_` vs `PRI_`), but we also create an optional union for convenience.


In [828]:
## 03.8 Merge Enrichment Flags into Backbones (CCD + PSS)

from pathlib import Path

print("=== 03.8 MERGE FLAGS INTO BACKBONES START ===")

# ---------------------------------------------------------
# 0) Preconditions + Paths
# ---------------------------------------------------------
assert "ccd_clean_df" in globals(), "ccd_clean_df is not defined. Run CCD standardization (03.1/03.2) first."
assert "pss_clean_df" in globals(), "pss_clean_df is not defined. Run PSS standardization (03.5) first."

# PROCESSED_DIR = globals().get("PROCESSED_DIR", Path("../data/processed"))
FLAGS_DIR = PROCESSED_DIR_notebook04 / "enrichment_flags"
FLAGS_DIR.mkdir(parents=True, exist_ok=True)

print("PROCESSED_DIR:", PROCESSED_DIR)
print("FLAGS_DIR:", FLAGS_DIR)

def load_flag_csv(path: Path, id_col: str = "school_id") -> pd.DataFrame:
    """
    Load a flag CSV like [school_id, has_x] and validate shape.
    """
    if not path.exists():
        print(f"Missing flag file: {path} (will skip)")
        return pd.DataFrame(columns=[id_col])
    df = pd.read_csv(path)
    assert id_col in df.columns, f"{path.name} missing required id column: {id_col}"
    return df

# Patch: avoid FutureWarning when filling NaN on object dtype
def merge_flags(base: pd.DataFrame, flags: list[tuple[str, pd.DataFrame]], id_col: str = "school_id") -> pd.DataFrame:
    """
    Left-join multiple flag tables onto base, then fill NaN -> False for flag columns.
    flags: list of (flag_col_name, df_with_flag_col)
    """
    out = base.copy()

    for flag_col, fdf in flags:
        # default column if missing
        if fdf is None or fdf.empty or flag_col not in fdf.columns:
            out[flag_col] = False
            continue

        fmin = fdf[[id_col, flag_col]].drop_duplicates(subset=[id_col])
        out = out.merge(fmin, on=id_col, how="left")

        # robust boolean normalization (no silent downcasting)
        out[flag_col] = (
            out[flag_col]
            .astype("boolean")      # pandas nullable boolean
            .fillna(False)
            .astype(bool)           # plain python bool dtype
        )

    return out

# ---------------------------------------------------------
# 1) CCD Backbone + Flags (Public)
#   - has_crdc is a "backbone feature presence" flag
#   - has_ib (CCD) comes from 03.7.3a output file
# ---------------------------------------------------------
print("\n--- 03.8.1 CCD Backbone + Flags ---")

ccd_base = ccd_clean_df.copy()
ccd_base["is_public"] = True
ccd_base["is_private"] = False

# Load IB→CCD flag
has_ib_ccd_path = FLAGS_DIR / "has_ib_ccd.csv"
ib_ccd_flag = load_flag_csv(has_ib_ccd_path)

# Build/Load CRDC presence flag
# If you already created crdc_presence_df earlier, we use it.
# Otherwise we attempt to load a saved file if you have one.
crdc_flag_df = None
if "crdc_presence_df" in globals():
    crdc_flag_df = crdc_presence_df.copy()
elif (FLAGS_DIR / "has_crdc_ccd.csv").exists():
    crdc_flag_df = load_flag_csv(FLAGS_DIR / "has_crdc_ccd.csv")
else:
    # Fallback: create empty (no CRDC flag) to avoid breaking the notebook
    crdc_flag_df = pd.DataFrame({"school_id": [], "has_crdc": []})
    print("has_crdc not found in memory or disk; creating empty has_crdc (all False).")

# Ensure the CRDC flag column is named has_crdc (if it exists)
if "has_crdc" not in crdc_flag_df.columns:
    # sometimes we had has_crdc named differently — try to infer
    possible = [c for c in crdc_flag_df.columns if c.startswith("has_")]
    if len(possible) == 1:
        crdc_flag_df = crdc_flag_df.rename(columns={possible[0]: "has_crdc"})
    else:
        crdc_flag_df["has_crdc"] = True  # if it has only school_id rows

ccd_with_flags = merge_flags(
    ccd_base,
    flags=[
        ("has_crdc", crdc_flag_df),
        ("has_ib", ib_ccd_flag),
    ],
    id_col="school_id"
)

print("CCD backbone w/ flags shape:", ccd_with_flags.shape)
print("CCD flags coverage:")
for col in ["has_crdc", "has_ib"]:
    if col in ccd_with_flags.columns:
        print(f" - {col}: {ccd_with_flags[col].sum():,} True ({ccd_with_flags[col].mean():.2%})")

CCD_OUT_PATH = PROCESSED_DIR_notebook04 / "ccd_backbone_with_flags.csv"
ccd_with_flags.to_csv(CCD_OUT_PATH, index=False)
print("Saved:", CCD_OUT_PATH)

display(ccd_with_flags.head(5))

# ---------------------------------------------------------
# 2) PSS Backbone + Flags (Private)
#   - CAIS, AMS, Waldorf, IB flags are keyed on PRI_* school_id
# ---------------------------------------------------------
print("\n--- 03.8.2 PSS Backbone + Flags ---")

pss_base = pss_clean_df.copy()
pss_base["is_public"] = False
pss_base["is_private"] = True

# Load PSS-side enrichment flags
cais_pss_flag = load_flag_csv(FLAGS_DIR / "has_cais_pss.csv")
ams_pss_flag  = load_flag_csv(FLAGS_DIR / "has_ams_montessori_pss.csv")
wald_pss_flag = load_flag_csv(FLAGS_DIR / "has_waldorf_pss.csv")
ib_pss_flag   = load_flag_csv(FLAGS_DIR / "has_ib_pss.csv")

pss_with_flags = merge_flags(
    pss_base,
    flags=[
        ("has_cais", cais_pss_flag),
        ("has_ams_montessori", ams_pss_flag),
        ("has_waldorf", wald_pss_flag),
        ("has_ib", ib_pss_flag),
    ],
    id_col="school_id"
)

print("PSS backbone w/ flags shape:", pss_with_flags.shape)
print("PSS flags coverage:")
for col in ["has_cais", "has_ams_montessori", "has_waldorf", "has_ib"]:
    if col in pss_with_flags.columns:
        print(f" - {col}: {pss_with_flags[col].sum():,} True ({pss_with_flags[col].mean():.2%})")

PSS_OUT_PATH = PROCESSED_DIR_notebook04 / "pss_backbone_with_flags.csv"
pss_with_flags.to_csv(PSS_OUT_PATH, index=False)
print("Saved:", PSS_OUT_PATH)

display(pss_with_flags.head(5))

# ---------------------------------------------------------
# 3) Optional: Union into one "backbone_master" table
# ---------------------------------------------------------
print("\n--- 03.8.3 Optional union backbone_master ---")

# Align columns (union-friendly): keep core schema + all known flag columns
core_cols = ["school_id", "school_name", "city", "state", "zip", "address", "join_key", "is_public", "is_private"]
flag_cols = sorted(list(set(
    [c for c in ccd_with_flags.columns if c.startswith("has_")] +
    [c for c in pss_with_flags.columns if c.startswith("has_")]
)))

# Ensure both frames have all columns
def ensure_cols(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    out = df.copy()
    for c in cols:
        if c not in out.columns:
            out[c] = False if c.startswith("has_") else np.nan
    return out[cols]

union_cols = core_cols + flag_cols

ccd_union = ensure_cols(ccd_with_flags, union_cols)
pss_union = ensure_cols(pss_with_flags, union_cols)

backbone_master = pd.concat([ccd_union, pss_union], ignore_index=True)
print("backbone_master shape:", backbone_master.shape)

BACKBONE_MASTER_PATH = PROCESSED_DIR_notebook04 / "backbone_master_with_flags.csv"
backbone_master.to_csv(BACKBONE_MASTER_PATH, index=False)
print("Saved:", BACKBONE_MASTER_PATH)

display(backbone_master.head(5))

print("=== 03.8 MERGE FLAGS INTO BACKBONES END ===")

=== 03.8 MERGE FLAGS INTO BACKBONES START ===
PROCESSED_DIR: ../data/processed
FLAGS_DIR: ../data/processed/notebook04/enrichment_flags

--- 03.8.1 CCD Backbone + Flags ---
CCD backbone w/ flags shape: (102274, 16)
CCD flags coverage:
 - has_crdc: 76,687 True (74.98%)
 - has_ib: 19 True (0.02%)
Saved: ../data/processed/notebook04/ccd_backbone_with_flags.csv


,school_id,ncessch,schid_ccd,school_name,city,state,zip,address,ccd_school_type,ccd_charter_text,join_key,has_ccd,is_public,is_private,has_crdc,has_ib
0,PUB_10000500870,10000500870,100870,albertville middle school,albertville,AL,35950,600 e alabama ave,Regular School,No,albertville middle school|albertville|AL,True,True,False,False,False
1,PUB_10000500871,10000500871,100871,albertville high school,albertville,AL,35950,402 e mccord ave,Regular School,No,albertville high school|albertville|AL,True,True,False,False,False
2,PUB_10000500879,10000500879,100879,albertville intermediate school,albertville,AL,35950,901 w mckinney ave,Regular School,No,albertville intermediate school|albertville|AL,True,True,False,False,False
3,PUB_10000500889,10000500889,100889,albertville elementary school,albertville,AL,35950,145 west end drive,Regular School,No,albertville elementary school|albertville|AL,True,True,False,False,False
4,PUB_10000501616,10000501616,101616,albertville kindergarten and prek,albertville,AL,35951,257 country club rd,Regular School,No,albertville kindergarten and prek|albertville|AL,True,True,False,False,False



--- 03.8.2 PSS Backbone + Flags ---
PSS backbone w/ flags shape: (22345, 17)
PSS flags coverage:
 - has_cais: 71 True (0.32%)
 - has_ams_montessori: 6 True (0.03%)
 - has_waldorf: 15 True (0.07%)
 - has_ib: 14 True (0.06%)
Saved: ../data/processed/notebook04/pss_backbone_with_flags.csv


,school_id,ppin,school_name,city,state,zip,zip4,address,join_key,backbone_source,has_pss,is_public,is_private,has_cais,has_ams_montessori,has_waldorf,has_ib
0,PRI_00000033,00000033,st james catholic school,gadsden,AL,35901,2564.0,700 albert rains blvd,st james catholic school|gadsden|AL,PSS_2122,True,False,True,False,False,False,False
1,PRI_00000044,00000044,holy spirit catholic school,tuscaloosa,AL,35405,3208.0,601 james i harrison jr pkwy e,holy spirit catholic school|tuscaloosa|AL,PSS_2122,True,False,True,False,False,False,False
2,PRI_00000055,00000055,holy family parochial school,huntsville,AL,35816,4004.0,2300 beasley ave nw,holy family parochial school|huntsville|AL,PSS_2122,True,False,True,False,False,False,False
3,PRI_00000077,00000077,holy spirit regional catholic school,huntsville,AL,35802,1310.0,619 airport rd sw,holy spirit regional catholic school|huntsvill...,PSS_2122,True,False,True,False,False,False,False
4,PRI_00000088,00000088,our lady of sorrows,birmingham,AL,35209,4097.0,1720 oxmoor rd,our lady of sorrows|birmingham|AL,PSS_2122,True,False,True,False,False,False,False



--- 03.8.3 Optional union backbone_master ---
backbone_master shape: (124619, 16)
Saved: ../data/processed/notebook04/backbone_master_with_flags.csv


,school_id,school_name,city,state,zip,address,join_key,is_public,is_private,has_ams_montessori,has_cais,has_ccd,has_crdc,has_ib,has_pss,has_waldorf
0,PUB_10000500870,albertville middle school,albertville,AL,35950,600 e alabama ave,albertville middle school|albertville|AL,True,False,False,False,True,False,False,False,False
1,PUB_10000500871,albertville high school,albertville,AL,35950,402 e mccord ave,albertville high school|albertville|AL,True,False,False,False,True,False,False,False,False
2,PUB_10000500879,albertville intermediate school,albertville,AL,35950,901 w mckinney ave,albertville intermediate school|albertville|AL,True,False,False,False,True,False,False,False,False
3,PUB_10000500889,albertville elementary school,albertville,AL,35950,145 west end drive,albertville elementary school|albertville|AL,True,False,False,False,True,False,False,False,False
4,PUB_10000501616,albertville kindergarten and prek,albertville,AL,35951,257 country club rd,albertville kindergarten and prek|albertville|AL,True,False,False,False,True,False,False,False,False


=== 03.8 MERGE FLAGS INTO BACKBONES END ===


In [830]:
print("=== CHECK: backbone_master has_cais for Harker ===")
display(
    backbone_master[backbone_master["school_id"] == "PRI_A2190096"][
        ["school_id","school_name","city","state","has_cais","is_private","is_public"]
    ]
)


=== CHECK: backbone_master has_cais for Harker ===


,school_id,school_name,city,state,has_cais,is_private,is_public
119411,PRI_A2190096,harker,san jose,CA,True,True,False


## 03.9 Assemble & Persist `schools_master_v1`

This section assembles the final Golden Record by merging the
standardized public and private school backbones with validated
enrichment flags.

The result is a single canonical dataset used as the source of truth
for all downstream feature engineering and matching.



In [833]:
## 03.9.1 Assemble & Persist schools_master_v1 (Golden Record only)

print("=== 03.9.1 START (ASSEMBLE SCHOOLS_MASTER_V1) ===")

from pathlib import Path
import pandas as pd

BACKBONE_MASTER_PATH = PROCESSED_DIR_notebook04 / "backbone_master_with_flags.csv"
assert BACKBONE_MASTER_PATH.exists(), f"Missing: {BACKBONE_MASTER_PATH}. Run 03.8 first."

schools_master_df = pd.read_csv(BACKBONE_MASTER_PATH)

print("Loaded backbone_master_with_flags.csv")
print("Shape:", schools_master_df.shape)

# Required columns checks
required_cols = ["school_id", "school_name", "city", "state"]
missing_required = [c for c in required_cols if c not in schools_master_df.columns]
assert not missing_required, f"Missing required columns: {missing_required}"

# Normalize key strings
for c in ["school_id", "school_name", "city", "state"]:
    schools_master_df[c] = schools_master_df[c].astype(str).str.strip()

# Ensure flag columns exist and are bool
flag_cols = [
    "has_cais",
    "has_ams_montessori",
    "has_waldorf",
    "has_ib",
    "has_crdc",
    "is_public",
    "is_private",
]
for c in flag_cols:
    if c not in schools_master_df.columns:
        schools_master_df[c] = False
    schools_master_df[c] = schools_master_df[c].fillna(False).astype(bool)

# Data quality gates
dup = schools_master_df["school_id"].duplicated().sum()
assert dup == 0, f"school_id is not unique. Duplicates: {dup}"
assert len(schools_master_df) > 0, "schools_master_df is empty."

# Persist (CSV only)
OUT_CSV = PROCESSED_DIR_notebook04 / "schools_master_v1.csv"
schools_master_df.to_csv(OUT_CSV, index=False)
print("Saved:", OUT_CSV)

display(schools_master_df.head(5))

print("=== 03.9.1 END ===")



=== 03.9.1 START (ASSEMBLE SCHOOLS_MASTER_V1) ===
Loaded backbone_master_with_flags.csv
Shape: (124619, 16)
Saved: ../data/processed/notebook04/schools_master_v1.csv


,school_id,school_name,city,state,zip,address,join_key,is_public,is_private,has_ams_montessori,has_cais,has_ccd,has_crdc,has_ib,has_pss,has_waldorf
0,PUB_10000500870,albertville middle school,albertville,AL,35950,600 e alabama ave,albertville middle school|albertville|AL,True,False,False,False,True,False,False,False,False
1,PUB_10000500871,albertville high school,albertville,AL,35950,402 e mccord ave,albertville high school|albertville|AL,True,False,False,False,True,False,False,False,False
2,PUB_10000500879,albertville intermediate school,albertville,AL,35950,901 w mckinney ave,albertville intermediate school|albertville|AL,True,False,False,False,True,False,False,False,False
3,PUB_10000500889,albertville elementary school,albertville,AL,35950,145 west end drive,albertville elementary school|albertville|AL,True,False,False,False,True,False,False,False,False
4,PUB_10000501616,albertville kindergarten and prek,albertville,AL,35951,257 country club rd,albertville kindergarten and prek|albertville|AL,True,False,False,False,True,False,False,False,False


=== 03.9.1 END ===


In [835]:
print("=== CHECK: reloaded backbone_master_with_flags.csv has_cais for Harker ===")
bm = pd.read_csv(PROCESSED_DIR_notebook04 / "backbone_master_with_flags.csv")
display(
    bm[bm["school_id"] == "PRI_A2190096"][
        ["school_id","school_name","city","state","has_cais","is_private","is_public"]
    ]
)
print("bm has_cais dtype:", bm["has_cais"].dtype)


=== CHECK: reloaded backbone_master_with_flags.csv has_cais for Harker ===


,school_id,school_name,city,state,has_cais,is_private,is_public
119411,PRI_A2190096,harker,san jose,CA,True,True,False


bm has_cais dtype: bool


In [837]:
print("=== CHECK: schools_master_v1.csv has_cais for Harker ===")
sm = pd.read_csv(PROCESSED_DIR_notebook04 / "schools_master_v1.csv")
display(
    sm[sm["school_id"] == "PRI_A2190096"][
        ["school_id","school_name","city","state","has_cais","is_private","is_public"]
    ]
)
print("sm has_cais dtype:", sm["has_cais"].dtype)


=== CHECK: schools_master_v1.csv has_cais for Harker ===


,school_id,school_name,city,state,has_cais,is_private,is_public
119411,PRI_A2190096,harker,san jose,CA,True,True,False


sm has_cais dtype: bool


## 03.9.2 Merge Summary & Data Quality Audit

This section produces a lightweight summary and audit of the final
`schools_master_v1` Golden Record.

The goal is to validate:
- overall row counts and public vs. private distribution
- coverage of enrichment flags (CAIS, Montessori, Waldorf, IB, CRDC)
- basic missingness of key identifying fields

### Artifacts Produced
- `../reports/notebook04_merge_report.md`  
  Human-readable summary of dataset composition and enrichment coverage.
- `../reports/notebook04_merge_audit.csv`  
  Machine-readable audit table with flag prevalence statistics.

These artifacts provide transparency into data quality and serve as
documentation for downstream feature engineering and matching notebooks.


In [840]:
## 03.9.2 Merge Summary + Audit Report

print("=== 03.9.2 START (SUMMARY + AUDIT) ===")

PROCESSED_DIR = globals().get("PROCESSED_DIR", Path("../data/processed/notebook04"))
REPORTS_DIR = globals().get("REPORTS_DIR", Path("../reports"))
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

MERGE_REPORT_PATH = REPORTS_DIR / "notebook04_merge_report.md"
MERGE_AUDIT_PATH = REPORTS_DIR / "notebook04_merge_audit.csv"

df = schools_master_df.copy()

# Core counts
total = len(df)
n_public = int(df["is_public"].sum()) if "is_public" in df.columns else 0
n_private = int(df["is_private"].sum()) if "is_private" in df.columns else 0

# Tag prevalence
tag_cols = [c for c in df.columns if c.startswith("has_")]
tag_stats = []
for c in tag_cols:
    if df[c].dtype != bool:
        # be defensive: coerce
        s = df[c].fillna(False).astype(bool)
    else:
        s = df[c]
    tag_stats.append({
        "flag": c,
        "true_count": int(s.sum()),
        "true_rate": float(s.mean()),
    })

tag_stats_df = pd.DataFrame(tag_stats).sort_values("true_rate", ascending=False)

# Missingness for key fields
key_fields = ["school_id", "school_name", "city", "state", "zip", "address"]
missing_df = pd.DataFrame({
    "field": [c for c in key_fields if c in df.columns],
    "missing_count": [(df[c].isna().sum()) for c in key_fields if c in df.columns],
    "missing_rate": [(df[c].isna().mean()) for c in key_fields if c in df.columns],
}).sort_values("missing_rate", ascending=False)

# Save audit CSV (machine-readable)
audit_out = tag_stats_df.copy()
audit_out.to_csv(MERGE_AUDIT_PATH, index=False)
print("Saved audit CSV:", MERGE_AUDIT_PATH)

# Save markdown report (human-readable)
lines = []
lines.append("# Notebook 04 Merge Report — schools_master_v1\n")
lines.append(f"- Total schools: **{total:,}**")
lines.append(f"- Public: **{n_public:,}**")
lines.append(f"- Private: **{n_private:,}**\n")

lines.append("## Flag Coverage\n")
lines.append(tag_stats_df.to_markdown(index=False))
lines.append("\n## Missingness (Key Fields)\n")
lines.append(missing_df.to_markdown(index=False))

MERGE_REPORT_PATH.write_text("\n".join(lines), encoding="utf-8")
print("Saved merge report:", MERGE_REPORT_PATH)

print("=== 03.9.2 END ===")


=== 03.9.2 START (SUMMARY + AUDIT) ===
Saved audit CSV: ../reports/notebook04_merge_audit.csv
Saved merge report: ../reports/notebook04_merge_report.md
=== 03.9.2 END ===
